In [151]:
import numpy as np
import os
import pandas as pd
from arcpy import env
from arcpy import TableToTable_conversion
import matplotlib.ticker as mtick

In [152]:
dirWork = os.getcwd()
dirData = os.path.join(dirWork, r'data')
dirTDM  = os.path.join(dirData, r'tdm')
dirFactors   = os.path.join(dirWork, r'data-processed\factors')
dirProcessed = os.path.join(dirWork, r'data-processed\tdm')
dirResults   = os.path.join(dirWork, r'results')

# Run Settings

In [153]:
#set to true for first time to create CSVs, false to speed up for subsequent times
process_TDMdbftocsv = False

# Define Scenarios

In [154]:
df_Scenarios = pd.DataFrame()


#define AdjCode for pivot when model is not related to forecast
#AADT will be adjusted based on difference between scenario and forecast scenario TDMs
#if ForecastScenarioID is note defined, then the associated forecast volume will be used for that year
#DBF don't include filename extension

df_Scenarios = df_Scenarios.append({'ScenarioCode' : 'Base19',
                                    'ScenarioName' : 'Base',
                                    'ScenarioYear' :  2019,
                                    'SegSummaryDBF': 'v831_SE19_Net19_Summary_SEGID_Detailed_BH',
                                    'Process'      : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode' : 'Base24',
                                    'ScenarioName' : 'STIP',
                                    'ScenarioYear' :  2024,
                                    'SegSummaryDBF': 'v831_SE24_Net24_Summary_SEGID_Detailed_BH',
                                    'Process'      : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode' : 'RTP30',
                                    'ScenarioName' : '2030 RTP',
                                    'ScenarioYear' :  2030,
                                    'SegSummaryDBF': 'v831_SE30_Net30_Summary_SEGID_Detailed_BH',
                                    'Process'      : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode' : 'RTP40',
                                    'ScenarioName' : '2040 RTP',
                                    'ScenarioYear' :  2040,
                                    'SegSummaryDBF': 'v831_SE40_Net40_Summary_SEGID_Detailed_BH',
                                    'Process'      : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode' : 'RTP50',
                                    'ScenarioName' : '2050 RTP',
                                    'ScenarioYear' :  2050,
                                    'SegSummaryDBF': 'v831_SE50_Net50_Summary_SEGID_Detailed_BH',
                                    'Process'      : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB30',
                                    'ScenarioName'   : '2030 No Build',
                                    'ScenarioYear'   :  2030,
                                    'AdjScenarioCode': 'RTP30',
                                    'SegSummaryDBF'  : 'v831_SE30_Net24_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB40',
                                    'ScenarioName'   : '2040 No Build',
                                    'ScenarioYear'   :  2040,
                                    'AdjScenarioCode': 'RTP40',
                                    'SegSummaryDBF'  : 'v831_SE40_Net24_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'NB50',
                                    'ScenarioName'   : '2050 No Build',
                                    'ScenarioYear'   :  2050,
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryDBF'  : 'v831_SE50_Net24_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF30',
                                    'ScenarioName'   : '2030 Unfunded',
                                    'ScenarioYear'   :  2030,
                                    'AdjScenarioCode': 'RTP30',
                                    'SegSummaryDBF'  : 'v831_SE30_Net30UF_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF40',
                                    'ScenarioName'   : '2040 Unfunded',
                                    'ScenarioYear'   :  2040,
                                    'AdjScenarioCode': 'RTP40',
                                    'SegSummaryDBF'  : 'v831_SE40_Net40UF_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes'
                                   },ignore_index=True)

df_Scenarios = df_Scenarios.append({'ScenarioCode'   : 'UF50',
                                    'ScenarioName'   : '2050 Unfunded',
                                    'ScenarioYear'   :  2050,
                                    'AdjScenarioCode': 'RTP50',
                                    'SegSummaryDBF'  : 'v831_SE50_Net50UF_Summary_SEGID_Detailed_BH',
                                    'Process'        : 'Yes'
                                   },ignore_index=True)

#all scenarios will be read in to allow for pivoting if needed
df_Scenarios.index.name = 'ScenarioID'
df_Scenarios['ScenarioYear'] = pd.to_numeric(df_Scenarios['ScenarioYear'], downcast='integer')
df_Scenarios

,Process,ScenarioCode,ScenarioName,ScenarioYear,SegSummaryDBF,AdjScenarioCode
ScenarioID,,,,,,
0,Yes,Base19,Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed_BH,NaN
1,Yes,Base24,STIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,NaN
2,Yes,RTP30,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,NaN
3,Yes,RTP40,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed_BH,NaN
4,Yes,RTP50,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed_BH,NaN
5,Yes,NB30,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,RTP30
6,Yes,NB40,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,RTP40
7,Yes,NB50,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,RTP50
8,Yes,UF30,2030 Unfunded,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,RTP30


In [155]:
df_Scenarios_toProcess = df_Scenarios[df_Scenarios['Process']=='Yes']
df_Scenarios_toProcess

,Process,ScenarioCode,ScenarioName,ScenarioYear,SegSummaryDBF,AdjScenarioCode
ScenarioID,,,,,,
0,Yes,Base19,Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed_BH,NaN
1,Yes,Base24,STIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,NaN
2,Yes,RTP30,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,NaN
3,Yes,RTP40,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed_BH,NaN
4,Yes,RTP50,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed_BH,NaN
5,Yes,NB30,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,RTP30
6,Yes,NB40,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,RTP40
7,Yes,NB50,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,RTP50
8,Yes,UF30,2030 Unfunded,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,RTP30


In [156]:
df_Scenarios_toAdjust = df_Scenarios_toProcess[df_Scenarios_toProcess['AdjScenarioCode'].notnull()]
df_Scenarios_toAdjust

,Process,ScenarioCode,ScenarioName,ScenarioYear,SegSummaryDBF,AdjScenarioCode
ScenarioID,,,,,,
5,Yes,NB30,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,RTP30
6,Yes,NB40,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,RTP40
7,Yes,NB50,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,RTP50
8,Yes,UF30,2030 Unfunded,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,RTP30
9,Yes,UF40,2040 Unfunded,2040,v831_SE40_Net40UF_Summary_SEGID_Detailed_BH,RTP40
10,Yes,UF50,2050 Unfunded,2050,v831_SE50_Net50UF_Summary_SEGID_Detailed_BH,RTP50


In [157]:
#define functions to get scenario data
def idFromCode (sCode):
    return df_Scenarios.index[df_Scenarios['ScenarioCode'] == sCode].tolist()[0]
def yrFromCode (sCode):
    return df_Scenarios[df_Scenarios['ScenarioCode'] == sCode].iloc[0]['ScenarioYear']
def codeFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioCode']
def yearFromID (sID):
    return df_Scenarios.loc[sID]['ScenarioYear']

#quick tests of functions
#display(idFromCode('50RTP'))
#display(yrFromCode('50RTP'))
#display(codeFromID(4))
#display(yearFromID(4))
#display(df_Scenarios.loc[7]['ScenarioCode'])

# Define Desired V/C Groupings

In [158]:
#season groups

df_VCGroup_Seasons = pd.DataFrame()

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Win',
                                                'SeasonName'    : 'Winter',
                                                'SeasonGroup'   : 'S01-Win'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Spr',
                                                'SeasonName'    : 'Spring',
                                                'SeasonGroup'   : 'S01-Win'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Sum',
                                                'SeasonName'    : 'Summer',
                                                'SeasonGroup'   : 'S01-Win'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Fal',
                                                'SeasonName'    : 'Fall',
                                                'SeasonGroup'   : 'S01-Win'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jan',
                                                'SeasonName'    : 'January',
                                                'SeasonGroup'   : 'M01-Jan'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Feb',
                                                'SeasonName'    : 'February',
                                                'SeasonGroup'   : 'M02-Feb'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Mar',
                                                'SeasonName'    : 'March',
                                                'SeasonGroup'   : 'M03-Mar'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Apr',
                                                'SeasonName'    : 'April',
                                                'SeasonGroup'   : 'M04-Apr'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'May',
                                                'SeasonName'    : 'May',
                                                'SeasonGroup'   : 'M05-May'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jun',
                                                'SeasonName'    : 'June',
                                                'SeasonGroup'   : 'M06-Jun'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Jul',
                                                'SeasonName'    : 'July',
                                                'SeasonGroup'   : 'M07-Jul'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Aug',
                                                'SeasonName'    : 'August',
                                                'SeasonGroup'   : 'M08-Aug'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Sep',
                                                'SeasonName'    : 'September',
                                                'SeasonGroup'   : 'M09-Sep'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Oct',
                                                'SeasonName'    : 'October',
                                                'SeasonGroup'   : 'M10-Oct'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Nov',
                                                'SeasonName'    : 'November',
                                                'SeasonGroup'   : 'M11-Nov'
                                               },ignore_index=True)

df_VCGroup_Seasons = df_VCGroup_Seasons.append({'SeasonCode'    : 'Dec',
                                                'SeasonName'    : 'December',
                                                'SeasonGroup'   : 'M12-Dec'
                                               },ignore_index=True)

display(df_VCGroup_Seasons)

#day-of-week groups

df_VCGroup_DOWPeak = pd.DataFrame()

df_VCGroup_DOWPeak = df_VCGroup_DOWPeak.append({'DOWPkCode'     : 'WkAM',
                                                'DOWPkName'     : 'Weekday AM Peak',
                                                'DOWGroup'      : 'W1-Weekday',
                                                'Period'        : 'AM',
                                                'HourPercentOf' : 'Period' #compare to period volume from model
                                               },ignore_index=True)

df_VCGroup_DOWPeak = df_VCGroup_DOWPeak.append({'DOWPkCode'     : 'WkPM',
                                                'DOWPkName'     : 'Weekday AM Peak',
                                                'DOWGroup'      : 'W1-Weekday',
                                                'Period'        : 'PM',
                                                'HourPercentOf' : 'Period' #compare to period volume from model
                                               },ignore_index=True)

df_VCGroup_DOWPeak = df_VCGroup_DOWPeak.append({'DOWPkCode'     : 'FrPM',
                                                'DOWPkName'     : 'Friday PM Peak',
                                                'DOWGroup'      : '5-Friday',
                                                'Period'        : 'PM',
                                                'HourPercentOf' : 'Daily' #compare to period volume from model
                                               },ignore_index=True)

df_VCGroup_DOWPeak = df_VCGroup_DOWPeak.append({'DOWPkCode'     : 'SaMD',
                                                'DOWPkName'     : 'Saturday Midday Peak',
                                                'DOWGroup'      : '6-Saturday',
                                                'Period'        : 'MD',
                                                'HourPercentOf' : 'Daily' #compare to daily volume from model
                                               },ignore_index=True)

df_VCGroup_DOWPeak = df_VCGroup_DOWPeak.append({'DOWPkCode'     : 'SuMD',
                                                'DOWPkName'     : 'Sunday Midday Peak',
                                                'DOWGroup'      : '7-Sunday',
                                                'Period'        : 'MD',
                                                'HourPercentOf' : 'Daily' #compare to daily volume from model
                                               },ignore_index=True)

display(df_VCGroup_DOWPeak)

#directional factor when using daily volumes for peak hour calcs, period volume calcs already include directionality
#taken from HCM 2000 Exhibit 9-2
dDailyDirectionalFactor = 0.6

,SeasonCode,SeasonGroup,SeasonName
0,Win,S01-Win,Winter
1,Spr,S01-Win,Spring
2,Sum,S01-Win,Summer
3,Fal,S01-Win,Fall
4,Jan,M01-Jan,January
5,Feb,M02-Feb,February
6,Mar,M03-Mar,March
7,Apr,M04-Apr,April
8,May,M05-May,May
9,Jun,M06-Jun,June


,DOWGroup,DOWPkCode,DOWPkName,HourPercentOf,Period
0,W1-Weekday,WkAM,Weekday AM Peak,Period,AM
1,W1-Weekday,WkPM,Weekday AM Peak,Period,PM
2,5-Friday,FrPM,Friday PM Peak,Daily,PM
3,6-Saturday,SaMD,Saturday Midday Peak,Daily,MD
4,7-Sunday,SuMD,Sunday Midday Peak,Daily,MD


In [159]:
import itertools

#create df with all possible season and day-of-weak/peak groups
df_VCGroups = pd.DataFrame(list(itertools.product(df_VCGroup_Seasons['SeasonCode'],df_VCGroup_DOWPeak['DOWPkCode'])),columns=['SeasonCode','DOWPkCode'])

df_VCGroups = pd.DataFrame(df_VCGroups)
pd.set_option('display.max_rows', df_VCGroups.shape[0]+1)

#add season data
df_VCGroups = pd.DataFrame.merge(df_VCGroups, df_VCGroup_Seasons, on='SeasonCode')

#add dow-peak data
df_VCGroups = pd.DataFrame.merge(df_VCGroups, df_VCGroup_DOWPeak, on='DOWPkCode')

#set id name
df_VCGroups.index.name = 'VCGroupID'

df_VCGroups['VCGroupCode'] = df_VCGroups['SeasonCode'] + df_VCGroups['DOWPkCode']

display(df_VCGroups)

,SeasonCode,DOWPkCode,SeasonGroup,SeasonName,DOWGroup,DOWPkName,HourPercentOf,Period,VCGroupCode
VCGroupID,,,,,,,,,
0,Win,WkAM,S01-Win,Winter,W1-Weekday,Weekday AM Peak,Period,AM,WinWkAM
1,Spr,WkAM,S01-Win,Spring,W1-Weekday,Weekday AM Peak,Period,AM,SprWkAM
2,Sum,WkAM,S01-Win,Summer,W1-Weekday,Weekday AM Peak,Period,AM,SumWkAM
3,Fal,WkAM,S01-Win,Fall,W1-Weekday,Weekday AM Peak,Period,AM,FalWkAM
4,Jan,WkAM,M01-Jan,January,W1-Weekday,Weekday AM Peak,Period,AM,JanWkAM
5,Feb,WkAM,M02-Feb,February,W1-Weekday,Weekday AM Peak,Period,AM,FebWkAM
6,Mar,WkAM,M03-Mar,March,W1-Weekday,Weekday AM Peak,Period,AM,MarWkAM
7,Apr,WkAM,M04-Apr,April,W1-Weekday,Weekday AM Peak,Period,AM,AprWkAM
8,May,WkAM,M05-May,May,W1-Weekday,Weekday AM Peak,Period,AM,MayWkAM


# Convert DBFs to CSVs

In [160]:
# Create function that converts dbfs to csvs using arcpy
def dbfToCSV(dbf, csv):
    TableToTable_conversion(dbf, os.path.dirname(csv), os.path.basename(csv))

# Allow arcpy to overwrite output files
env.overwriteOutput = True

#convert DBFs to CSVs
if process_TDMdbftocsv:
    
    for (idx, row) in df_Scenarios.iterrows():    

        print(row.loc[nameS] + ' TDM DBF converting to CSV')

        dbfFileName = os.path.join(dirTDM, row.SegSummaryDBF + ".dbf")
        csvFileName = os.path.join(dirProcessed, row.SegSummaryDBF + ".csv")

        dbfToCSV(dbfFileName, csvFileName)
        
        print(row.loc[nameS] + ' TDM done.')
else:
    print("Don't convert TDM Segment Summaries DBFs.")

print('Done')
        

Don't convert TDM Segment Summaries DBFs.
Done


# TDM SEGID Detail

In [161]:
#column names from TDM
nameSegID   = 'SEGID'
nameFG      = 'FUNCGROUP'
nameLinks   = 'LINKS'
nameLanes   = 'LANES'
nameVol     = 'VOL'
nameCap1HL  = 'CAP1HL'
nameAT      = 'AREATYPE'

aNameDirPeriodVol = ['D1_AM_VOL','D1_PM_VOL','D1_MD_VOL','D1_EV_VOL','D1_AM_LT','D1_MD_LT','D1_PM_LT','D1_EV_LT','D1_AM_MD','D1_MD_MD','D1_PM_MD','D1_EV_MD','D1_AM_HV','D1_MD_HV','D1_PM_HV','D1_EV_HV','D2_AM_VOL','D2_PM_VOL','D2_MD_VOL','D2_EV_VOL','D2_AM_LT','D2_MD_LT','D2_PM_LT','D2_EV_LT','D2_AM_MD','D2_MD_MD','D2_PM_MD','D2_EV_MD','D2_AM_HV','D2_MD_HV','D2_PM_HV','D2_EV_HV']

#value for functional group (FG) total row, so that it can be removed
nameFGTotalValue = 'Total'

#column for scenario year
nameSID = 'ScenarioID'
nameS =   'ScenarioName'
nameSYr = 'ScenarioYear'

#additional column name variables
nameLT = 'LT'
nameMD = 'MD'
nameHV = 'HV'

nameVolType = 'VolType'
namePeriod  = 'Period'
nameCapPer  = 'CapPer'
nameCap1Hr  = 'Cap1Hr'
nameDir     = 'Dir'

nameSegTotal        = 'SegTotalDYVol'
namePercentSegTotal = 'VolPercentSegDY'
namePercentTrucksMD = 'PercentTrucksMD'
namePercentTrucksHV = 'PercentTrucksHV'

indexDirPeriodwSegData = [nameSegID,nameFG,nameCap1HL,nameLinks,nameLanes,nameDir,namePeriod,nameAT]
indexDirPeriod         = [nameSegID,nameDir,namePeriod]
indexDir               = [nameSegID,nameDir,nameAT]
indexTotal             = [nameSegID]
indexMelt              = [nameSegID,nameFG,nameAT,nameLinks,nameLanes,nameCap1HL]
indexDaily             = [nameSegID,nameFG,nameCap1HL,nameLinks,nameLanes,nameAT]

#hours within period, for capacity
df_PeriodDuration = {
     'Period'     : ['AM','MD','PM','EV','Daily'],
     'PeriodHours': [   3,   6,   3,  12,     24]
     
}
df_PeriodDuration = pd.DataFrame(df_PeriodDuration)

In [162]:
dfs_TDM_import = []
dfs_TDM =[]
dfs_TDM_Total = []

#all scenarios will be read in to allow for pivoting

for (idx, row) in df_Scenarios.iterrows():

    print(row.loc[nameS] + ' CSV Importing')

    csvFileName = os.path.join(dirProcessed, row.SegSummaryDBF + ".csv")
    #csvFileName = os.path.splitext(dbfFileName)[0]+'.csv'

    df_TDM_import = pd.read_csv(csvFileName)

    #Remove where exclude not 0
    df_TDM_import = df_TDM_import[df_TDM_import.LINKS > 0]
    df_TDM_import = df_TDM_import[df_TDM_import.FUNCGROUP != nameFGTotalValue]
    dfs_TDM_import.append(df_TDM_import)
        
    print(row.loc[nameS] + ' Processing.')

    #normalize data with period/volume type fields
    df_TDM_melt = pd.melt(df_TDM_import, id_vars=indexMelt, value_vars=aNameDirPeriodVol)

    #Calculate new columns for period, volume type
    df_TDM_melt[nameDir]     = df_TDM_melt['variable'].str.get(0) + df_TDM_melt['variable'].str.get(1)
    df_TDM_melt[namePeriod]  = df_TDM_melt['variable'].str.get(3) + df_TDM_melt['variable'].str.get(4)
    df_TDM_melt[nameVolType] = df_TDM_melt['variable'].str[6:]
    
    #display(df_TDM_melt[(df_TDM_melt[nameSegID]=='0015_295.6')])
    
    df_TDM_byDirPeriod = pd.pivot_table(df_TDM_melt,
        values='value',
        index=indexDirPeriodwSegData,
        columns=[nameVolType]
    )
    
    df_TDM_byDirPeriod.reset_index(level=df_TDM_byDirPeriod.index.names, inplace=True)
    #display(df_TDM_byDirPeriod)
    #display(df_TDM_byDirPeriod[(df_TDM_byDirPeriod[nameSegID]=='0015_295.6')])
    
    df_TDM_Total         = df_TDM_byDirPeriod.groupby(indexTotal).agg({nameVol: [np.sum]})
    df_TDM_Total.columns = [nameSegTotal]
    dfs_TDM_Total.append(df_TDM_Total) #add to collection for volume adjusting
    
    #display(df_TDM_Total)
    #display(df_TDM_Total[(df_TDM_Total.index=='0015_295.6')])

    #display(df_TDM_byDirPeriod)
    #display(df_TDM_byDirPeriod[(df_TDM_byDirPeriod[nameSegID]=='0015_295.6')])
    
    #calculate daily directional for peak-hour-percent-of-daily calcs
    df_TDM_Daily             = df_TDM_byDirPeriod.groupby(indexDaily, as_index=False).agg({nameVol: [np.sum], nameLT: [np.sum], nameMD: [np.sum], nameHV: [np.sum]})
    df_TDM_Daily.columns     = df_TDM_Daily.columns.droplevel(1)
    df_TDM_Daily[namePeriod] = 'Daily'
    
    df_TDM_Daily_D1          = df_TDM_Daily.copy()
    df_TDM_Daily_D2          = df_TDM_Daily.copy()
    
    df_TDM_Daily_D1[nameDir] = 'D1'
    df_TDM_Daily_D2[nameDir] = 'D2'
    
    df_TDM_Daily_D1[nameVol] = df_TDM_Daily[nameVol] * dDailyDirectionalFactor
    df_TDM_Daily_D2[nameVol] = df_TDM_Daily[nameVol] * (1-dDailyDirectionalFactor)

    df_TDM_Daily_D1[nameLT]  = df_TDM_Daily[nameLT] * dDailyDirectionalFactor
    df_TDM_Daily_D2[nameLT]  = df_TDM_Daily[nameLT] * (1-dDailyDirectionalFactor)
    
    df_TDM_Daily_D1[nameMD]  = df_TDM_Daily[nameMD] * dDailyDirectionalFactor
    df_TDM_Daily_D2[nameMD]  = df_TDM_Daily[nameMD] * (1-dDailyDirectionalFactor)
    
    df_TDM_Daily_D1[nameHV]  = df_TDM_Daily[nameHV] * dDailyDirectionalFactor
    df_TDM_Daily_D2[nameHV]  = df_TDM_Daily[nameHV] * (1-dDailyDirectionalFactor)
    
    #display(df_TDM_Daily)
    #display(df_TDM_Daily_D1)
    #display(df_TDM_Daily_D2)
    
    df_TDM_byDirPeriod = df_TDM_byDirPeriod.append(df_TDM_Daily_D1)
    df_TDM_byDirPeriod = df_TDM_byDirPeriod.append(df_TDM_Daily_D2)
    
    #display(df_TDM_byDirPeriod)
    #display(df_TDM_byDirPeriod[(df_TDM_byDirPeriod[nameSegID]=='0015_295.6')])
    #display(df_TDM_Total[(df_TDM_Total.index=='0015_295.6')])
    
    df_TDM_byDirPeriodwTotal = pd.DataFrame.merge(df_TDM_byDirPeriod,df_TDM_Total, on=nameSegID)
    #display(df_TDM_byDirPeriodwTotal)
    #display(df_TDM_byDirPeriodwTotal[(df_TDM_byDirPeriodwTotal[nameSegID]=='0015_295.6')])
    
    #calculate percent trucks
    df_TDM_byDirPeriodwTotal[namePercentTrucksMD] = df_TDM_byDirPeriodwTotal[nameMD] / df_TDM_byDirPeriodwTotal[nameVol]
    df_TDM_byDirPeriodwTotal[namePercentTrucksHV] = df_TDM_byDirPeriodwTotal[nameHV] / df_TDM_byDirPeriodwTotal[nameVol]
    
    #calculate period volume as percent of segment total
    df_TDM_byDirPeriodwTotal[namePercentSegTotal] = df_TDM_byDirPeriodwTotal[nameVol] / df_TDM_byDirPeriodwTotal[nameSegTotal]
 
    #display(df_TDM_byDirPeriodwTotal)
    #display(df_TDM_byDirPeriodwTotal[(df_TDM_byDirPeriodwTotal[nameSegID]=='0015_295.6')]) 

    #calculate capacities
    df_TDM_byDirPeriodwTotal[nameCap1Hr] = df_TDM_byDirPeriodwTotal[nameCap1HL] * df_TDM_byDirPeriodwTotal[nameLanes]
    
    #limit columns in dataframe
    df_TDM_byDirPeriodwTotal = df_TDM_byDirPeriodwTotal[[nameSegID,nameFG,nameAT,nameDir,namePeriod,nameVol,namePercentTrucksMD,namePercentTrucksHV,namePercentSegTotal,nameCap1Hr]]
    #display(df_TDM_byDirPeriodwTotal)
    #display(df_TDM_byDirPeriodwTotal[(df_TDM_byDirPeriodwTotal[nameSegID]=='0015_295.6')])
    
    dfs_TDM.append(df_TDM_byDirPeriodwTotal)
    
    #The sum of all volume percent of segment daily should be 1.0, add little extra on each side for rounding
    df_Check = df_TDM_byDirPeriodwTotal.groupby(indexTotal).agg({namePercentSegTotal:[np.sum]})
    df_Check.columns = df_Check.columns.droplevel(1)
    df_Check = df_Check[(df_Check[namePercentSegTotal] > 1.0000000001) | (df_Check[namePercentSegTotal] < 0.99999999)]
    #display(df_Check)

print('Done')

Base CSV Importing
Base Processing.
STIP CSV Importing
STIP Processing.
2030 RTP CSV Importing
2030 RTP Processing.
2040 RTP CSV Importing
2040 RTP Processing.
2050 RTP CSV Importing
2050 RTP Processing.
2030 No Build CSV Importing
2030 No Build Processing.
2040 No Build CSV Importing
2040 No Build Processing.
2050 No Build CSV Importing
2050 No Build Processing.
2030 Unfunded CSV Importing
2030 Unfunded Processing.
2040 Unfunded CSV Importing
2040 Unfunded Processing.
2050 Unfunded CSV Importing
2050 Unfunded Processing.
Done


In [163]:
df = dfs_TDM[0]

df = df[(df[nameSegID]=='0015_295.6')]
df = df[(df[namePeriod]!='Daily')]

display(df)

df2 = df.groupby('SEGID').agg({'VOL':[np.sum]})

display(df2)

,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr
1120,0015_295.6,CD Road,4.0,D1,AM,9076.7,0.073044,0.035564,0.026847,7955.0
1121,0015_295.6,CD Road,4.0,D1,EV,11744.5,0.143216,0.000000,0.034737,7955.0
1122,0015_295.6,CD Road,4.0,D1,MD,16926.0,0.127041,0.059843,0.050063,7955.0
1123,0015_295.6,CD Road,4.0,D1,PM,9687.2,0.095322,0.047248,0.028652,7955.0
1124,0015_295.6,CD Road,4.0,D2,AM,7403.7,0.103759,0.049975,0.021898,7955.0
1125,0015_295.6,CD Road,4.0,D2,EV,11345.9,0.127720,0.000000,0.033558,7955.0
1126,0015_295.6,CD Road,4.0,D2,MD,14112.4,0.130949,0.063264,0.041741,7955.0
1127,0015_295.6,CD Road,4.0,D2,PM,9493.0,0.074687,0.036722,0.028078,7955.0
1128,0015_295.6,Freeway,4.0,D1,AM,22987.0,0.072341,0.034454,0.067990,14320.0
1129,0015_295.6,Freeway,4.0,D1,EV,30226.5,0.168501,0.000000,0.089402,14320.0


,VOL
,sum
SEGID,
0015_295.6,338095.4


In [164]:
display(dfs_TDM_Total[0])

df = dfs_TDM_Total[0]

df = df[(df.index=='0015_295.6')]

display(df)

,SegTotalDYVol
SEGID,
0006_146.9,1392.4
0006_149.9,1510.9
0006_150.6,1718.0
0006_152.6,2643.9
0006_152.9,3656.2
...,...
WFRC_8208,1419.3
WFRC_8209,6019.0
WFRC_8213,562.5


,SegTotalDYVol
SEGID,
0015_295.6,338095.4


In [165]:
df_Scenarios_toProcess

,Process,ScenarioCode,ScenarioName,ScenarioYear,SegSummaryDBF,AdjScenarioCode
ScenarioID,,,,,,
0,Yes,Base19,Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed_BH,NaN
1,Yes,Base24,STIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,NaN
2,Yes,RTP30,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,NaN
3,Yes,RTP40,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed_BH,NaN
4,Yes,RTP50,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed_BH,NaN
5,Yes,NB30,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,RTP30
6,Yes,NB40,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,RTP40
7,Yes,NB50,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,RTP50
8,Yes,UF30,2030 Unfunded,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,RTP30


In [166]:
df_Scenarios_toAdjust

,Process,ScenarioCode,ScenarioName,ScenarioYear,SegSummaryDBF,AdjScenarioCode
ScenarioID,,,,,,
5,Yes,NB30,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,RTP30
6,Yes,NB40,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,RTP40
7,Yes,NB50,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,RTP50
8,Yes,UF30,2030 Unfunded,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,RTP30
9,Yes,UF40,2040 Unfunded,2040,v831_SE40_Net40UF_Summary_SEGID_Detailed_BH,RTP40
10,Yes,UF50,2050 Unfunded,2050,v831_SE50_Net50UF_Summary_SEGID_Detailed_BH,RTP50


In [167]:
#calculate adjustment for forecast for those non-RTP scenarios

dfs_ForecastAdj = []

for (idx, row) in df_Scenarios_toAdjust.iterrows():

    #print(row.loc[nameS] + ' CSV Importing')
    
    df_base = dfs_TDM_Total[idFromCode(row.loc['AdjScenarioCode'])]
    df_base.columns = (['DailyVolBase'])
    df_scenario  = dfs_TDM_Total[idFromCode(row.loc['ScenarioCode'])]
    df_scenario.columns = (['DailyVolScenario'])
    
    display(df_base)
    display(df_scenario)
    
    df_ForecastAdj = pd.DataFrame.merge(df_scenario,df_base,on=nameSegID,how='left')
    df_ForecastAdj['DailyVolAdj'] = df_ForecastAdj['DailyVolScenario'] - df_ForecastAdj['DailyVolBase']
    display(df_ForecastAdj)
    
    dfs_ForecastAdj.append(df_ForecastAdj)

,DailyVolBase
SEGID,
0006_146.9,1559.0
0006_149.9,1755.5
0006_150.6,1970.7
0006_152.6,2894.4
0006_152.9,3986.8
...,...
WFRC_8215,7150.5
WFRC_8216,18404.1
WFRC_8217,0.0


,DailyVolScenario
SEGID,
0006_146.9,1546.0
0006_149.9,1998.1
0006_150.6,2226.3
0006_152.6,3243.1
0006_152.9,4373.1
...,...
WFRC_8213,706.9
WFRC_8215,6656.0
WFRC_8216,17437.9


,DailyVolScenario,DailyVolBase,DailyVolAdj
SEGID,,,
0006_146.9,1546.0,1559.0,-13.0
0006_149.9,1998.1,1755.5,242.6
0006_150.6,2226.3,1970.7,255.6
0006_152.6,3243.1,2894.4,348.7
0006_152.9,4373.1,3986.8,386.3
...,...,...,...
WFRC_8213,706.9,709.8,-2.9
WFRC_8215,6656.0,7150.5,-494.5
WFRC_8216,17437.9,18404.1,-966.2


,DailyVolBase
SEGID,
0006_146.9,1684.9
0006_149.9,2230.4
0006_150.6,2456.5
0006_152.6,3346.8
0006_152.9,4750.8
...,...
WFRC_8215,9600.4
WFRC_8216,22968.9
WFRC_8217,0.0


,DailyVolScenario
SEGID,
0006_146.9,1669.8
0006_149.9,3098.9
0006_150.6,3352.3
0006_152.6,4473.5
0006_152.9,6005.6
...,...
WFRC_8213,829.4
WFRC_8215,8957.0
WFRC_8216,23391.1


...

,DailyVolScenario
SEGID,
0006_146.9,1685.8
0006_149.9,2111.0
0006_150.6,2340.3
0006_152.6,3251.5
0006_152.9,4679.9
...,...
WFRC_8215,9404.0
WFRC_8216,24993.0
WFRC_8217,0.0


,DailyVolScenario,DailyVolBase,DailyVolAdj
SEGID,,,
0006_146.9,1685.8,1684.9,0.9
0006_149.9,2111.0,2230.4,-119.4
0006_150.6,2340.3,2456.5,-116.2
0006_152.6,3251.5,3346.8,-95.3
0006_152.9,4679.9,4750.8,-70.9
...,...,...,...
WFRC_8215,9404.0,9600.4,-196.4
WFRC_8216,24993.0,22968.9,2024.1
WFRC_8217,0.0,0.0,0.0


,DailyVolBase
SEGID,
0006_146.9,1811.9
0006_149.9,3020.6
0006_150.6,3267.3
0006_152.6,4370.0
0006_152.9,5979.0
...,...
WFRC_8215,12274.5
WFRC_8216,28432.9
WFRC_8217,25756.0


,DailyVolScenario
SEGID,
0006_146.9,1813.4
0006_149.9,3029.6
0006_150.6,3276.8
0006_152.6,4372.0
0006_152.9,5982.6
...,...
WFRC_8215,11926.5
WFRC_8216,28419.9
WFRC_8217,25320.7


,DailyVolScenario,DailyVolBase,DailyVolAdj
SEGID,,,
0006_146.9,1813.4,1811.9,1.5
0006_149.9,3029.6,3020.6,9.0
0006_150.6,3276.8,3267.3,9.5
0006_152.6,4372.0,4370.0,2.0
0006_152.9,5982.6,5979.0,3.6
...,...,...,...
WFRC_8215,11926.5,12274.5,-348.0
WFRC_8216,28419.9,28432.9,-13.0
WFRC_8217,25320.7,25756.0,-435.3


# AADT and AAPeriodT Forecasts (Segments, Forecasts, TDM)

In [168]:
#import arcgis libraries
from arcgis.gis import *
gis = GIS()

shp_Forecasts = os.path.join(dirData,  r'forecasts\Master_Segs_withFactors_WFRCForecasts_20200506.shp')
                        
#import into spatially-enabled DataFrame
sdf_Forecasts = pd.DataFrame.spatial.from_featureclass(shp_Forecasts)
sdf_Forecasts

,FID,SEGID,BMP,EMP,LRS_ROUTE,ROUTE_ID,FULLNAME,FNCLASS,FC_CODE,FC_GROUP,FC_NAME,CO_FIPS,CO_NAME,PLANAREA,X,Y,ONLRS,T_MODEL,USE,RT_NUM,AADT2017,AADT2016,AADT2015,AADT2014,AADT2013,AADT2012,AADT2011,AADT2010,AADT2009,AADT2008,AADT2007,AADT2006,AADT2005,AADT2004,AADT2003,AADT2002,AADT2001,AADT2000,AADT1999,AADT1998,AADT1997,AADT1996,AADT1994,AADT1993,AADT1992,AADT1991,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981,SUTRK2017,CUTRK2017,DOWFACFC,DOWFACAT,FAC_MON,FAC_TUE,FAC_WED,FAC_THU,FAC_FRI,FAC_SAT,FAC_SUN,FAC_WDAVG,FAC_WEAVG,FAC_WEMAX,SSNGRP,SSNVOLCLS,SSNATGROUP,FAC_JAN,FAC_FEB,FAC_MAR,FAC_APR,FAC_MAY,FAC_JUN,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,FAC_MAXMO,FAC_MAX,FACMANADJ,SUTRUCKS,CUTRUCKS,SUBAREAID,WFRC_FLG,DISTANCE,SR_FAR,Publish,SEGID_1,CO_FIPS_1,F2015,F2017,F2019,F2024,F2030,F2040,F2050,Ch15to19,Ch17to19,Ch19to24,Ch24to30,Ch30to40,Ch40to50,Ch17to50,Notes,SHAPE
0,0,0068_036.0,36.038,38.107,0068P,68,CAMP WILLIAMS RD,Arterial,3,Arterial,Other Principal Arterial,35,SALT LAKE,WFRC,420614.3187,4.477961e+06,Yes,,1,68,31000,22998,22030,21000,20000,20000,20000,20000,20000,16000,16000,15000,11000,10000,8900,7900,6600,4300,4300,3500,2900,2700,2400,2200,2100,2000,1500,1400,1500,1500,1400,1300,1200,1100,990,980,0.039000,0.018000,Arterial,Transition,0.985,0.995,1.020,1.044,1.142,1.036,0.773,1.012,0.985,1.142,StatewideArterial,0-AllVolumes,2-Transition,0.851,0.896,0.980,1.015,1.057,1.067,1.039,1.067,1.055,1.043,0.973,0.927,0.892,1.017,1.057,1.022,6.0,1.067,0.0,0.063,0.030,1,0,2.070848,SR,1,0068_036.0,35,22030,31000,32500,41000,8100,8700,10500,10470,1500,8500,-32900,600,1800,-20500,new parallel MVC freeway and N-S arterial,"{""paths"": [[[421160.5499999998, 4476386.34], [..."
1,1,WFRC_8000,1.488,1.488,0131P,131,,Arterial,30,Arterial,Proposed Other Principal Arterial,35,SALT LAKE,WFRC,420880.8541,4.479653e+06,No,,1,131,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,,,Arterial,Transition,0.985,0.995,1.020,1.044,1.142,1.036,0.773,1.012,0.985,1.142,StatewideArterial,0-AllVolumes,2-Transition,0.851,0.896,0.980,1.015,1.057,1.067,1.039,1.067,1.055,1.043,0.973,0.927,0.892,1.017,1.057,1.022,6.0,1.067,0.0,0.035,0.025,1,0,1.021232,SR,1,WFRC_8000,35,0,0,0,15000,35500,47000,57000,0,0,15000,20500,11500,10000,57000,,"{""paths"": [[[420067.91000000015, 4479534.80000..."
2,2,0131_001.7,1.742,3.047,0131P,131,PORTER ROCKWELL BLVD,Arterial,3,Arterial,Other Principal Arterial,35,SALT LAKE,WFRC,422891.2498,4.481145e+06,Yes,,1,131,1000,1000,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.198000,0.032000,Arterial,Suburban,1.083,1.116,1.129,1.137,1.146,0.828,0.566,1.118,0.847,1.146,StatewideArterial,0-AllVolumes,3-Suburban,0.885,0.954,1.001,1.010,1.029,1.038,1.009,1.033,1.044,1.032,0.978,0.956,0.936,1.013,1.026,1.018,9.0,1.044,0.0,0.032,0.018,1,2,1.298305,SR,1,0131_001.7,35,0,1000,1700,8800,21000,29000,24000,1700,700,7100,12200,8000,-5000,23000,Porter Rockwell connection to Redwood/MVC in 2...,"{""paths"": [[[422222.28000000026, 4480359.77], ..."
3,3,0068_038.1,38.107,39.167,0068P,68,CAMP WILLIAMS RD,Arterial,3,Arterial,Other Principal Arterial,35,SALT LAKE,WFRC,419789.5853,4.480342e+06,Yes,,1,68,22000,21392,21330,18000,19000,22000,22000,20000,16000,16000,16000,15000,11000,9900,9800,8700,7400,6300,5900,6700,4600,4000,3500,3100,2900,2700,2200,2000,2500,2400,2300,2100,1900,2000,0,0,0.015000,0.014000,Arterial,Transition,0.985,0.995,1.020,1.044,1.142,1.036,0.773,1.012,0.985,1.142,StatewideArterial,0-AllVolumes,2-Transition,0.851,0.896,0.980,1.015,1.057,1.067,1.039,1.067,1.055,1.043,0.973,0.927,0.892,1.017,1.057,1.022,6.0,1.067,0.0,0.040,0.020,1,0,1.061080,SR,1,0068_038.1,35,21330,22000,24500,26500,31000,33500,35500,3170,2500,2000,4500,2500,2000,13500,,"{""paths"": [[[420067.91000000015, 4479534.80000..."
4,4,2927_000.0,0.000,1.019,2927P,2927,DEER RIDGE DR,Arterial,6,Arterial,Minor Collector,35,SALT LAKE,WFRC,428613.1111,4.481094e+06,Yes,

In [169]:
for col in sdf_Forecasts.columns: 
    print(col) 

FID
SEGID
BMP
EMP
LRS_ROUTE
ROUTE_ID
FULLNAME
FNCLASS
FC_CODE
FC_GROUP
FC_NAME
CO_FIPS
CO_NAME
PLANAREA
X
Y
ONLRS
T_MODEL
USE
RT_NUM
AADT2017
AADT2016
AADT2015
AADT2014
AADT2013
AADT2012
AADT2011
AADT2010
AADT2009
AADT2008
AADT2007
AADT2006
AADT2005
AADT2004
AADT2003
AADT2002
AADT2001
AADT2000
AADT1999
AADT1998
AADT1997
AADT1996
AADT1994
AADT1993
AADT1992
AADT1991
AADT1990
AADT1989
AADT1988
AADT1987
AADT1986
AADT1985
AADT1984
AADT1983
AADT1982
AADT1981
SUTRK2017
CUTRK2017
DOWFACFC
DOWFACAT
FAC_MON
FAC_TUE
FAC_WED
FAC_THU
FAC_FRI
FAC_SAT
FAC_SUN
FAC_WDAVG
FAC_WEAVG
FAC_WEMAX
SSNGRP
SSNVOLCLS
SSNATGROUP
FAC_JAN
FAC_FEB
FAC_MAR
FAC_APR
FAC_MAY
FAC_JUN
FAC_JUL
FAC_AUG
FAC_SEP
FAC_OCT
FAC_NOV
FAC_DEC
FAC_WIN
FAC_SPR
FAC_SUM
FAC_FAL
FAC_MAXMO
FAC_MAX
FACMANADJ
SUTRUCKS
CUTRUCKS
SUBAREAID
WFRC_FLG
DISTANCE
SR_FAR
Publish
SEGID_1
CO_FIPS_1
F2015
F2017
F2019
F2024
F2030
F2040
F2050
Ch15to19
Ch17to19
Ch19to24
Ch24to30
Ch30to40
Ch40to50
Ch17to50
Notes
SHAPE


In [170]:
df = sdf_Forecasts

df = df[(df[nameSegID]=='0015_295.6')]

df

,FID,SEGID,BMP,EMP,LRS_ROUTE,ROUTE_ID,FULLNAME,FNCLASS,FC_CODE,FC_GROUP,FC_NAME,CO_FIPS,CO_NAME,PLANAREA,X,Y,ONLRS,T_MODEL,USE,RT_NUM,AADT2017,AADT2016,AADT2015,AADT2014,AADT2013,AADT2012,AADT2011,AADT2010,AADT2009,AADT2008,AADT2007,AADT2006,AADT2005,AADT2004,AADT2003,AADT2002,AADT2001,AADT2000,AADT1999,AADT1998,AADT1997,AADT1996,AADT1994,AADT1993,AADT1992,AADT1991,AADT1990,AADT1989,AADT1988,AADT1987,AADT1986,AADT1985,AADT1984,AADT1983,AADT1982,AADT1981,SUTRK2017,CUTRK2017,DOWFACFC,DOWFACAT,FAC_MON,FAC_TUE,FAC_WED,FAC_THU,FAC_FRI,FAC_SAT,FAC_SUN,FAC_WDAVG,FAC_WEAVG,FAC_WEMAX,SSNGRP,SSNVOLCLS,SSNATGROUP,FAC_JAN,FAC_FEB,FAC_MAR,FAC_APR,FAC_MAY,FAC_JUN,FAC_JUL,FAC_AUG,FAC_SEP,FAC_OCT,FAC_NOV,FAC_DEC,FAC_WIN,FAC_SPR,FAC_SUM,FAC_FAL,FAC_MAXMO,FAC_MAX,FACMANADJ,SUTRUCKS,CUTRUCKS,SUBAREAID,WFRC_FLG,DISTANCE,SR_FAR,Publish,SEGID_1,CO_FIPS_1,F2015,F2017,F2019,F2024,F2030,F2040,F2050,Ch15to19,Ch17to19,Ch19to24,Ch24to30,Ch30to40,Ch40to50,Ch17to50,Notes,SHAPE
388,388,0015_295.6,295.641,297.944,0015P,15,I-15 NB FWY,Freeway,1,Freeway,Interstate,35,SALT LAKE,WFRC,423474.6492,4.495228e+06,Yes,,4,15,261000,259218,238075,230000,206000,203000,200000,197000,194000,192000,200000,197000,191000,188000,186000,181000,172000,129000,118000,114000,134000,157000,138000,128000,119000,111000,104000,102000,87000,85000,80000,79000,74000,71000,68000,67000,0.123000,0.057000,FreewayI15,Urban,1.019,1.047,1.067,1.091,1.149,0.959,0.668,1.057,0.926,1.149,FreewayI15,0-AllVolumes,4-Urban,0.879,0.936,0.984,1.028,1.03,1.038,1.02,1.039,1.021,1.025,0.989,0.99,0.938,1.014,1.033,1.012,8.0,1.039,0.0,0.24,0.067,1,0,2.30286,SR,1,0015_295.6,35,238075,261000,277500,297500,308500,313000,327500,39425,16500,20000,11000,4500,14500,66500,,"{""paths"": [[[423819.33999999985, 4493412.10999..."


In [171]:
#normalize data with forecasts

meltIndex    = ['SEGID','DOWFACFC','SSNGRP']
valueColumns = ['AADT2017','F2019','F2024','F2030','F2040','F2050']
df_Forecasts = pd.melt(sdf_Forecasts, id_vars=meltIndex, value_vars=valueColumns)
df_Forecasts.columns = ('SEGID','DOWFACFC','SsnGrp','FCOL','ForecastAADT')
df_Forecasts['FYEAR'] = (df_Forecasts['FCOL'].str[-4:]).astype('int64')
df_Forecasts = df_Forecasts[['SEGID','DOWFACFC','SsnGrp','FYEAR','ForecastAADT']]
df_Forecasts

,SEGID,DOWFACFC,SsnGrp,FYEAR,ForecastAADT
0,0068_036.0,Arterial,StatewideArterial,2017,31000
1,WFRC_8000,Arterial,StatewideArterial,2017,0
2,0131_001.7,Arterial,StatewideArterial,2017,1000
3,0068_038.1,Arterial,StatewideArterial,2017,22000
4,2927_000.0,Arterial,StatewideArterial,2017,500
...,...,...,...,...,...
16699,0080_127.0,FreewayI80,FreewayI80,2050,63500
16700,WFRC_8217,Arterial,StatewideArterial,2050,19500
16701,WFRC_8218,FreewayI215,FreewayI215,2050,37500
16702,1508_000.4,Arterial,StatewideArterial,2050,18000


In [172]:
dfs_ForecastAdj[0]

,DailyVolScenario,DailyVolBase,DailyVolAdj
SEGID,,,
0006_146.9,1546.0,1559.0,-13.0
0006_149.9,1998.1,1755.5,242.6
0006_150.6,2226.3,1970.7,255.6
0006_152.6,3243.1,2894.4,348.7
0006_152.9,4373.1,3986.8,386.3
...,...,...,...
WFRC_8213,706.9,709.8,-2.9
WFRC_8215,6656.0,7150.5,-494.5
WFRC_8216,17437.9,18404.1,-966.2


In [173]:
df_Scenarios_toProcess

,Process,ScenarioCode,ScenarioName,ScenarioYear,SegSummaryDBF,AdjScenarioCode
ScenarioID,,,,,,
0,Yes,Base19,Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed_BH,NaN
1,Yes,Base24,STIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,NaN
2,Yes,RTP30,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,NaN
3,Yes,RTP40,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed_BH,NaN
4,Yes,RTP50,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed_BH,NaN
5,Yes,NB30,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,RTP30
6,Yes,NB40,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,RTP40
7,Yes,NB50,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,RTP50
8,Yes,UF30,2030 Unfunded,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,RTP30


In [174]:
#calculate forecast daily volume for each scenario to process

dfs_PeriodForecasts = []

for (idx, row) in df_Scenarios_toProcess.iterrows():

    _scenarioID = idx
    
    print(row.loc[nameS] + ' Processing')
    print(idx)
    #df_base = dfs_TDM_Total[idFromCode(row.loc['AdjScenarioCode'])]

    #display(dfs_TDM[idx])
    
    df_TDM = dfs_TDM[idx]
    
    #get TDM year
    _scenarioyear = yearFromID(_scenarioID)
    
    #filter only for year of forecast
    df_Forecasts_SYEAR = df_Forecasts[df_Forecasts.FYEAR == _scenarioyear]
    
    #adjust forecast
    
    for (idx, row) in df_Scenarios_toAdjust.iterrows():
        if (_scenarioID == idx):
            #display("adjust this")
            
            #get adjust id
            _adjustId = df_Scenarios_toAdjust.index.get_loc(idx)

            #display(_adjustId)
            #display(dfs_ForecastAdj[_adjustId])
            
            df_Forecasts_Adjust = pd.DataFrame.merge(df_Forecasts_SYEAR,dfs_ForecastAdj[_adjustId],on='SEGID')
            
            #adjust volume. if scenario vol = 0 then zero out forecast volume, otherwise forecast volume is adjusted
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DailyVolScenario'] == 0), 'ForecastAADTAdj'] = 0
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['DailyVolScenario']  > 0), 'ForecastAADTAdj'] = df_Forecasts_Adjust['ForecastAADT'] + df_Forecasts_Adjust['DailyVolAdj'] 
            
            #if adjustment < 0 then bring to 0
            df_Forecasts_Adjust.loc[(df_Forecasts_Adjust['ForecastAADTAdj']   < 0), 'ForecastAADTAdj'] = 0  
            
            display(df_Forecasts_Adjust)
            
            df_Forecasts_Adjust = df_Forecasts_Adjust[['SEGID','DOWFACFC','SsnGrp','FYEAR','ForecastAADTAdj']]
            df_Forecasts_Adjust = df_Forecasts_Adjust.rename(columns={"ForecastAADTAdj": "ForecastAADT"})
            
            df_Forecasts_SYEAR = df_Forecasts_Adjust.copy()
            
    #display(df_Forecasts_SYEAR)
    
    #join with TDM data
    df_Forecasts_withPeriod = pd.DataFrame.merge(df_TDM, df_Forecasts_SYEAR, on=('SEGID'))
    
    df_Forecasts_withPeriod['ForecastAAPeriodT'] = round(df_Forecasts_withPeriod['ForecastAADT'] * df_Forecasts_withPeriod['VolPercentSegDY'],0)
    
    display(df_Forecasts_withPeriod.head(20))
    
    dfs_PeriodForecasts.append(df_Forecasts_withPeriod)
    

Base Processing
0


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0
1,0013_000.0,Arterial,3.0,D1,EV,1416.00,0.181921,0.000000,0.117970,3716.0,Arterial,StatewideArterial,2019,19000,2241.0
2,0013_000.0,Arterial,3.0,D1,MD,2038.60,0.174826,0.084224,0.169839,3716.0,Arterial,StatewideArterial,2019,19000,3227.0
3,0013_000.0,Arterial,3.0,D1,PM,1333.00,0.166092,0.077194,0.111055,3716.0,Arterial,StatewideArterial,2019,19000,2110.0
4,0013_000.0,Arterial,3.0,D2,AM,782.40,0.207183,0.090874,0.065183,3716.0,Arterial,StatewideArterial,2019,19000,1238.0
5,0013_000.0,Arterial,3.0,D2,EV,1593.30,0.173476,0.000000,0.132741,3716.0,Arterial,StatewideArterial,2019,19000,2522.0
6,0013_000.0,Arterial,3.0,D2,MD,2098.80,0.181532,0.086621,0.174855,3716.0,Arterial,StatewideArterial,2019,19000,3322.0
7,0013_000.0,Arterial,3.0,D2,PM,1684.90,0.139118,0.064752,0.140372,3716.0,Arterial,StatewideArterial,2019,19000,2667.0
8,0013_000.0,Arterial,3.0,D1,Daily,7201.86,0.170056,0.058718,0.600000,3716.0,Arterial,StatewideArterial,2019,19000,11400.0
9,0013_000.0,Arterial,3.0,D2,Daily,4801.24,0.170056,0.058718,0.400000,3716.0,Arterial,StatewideArterial,2019,19000,7600.0


STIP Processing
1


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0
1,0013_000.0,Arterial,3.0,D1,EV,1481.50,0.185960,0.000000,0.117709,3716.0,Arterial,StatewideArterial,2024,19500,2295.0
2,0013_000.0,Arterial,3.0,D1,MD,2128.20,0.178978,0.088009,0.169091,3716.0,Arterial,StatewideArterial,2024,19500,3297.0
3,0013_000.0,Arterial,3.0,D1,PM,1401.60,0.172303,0.081122,0.111361,3716.0,Arterial,StatewideArterial,2024,19500,2172.0
4,0013_000.0,Arterial,3.0,D2,AM,823.10,0.209209,0.095007,0.065398,3716.0,Arterial,StatewideArterial,2024,19500,1275.0
5,0013_000.0,Arterial,3.0,D2,EV,1665.40,0.173832,0.000000,0.132321,3716.0,Arterial,StatewideArterial,2024,19500,2580.0
6,0013_000.0,Arterial,3.0,D2,MD,2185.00,0.183844,0.089291,0.173604,3716.0,Arterial,StatewideArterial,2024,19500,3385.0
7,0013_000.0,Arterial,3.0,D2,PM,1779.40,0.141789,0.066427,0.141378,3716.0,Arterial,StatewideArterial,2024,19500,2757.0
8,0013_000.0,Arterial,3.0,D1,Daily,7551.66,0.172992,0.060980,0.600000,3716.0,Arterial,StatewideArterial,2024,19500,11700.0
9,0013_000.0,Arterial,3.0,D2,Daily,5034.44,0.172992,0.060980,0.400000,3716.0,Arterial,StatewideArterial,2024,19500,7800.0


2030 RTP Processing
2


...

,SEGID,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,DailyVolScenario,DailyVolBase,DailyVolAdj,ForecastAADTAdj
0,0068_036.0,Arterial,StatewideArterial,2040,8700,14139.2,12791.8,1347.4,10047.4
1,WFRC_8000,Arterial,StatewideArterial,2040,47000,48358.0,47350.0,1008.0,48008.0
2,0131_001.7,Arterial,StatewideArterial,2040,29000,36808.3,37345.4,-537.1,28462.9
3,0068_038.1,Arterial,StatewideArterial,2040,33500,34392.3,30279.5,4112.8,37612.8
4,2046_002.3,Arterial,StatewideArterial,2040,3500,627.5,642.6,-15.1,3484.9
...,...,...,...,...,...,...,...,...,...
2581,0080_127.0,FreewayI80,FreewayI80,2040,58000,46838.5,47338.1,-499.6,57500.4
2582,WFRC_8217,Arterial,StatewideArterial,2040,0,0.0,0.0,0.0,0.0
2583,WFRC_8218,FreewayI215,FreewayI215,2040,33500,38149.5,37570.2,579.3,34079.3
2584,1508_000.4,Arterial,StatewideArterial,2040,11000,12826.4,13445.3,-618.9,10381.1


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.091530,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0
1,0013_000.0,Arterial,3.0,D1,EV,1540.40,0.183134,0.000000,0.116312,3716.0,Arterial,StatewideArterial,2040,20487.5,2383.0
2,0013_000.0,Arterial,3.0,D1,MD,2192.00,0.153422,0.081980,0.165513,3716.0,Arterial,StatewideArterial,2040,20487.5,3391.0
3,0013_000.0,Arterial,3.0,D1,PM,1495.10,0.171761,0.074711,0.112891,3716.0,Arterial,StatewideArterial,2040,20487.5,2313.0
4,0013_000.0,Arterial,3.0,D2,AM,849.50,0.204944,0.086404,0.064144,3716.0,Arterial,StatewideArterial,2040,20487.5,1314.0
5,0013_000.0,Arterial,3.0,D2,EV,1755.90,0.164189,0.000000,0.132584,3716.0,Arterial,StatewideArterial,2040,20487.5,2716.0
6,0013_000.0,Arterial,3.0,D2,MD,2294.00,0.175153,0.078945,0.173214,3716.0,Arterial,StatewideArterial,2040,20487.5,3549.0
7,0013_000.0,Arterial,3.0,D2,PM,1904.60,0.136354,0.060853,0.143812,3716.0,Arterial,StatewideArterial,2040,20487.5,2946.0
8,0013_000.0,Arterial,3.0,D1,Daily,7946.22,0.163980,0.055415,0.600000,3716.0,Arterial,StatewideArterial,2040,20487.5,12292.0
9,0013_000.0,Arterial,3.0,D2,Daily,5297.48,0.163980,0.055415,0.400000,3716.0,Arterial,StatewideArterial,2040,20487.5,8195.0


2050 Unfunded Processing
10


,SEGID,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,DailyVolScenario,DailyVolBase,DailyVolAdj,ForecastAADTAdj
0,0068_036.0,Arterial,StatewideArterial,2050,10500,14883.0,15250.3,-367.3,10132.7
1,WFRC_8000,Arterial,StatewideArterial,2050,57000,56900.5,57725.8,-825.3,56174.7
2,0131_001.7,Arterial,StatewideArterial,2050,24000,29477.1,32527.2,-3050.1,20949.9
3,0068_038.1,Arterial,StatewideArterial,2050,35500,29538.0,33046.2,-3508.2,31991.8
4,2046_002.3,Arterial,StatewideArterial,2050,6800,4934.0,7757.5,-2823.5,3976.5
...,...,...,...,...,...,...,...,...,...
2632,0080_127.0,FreewayI80,FreewayI80,2050,63500,52997.3,52951.0,46.3,63546.3
2633,WFRC_8217,Arterial,StatewideArterial,2050,19500,25320.7,25756.0,-435.3,19064.7
2634,WFRC_8218,FreewayI215,FreewayI215,2050,37500,42391.0,42419.0,-28.0,37472.0
2635,1508_000.4,Arterial,StatewideArterial,2050,18000,21884.4,21946.2,-61.8,17938.2


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0
1,0013_000.0,Arterial,3.0,D1,EV,1524.80,0.156676,0.000000,0.111613,3716.0,Arterial,StatewideArterial,2050,20511.0,2289.0
2,0013_000.0,Arterial,3.0,D1,MD,2230.00,0.147085,0.081659,0.163232,3716.0,Arterial,StatewideArterial,2050,20511.0,3348.0
3,0013_000.0,Arterial,3.0,D1,PM,1608.30,0.170242,0.074551,0.117725,3716.0,Arterial,StatewideArterial,2050,20511.0,2415.0
4,0013_000.0,Arterial,3.0,D2,AM,874.30,0.210111,0.088757,0.063997,3716.0,Arterial,StatewideArterial,2050,20511.0,1313.0
5,0013_000.0,Arterial,3.0,D2,EV,1825.10,0.163279,0.000000,0.133594,3716.0,Arterial,StatewideArterial,2050,20511.0,2740.0
6,0013_000.0,Arterial,3.0,D2,MD,2356.50,0.173944,0.078803,0.172492,3716.0,Arterial,StatewideArterial,2050,20511.0,3538.0
7,0013_000.0,Arterial,3.0,D2,PM,2018.90,0.133439,0.059191,0.147780,3716.0,Arterial,StatewideArterial,2050,20511.0,3031.0
8,0013_000.0,Arterial,3.0,D1,Daily,8196.90,0.157384,0.055550,0.600000,3716.0,Arterial,StatewideArterial,2050,20511.0,12307.0
9,0013_000.0,Arterial,3.0,D2,Daily,5464.60,0.157384,0.055550,0.400000,3716.0,Arterial,StatewideArterial,2050,20511.0,8204.0


In [175]:
dfs_PeriodForecasts[0]

,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0
1,0013_000.0,Arterial,3.0,D1,EV,1416.00,0.181921,0.000000,0.117970,3716.0,Arterial,StatewideArterial,2019,19000,2241.0
2,0013_000.0,Arterial,3.0,D1,MD,2038.60,0.174826,0.084224,0.169839,3716.0,Arterial,StatewideArterial,2019,19000,3227.0
3,0013_000.0,Arterial,3.0,D1,PM,1333.00,0.166092,0.077194,0.111055,3716.0,Arterial,StatewideArterial,2019,19000,2110.0
4,0013_000.0,Arterial,3.0,D2,AM,782.40,0.207183,0.090874,0.065183,3716.0,Arterial,StatewideArterial,2019,19000,1238.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25645,WFRC_8218,CD Road,3.0,D2,EV,3624.60,0.138277,0.000000,0.126594,2946.0,FreewayI215,FreewayI215,2019,25500,3228.0
25646,WFRC_8218,CD Road,3.0,D2,MD,4648.90,0.145282,0.074491,0.162368,2946.0,FreewayI215,FreewayI215,2019,25500,4140.0
25647,WFRC_8218,CD Road,3.0,D2,PM,3819.30,0.114157,0.058649,0.133394,2946.0,FreewayI215,FreewayI215,2019,25500,3402.0
25648,WFRC_8218,CD Road,3.0,D1,Daily,17179.08,0.129960,0.049752,0.600000,2946.0,FreewayI215,FreewayI215,2019,25500,15300.0


In [176]:
df = dfs_PeriodForecasts[0]

df = df[(df[nameSegID]=='0015_295.6')]

display(df)

df2 = df[df[namePeriod]!='Daily']

display(df2)

df2 = df2.groupby(nameSegID).agg({'VOL':[np.sum],'VolPercentSegDY':[np.sum]})

display(df2)

,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT
260,0015_295.6,CD Road,4.0,D1,AM,9076.70,0.073044,0.035564,0.026847,7955.0,FreewayI15,FreewayI15,2019,277500,7450.0
261,0015_295.6,CD Road,4.0,D1,EV,11744.50,0.143216,0.000000,0.034737,7955.0,FreewayI15,FreewayI15,2019,277500,9640.0
262,0015_295.6,CD Road,4.0,D1,MD,16926.00,0.127041,0.059843,0.050063,7955.0,FreewayI15,FreewayI15,2019,277500,13892.0
263,0015_295.6,CD Road,4.0,D1,PM,9687.20,0.095322,0.047248,0.028652,7955.0,FreewayI15,FreewayI15,2019,277500,7951.0
264,0015_295.6,CD Road,4.0,D2,AM,7403.70,0.103759,0.049975,0.021898,7955.0,FreewayI15,FreewayI15,2019,277500,6077.0
265,0015_295.6,CD Road,4.0,D2,EV,11345.90,0.127720,0.000000,0.033558,7955.0,FreewayI15,FreewayI15,2019,277500,9312.0
266,0015_295.6,CD Road,4.0,D2,MD,14112.40,0.130949,0.063264,0.041741,7955.0,FreewayI15,FreewayI15,2019,277500,11583.0
267,0015_295.6,CD Road,4.0,D2,PM,9493.00,0.074687,0.036722,0.028078,7955.0,FreewayI15,FreewayI15,2019,277500,7792.0
268,0015_295.6,Freeway,4.0,D1,AM,22987.00,0.072341,0.034454,0.067990,14320.0,FreewayI15,FreewayI15,2019,277500,18867.0
269,0015_295.6,Freeway,4.0,D1,EV,30226.50,0.168501,0.000000,0.089402,14320.0,FreewayI15,FreewayI15,2019,277500,24809.0


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT
260,0015_295.6,CD Road,4.0,D1,AM,9076.7,0.073044,0.035564,0.026847,7955.0,FreewayI15,FreewayI15,2019,277500,7450.0
261,0015_295.6,CD Road,4.0,D1,EV,11744.5,0.143216,0.000000,0.034737,7955.0,FreewayI15,FreewayI15,2019,277500,9640.0
262,0015_295.6,CD Road,4.0,D1,MD,16926.0,0.127041,0.059843,0.050063,7955.0,FreewayI15,FreewayI15,2019,277500,13892.0
263,0015_295.6,CD Road,4.0,D1,PM,9687.2,0.095322,0.047248,0.028652,7955.0,FreewayI15,FreewayI15,2019,277500,7951.0
264,0015_295.6,CD Road,4.0,D2,AM,7403.7,0.103759,0.049975,0.021898,7955.0,FreewayI15,FreewayI15,2019,277500,6077.0
265,0015_295.6,CD Road,4.0,D2,EV,11345.9,0.127720,0.000000,0.033558,7955.0,FreewayI15,FreewayI15,2019,277500,9312.0
266,0015_295.6,CD Road,4.0,D2,MD,14112.4,0.130949,0.063264,0.041741,7955.0,FreewayI15,FreewayI15,2019,277500,11583.0
267,0015_295.6,CD Road,4.0,D2,PM,9493.0,0.074687,0.036722,0.028078,7955.0,FreewayI15,FreewayI15,2019,277500,7792.0
268,0015_295.6,Freeway,4.0,D1,AM,22987.0,0.072341,0.034454,0.067990,14320.0,FreewayI15,FreewayI15,2019,277500,18867.0
269,0015_295.6,Freeway,4.0,D1,EV,30226.5,0.168501,0.000000,0.089402,14320.0,FreewayI15,FreewayI15,2019,277500,24809.0


,VOL,VolPercentSegDY
,sum,sum
SEGID,,
0015_295.6,338095.4,1.0


# Adjust for Season, DOW

In [177]:
#read in data from other notebook pre-processing

csv_SeasonFactors = os.path.join(dirFactors, r'SeasonFactors.csv')
csv_DOWFactors = os.path.join(dirFactors, r'DOWFactors.csv')
csv_TODFactors = os.path.join(dirFactors, r'TODFactors.csv')

df_SeasonFactors = pd.read_csv(csv_SeasonFactors)
df_DOWFactors = pd.read_csv(csv_DOWFactors)
df_TODFactors = pd.read_csv(csv_TODFactors)

df_SeasonFactors = df_SeasonFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})
df_DOWFactors = df_DOWFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})
df_TODFactors = df_TODFactors.rename(columns={"FuncGroup": nameFG, "AreaType": nameAT})

display(df_SeasonFactors)
display(df_DOWFactors)
display(df_TODFactors)

,SsnGrp,AREATYPE,Vol_From,Vol_To,FUNCGROUP,SiteGroupSeason,SeasonGroup,SeasonFactor
0,AlpineLoop,1,0,1000000,Arterial,SN0,M01-Jan,0.273
1,AlpineLoop,1,0,1000000,Arterial,SN0,M02-Feb,0.303
2,AlpineLoop,1,0,1000000,Arterial,SN0,M03-Mar,0.397
3,AlpineLoop,1,0,1000000,Arterial,SN0,M04-Apr,0.499
4,AlpineLoop,1,0,1000000,Arterial,SN0,M05-May,1.005
...,...,...,...,...,...,...,...,...
8419,WashingtonFreeway,5,0,1000000,CD Road,SNX,S02-Spr,1.026
8420,WashingtonFreeway,5,0,1000000,CD Road,SNX,S03-Sum,1.066
8421,WashingtonFreeway,5,0,1000000,CD Road,SNX,S04-Fal,1.001
8422,WashingtonFreeway,5,0,1000000,CD Road,SNX,MaxMonth,1.099


,DOWFACFC,AREATYPE,FUNCGROUP,SiteGroupDOW,DOWGroup,DOWFactor
0,Arterial,1,Arterial,WD0,1-Monday,0.907
1,Arterial,1,Arterial,WD0,2-Tuesday,0.902
2,Arterial,1,Arterial,WD0,3-Wednesday,0.943
3,Arterial,1,Arterial,WD0,4-Thursday,0.995
4,Arterial,1,Arterial,WD0,5-Friday,1.170
...,...,...,...,...,...,...
2335,FreewayUS40,5,CD Road,WDK,5-Friday,1.170
2336,FreewayUS40,5,CD Road,WDK,6-Saturday,1.029
2337,FreewayUS40,5,CD Road,WDK,7-Sunday,0.777
2338,FreewayUS40,5,CD Road,WDK,W1-Weekday,1.007


,SiteGroupSeason,SeasonGroup,DOWGroup,Period,HourPercentOf,VolMaxHourPercent
0,SN0,M01-Jan,1-Monday,AM,Period,0.552833
1,SN0,M01-Jan,1-Monday,EV,Period,0.292623
2,SN0,M01-Jan,1-Monday,MD,Period,0.253412
3,SN0,M01-Jan,1-Monday,PM,Period,0.417949
4,SN0,M01-Jan,2-Tuesday,AM,Period,0.493776
...,...,...,...,...,...,...
39163,SNX,Year,W1-Weekday,PM,Daily,0.077093
39164,SNX,Year,W2-Weekend,AM,Daily,0.040223
39165,SNX,Year,W2-Weekend,EV,Daily,0.060936
39166,SNX,Year,W2-Weekend,MD,Daily,0.077494


In [178]:
#add season factor

dfs_PeriodForecasts_Season = []

#for (idx, row) in dfs_PeriodForecasts.iterrows():

for df_PeriodForecasts in dfs_PeriodForecasts: 
    
    #joing forecasts data with season factors
    df_PeriodForecasts_Season = pd.DataFrame.merge(df_PeriodForecasts,df_SeasonFactors,on=('SsnGrp',nameFG,nameAT),how='left')
    
    #remove unapplicable volume classes
    df_PeriodForecasts_Season = df_PeriodForecasts_Season[(df_PeriodForecasts_Season.ForecastAADT >= df_PeriodForecasts_Season.Vol_From) & (df_PeriodForecasts_Season.ForecastAADT < df_PeriodForecasts_Season.Vol_To)]
    
    #calculate Average Season
    df_PeriodForecasts_Season['ForecastASeasonPeriodT'] = df_PeriodForecasts_Season['ForecastAAPeriodT'] *df_PeriodForecasts_Season['SeasonFactor'] 
    
    display(df_PeriodForecasts_Season)
    
    dfs_PeriodForecasts_Season.append(df_PeriodForecasts_Season)
    

,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M01-Jan,0.885,1479.720
1,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M02-Feb,0.954,1595.088
2,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M03-Mar,1.001,1673.672
3,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M04-Apr,1.010,1688.720
4,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M05-May,1.029,1720.488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
481495,WFRC_8218,CD Road,3.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,2946.0,FreewayI215,FreewayI215,2019,25500,10200.0,0,1000000,SN8,S02-Spr,1.015,10353.000
481496,WFRC_8218,CD Road,3.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,2946.0,FreewayI215,FreewayI215,2019,25500,10200.0,0,1000000,SN8,S03-Sum,1.012,10322.400
481497,WFRC_8218,CD Road,3.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,2946.0,FreewayI215,FreewayI215,2019,25500,10200.0,0,1000000,SN8,S04-Fal,1.003,10230.600
481498,WFRC_8218,CD Road,3.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,2946.0,FreewayI215,FreewayI215,2019,25500,10200.0,0,1000000,SN8,MaxMonth,1.025,10455.000


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M01-Jan,0.885,1538.130
1,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M02-Feb,0.954,1658.052
2,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M03-Mar,1.001,1739.738
3,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M04-Apr,1.010,1755.380
4,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M05-May,1.029,1788.402
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
484195,WFRC_8218,CD Road,3.0,D2,Daily,12412.32,0.134747,0.051133,0.400000,2946.0,FreewayI215,FreewayI215,2024,27500,11000.0,0,1000000,SN8,S02-Spr,1.015,11165.000
484196,WFRC_8218,CD Road,3.0,D2,Daily,12412.32,0.134747,0.051133,0.400000,2946.0,FreewayI215,FreewayI215,2024,27500,11000.0,0,1000000,SN8,S03-Sum,1.012,11132.000
484197,WFRC_8218,CD Road,3.0,D2,Daily,12412.32,0.134747,0.051133,0.400000,2946.0,FreewayI215,FreewayI215,2024,27500,11000.0,0,1000000,SN8,S04-Fal,1.003,11033.000
484198,WFRC_8218,CD Road,3.0,D2,Daily,12412.32,0.134747,0.051133,0.400000,2946.0,FreewayI215,FreewayI215,2024,27500,11000.0,0,1000000,SN8,MaxMonth,1.025,11275.000


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M01-Jan,0.885,1554.060
1,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M02-Feb,0.954,1675.224
2,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M03-Mar,1.001,1757.756
3,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M04-Apr,1.010,1773.560
4,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M05-May,1.029,1806.924
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
489775,WFRC_8219,Arterial,2.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,1878.0,Arterial,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,S02-Spr,1.017,2522.160
489776,WFRC_8219,Arterial,2.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,1878.0,Arterial,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,S03-Sum,1.057,2621.360
489777,WFRC_8219,Arterial,2.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,1878.0,Arterial,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,S04-Fal,1.022,2534.560
489778,WFRC_8219,Arterial,2.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,1878.0,Arterial,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,MaxMonth,1.067,2646.160


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M01-Jan,0.885,1667.340
1,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M02-Feb,0.954,1797.336
2,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M03-Mar,1.001,1885.884
3,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M04-Apr,1.010,1902.840
4,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M05-May,1.029,1938.636
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495715,WFRC_8219,Arterial,2.0,D2,Daily,1974.12,0.094239,0.040403,0.400000,1878.0,Arterial,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,S02-Spr,1.017,2603.520
495716,WFRC_8219,Arterial,2.0,D2,Daily,1974.12,0.094239,0.040403,0.400000,1878.0,Arterial,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,S03-Sum,1.057,2705.920
495717,WFRC_8219,Arterial,2.0,D2,Daily,1974.12,0.094239,0.040403,0.400000,1878.0,Arterial,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,S04-Fal,1.022,2616.320
495718,WFRC_8219,Arterial,2.0,D2,Daily,1974.12,0.094239,0.040403,0.400000,1878.0,Arterial,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,MaxMonth,1.067,2731.520


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.09037,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M01-Jan,0.885,1639.905
1,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.09037,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M02-Feb,0.954,1767.762
2,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.09037,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M03-Mar,1.001,1854.853
3,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.09037,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M04-Apr,1.010,1871.530
4,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.09037,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M05-May,1.029,1906.737
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506335,WFRC_8219,Arterial,2.0,D2,Daily,5245.32,0.156581,0.058498,0.40000,1878.0,Arterial,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,S02-Spr,1.017,5491.800
506336,WFRC_8219,Arterial,2.0,D2,Daily,5245.32,0.156581,0.058498,0.40000,1878.0,Arterial,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,S03-Sum,1.057,5707.800
506337,WFRC_8219,Arterial,2.0,D2,Daily,5245.32,0.156581,0.058498,0.40000,1878.0,Arterial,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,S04-Fal,1.022,5518.800
506338,WFRC_8219,Arterial,2.0,D2,Daily,5245.32,0.156581,0.058498,0.40000,1878.0,Arterial,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,MaxMonth,1.067,5761.800


...

,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M01-Jan,0.885,1745.220
1,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M02-Feb,0.954,1881.288
2,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M03-Mar,1.001,1973.972
3,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M04-Apr,1.010,1991.720
4,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M05-May,1.029,2029.188
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478795,WFRC_8218,CD Road,3.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,2946.0,FreewayI215,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,S02-Spr,1.015,13981.625
478796,WFRC_8218,CD Road,3.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,2946.0,FreewayI215,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,S03-Sum,1.012,13940.300
478797,WFRC_8218,CD Road,3.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,2946.0,FreewayI215,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,S04-Fal,1.003,13816.325
478798,WFRC_8218,CD Road,3.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,2946.0,FreewayI215,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,MaxMonth,1.025,14119.375


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1366.5,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M01-Jan,0.885,1776.195
1,0013_000.0,Arterial,3.0,D1,AM,1366.5,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M02-Feb,0.954,1914.678
2,0013_000.0,Arterial,3.0,D1,AM,1366.5,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M03-Mar,1.001,2009.007
3,0013_000.0,Arterial,3.0,D1,AM,1366.5,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M04-Apr,1.010,2027.070
4,0013_000.0,Arterial,3.0,D1,AM,1366.5,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M05-May,1.029,2065.203
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
475915,WFRC_8218,CD Road,3.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,2946.0,FreewayI215,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,S02-Spr,1.015,15245.300
475916,WFRC_8218,CD Road,3.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,2946.0,FreewayI215,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,S03-Sum,1.012,15200.240
475917,WFRC_8218,CD Road,3.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,2946.0,FreewayI215,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,S04-Fal,1.003,15065.060
475918,WFRC_8218,CD Road,3.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,2946.0,FreewayI215,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,MaxMonth,1.025,15395.500


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M01-Jan,0.885,1547.865
1,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M02-Feb,0.954,1668.546
2,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M03-Mar,1.001,1750.749
3,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M04-Apr,1.010,1766.490
4,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M05-May,1.029,1799.721
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
490315,WFRC_8219,Arterial,2.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,1878.0,Arterial,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,S02-Spr,1.017,2442.834
490316,WFRC_8219,Arterial,2.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,1878.0,Arterial,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,S03-Sum,1.057,2538.914
490317,WFRC_8219,Arterial,2.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,1878.0,Arterial,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,S04-Fal,1.022,2454.844
490318,WFRC_8219,Arterial,2.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,1878.0,Arterial,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,MaxMonth,1.067,2562.934


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.09153,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M01-Jan,0.885,1659.375
1,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.09153,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M02-Feb,0.954,1788.750
2,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.09153,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M03-Mar,1.001,1876.875
3,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.09153,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M04-Apr,1.010,1893.750
4,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.09153,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M05-May,1.029,1929.375
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
496435,WFRC_8219,Arterial,2.0,D2,Daily,1982.12,0.092648,0.037374,0.40000,1878.0,Arterial,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,S02-Spr,1.017,2611.656
496436,WFRC_8219,Arterial,2.0,D2,Daily,1982.12,0.092648,0.037374,0.40000,1878.0,Arterial,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,S03-Sum,1.057,2714.376
496437,WFRC_8219,Arterial,2.0,D2,Daily,1982.12,0.092648,0.037374,0.40000,1878.0,Arterial,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,S04-Fal,1.022,2624.496
496438,WFRC_8219,Arterial,2.0,D2,Daily,1982.12,0.092648,0.037374,0.40000,1878.0,Arterial,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,MaxMonth,1.067,2740.056


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M01-Jan,0.885,1625.745
1,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M02-Feb,0.954,1752.498
2,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M03-Mar,1.001,1838.837
3,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M04-Apr,1.010,1855.370
4,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M05-May,1.029,1890.273
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
506695,WFRC_8219,Arterial,2.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,1878.0,Arterial,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,S02-Spr,1.017,5593.500
506696,WFRC_8219,Arterial,2.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,1878.0,Arterial,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,S03-Sum,1.057,5813.500
506697,WFRC_8219,Arterial,2.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,1878.0,Arterial,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,S04-Fal,1.022,5621.000
506698,WFRC_8219,Arterial,2.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,1878.0,Arterial,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,MaxMonth,1.067,5868.500


In [179]:
#add dow factor

dfs_PeriodForecasts_Season_DOW = []

#for (idx, row) in dfs_PeriodForecasts.iterrows():

for df_PeriodForecasts_Season in dfs_PeriodForecasts_Season: 
    
    #joing forecasts data with season factors
    df_PeriodForecasts_Season_DOW = pd.DataFrame.merge(df_PeriodForecasts_Season,df_DOWFactors,on=('DOWFACFC',nameFG,nameAT),how='left')
    
    #calculate Average Season DOW
    df_PeriodForecasts_Season_DOW['ForecastASeasonDOWPeriodT'] = df_PeriodForecasts_Season_DOW['ForecastASeasonPeriodT'] *df_PeriodForecasts_Season_DOW['DOWFactor'] 
    
    display(df_PeriodForecasts_Season_DOW)
    
    dfs_PeriodForecasts_Season_DOW.append(df_PeriodForecasts_Season_DOW)

,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M01-Jan,0.885,1479.72,WD2,1-Monday,1.083,1602.53676
1,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M01-Jan,0.885,1479.72,WD2,2-Tuesday,1.116,1651.36752
2,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M01-Jan,0.885,1479.72,WD2,3-Wednesday,1.129,1670.60388
3,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M01-Jan,0.885,1479.72,WD2,4-Thursday,1.137,1682.44164
4,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M01-Jan,0.885,1479.72,WD2,5-Friday,1.146,1695.75912
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4155295,WFRC_8218,CD Road,3.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,2946.0,FreewayI215,FreewayI215,2019,25500,10200.0,0,1000000,SN8,MaxSeason,1.015,10353.00,WDB,5-Friday,1.154,11947.36200
4155296,WFRC_8218,CD Road,3.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,2946.0,FreewayI215,FreewayI215,2019,25500,10200.0,0,1000000,SN8,MaxSeason,1.015,10353.00,WDB,6-Saturday,0.788,8158.16400
4155297,WFRC_8218,CD Road,3.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,2946.0,FreewayI215,FreewayI215,2019,25500,10200.0,0,1000000,SN8,MaxSeason,1.015,10353.00,WDB,7-Sunday,0.576,5963.32800
4155298,WFRC_8218,CD Road,3.0,D2,Daily,11452.72,0.129960,0.049752,0.400000,2946.0,FreewayI215,FreewayI215,2019,25500,10200.0,0,1000000,SN8,MaxSeason,1.015,10353.00,WDB,W1-Weekday,1.123,11626.41900


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M01-Jan,0.885,1538.13,WD2,1-Monday,1.083,1665.79479
1,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M01-Jan,0.885,1538.13,WD2,2-Tuesday,1.116,1716.55308
2,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M01-Jan,0.885,1538.13,WD2,3-Wednesday,1.129,1736.54877
3,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M01-Jan,0.885,1538.13,WD2,4-Thursday,1.137,1748.85381
4,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M01-Jan,0.885,1538.13,WD2,5-Friday,1.146,1762.69698
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4211995,WFRC_8218,CD Road,3.0,D2,Daily,12412.32,0.134747,0.051133,0.400000,2946.0,FreewayI215,FreewayI215,2024,27500,11000.0,0,1000000,SN8,MaxSeason,1.015,11165.00,WDB,5-Friday,1.154,12884.41000
4211996,WFRC_8218,CD Road,3.0,D2,Daily,12412.32,0.134747,0.051133,0.400000,2946.0,FreewayI215,FreewayI215,2024,27500,11000.0,0,1000000,SN8,MaxSeason,1.015,11165.00,WDB,6-Saturday,0.788,8798.02000
4211997,WFRC_8218,CD Road,3.0,D2,Daily,12412.32,0.134747,0.051133,0.400000,2946.0,FreewayI215,FreewayI215,2024,27500,11000.0,0,1000000,SN8,MaxSeason,1.015,11165.00,WDB,7-Sunday,0.576,6431.04000
4211998,WFRC_8218,CD Road,3.0,D2,Daily,12412.32,0.134747,0.051133,0.400000,2946.0,FreewayI215,FreewayI215,2024,27500,11000.0,0,1000000,SN8,MaxSeason,1.015,11165.00,WDB,W1-Weekday,1.123,12538.29500


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M01-Jan,0.885,1554.06,WD2,1-Monday,1.083,1683.04698
1,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M01-Jan,0.885,1554.06,WD2,2-Tuesday,1.116,1734.33096
2,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M01-Jan,0.885,1554.06,WD2,3-Wednesday,1.129,1754.53374
3,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M01-Jan,0.885,1554.06,WD2,4-Thursday,1.137,1766.96622
4,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M01-Jan,0.885,1554.06,WD2,5-Friday,1.146,1780.95276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4291375,WFRC_8219,Arterial,2.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,1878.0,Arterial,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,MaxSeason,1.057,2621.36,WD1,5-Friday,1.142,2993.59312
4291376,WFRC_8219,Arterial,2.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,1878.0,Arterial,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,MaxSeason,1.057,2621.36,WD1,6-Saturday,1.036,2715.72896
4291377,WFRC_8219,Arterial,2.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,1878.0,Arterial,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,MaxSeason,1.057,2621.36,WD1,7-Sunday,0.773,2026.31128
4291378,WFRC_8219,Arterial,2.0,D2,Daily,1885.52,0.091837,0.039162,0.400000,1878.0,Arterial,StatewideArterial,2030,6200,2480.0,0,1000000,SNS,MaxSeason,1.057,2621.36,WD1,W1-Weekday,1.012,2652.81632


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M01-Jan,0.885,1667.34,WD2,1-Monday,1.083,1805.72922
1,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M01-Jan,0.885,1667.34,WD2,2-Tuesday,1.116,1860.75144
2,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M01-Jan,0.885,1667.34,WD2,3-Wednesday,1.129,1882.42686
3,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M01-Jan,0.885,1667.34,WD2,4-Thursday,1.137,1895.76558
4,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M01-Jan,0.885,1667.34,WD2,5-Friday,1.146,1910.77164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4364275,WFRC_8219,Arterial,2.0,D2,Daily,1974.12,0.094239,0.040403,0.400000,1878.0,Arterial,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,MaxSeason,1.057,2705.92,WD1,5-Friday,1.142,3090.16064
4364276,WFRC_8219,Arterial,2.0,D2,Daily,1974.12,0.094239,0.040403,0.400000,1878.0,Arterial,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,MaxSeason,1.057,2705.92,WD1,6-Saturday,1.036,2803.33312
4364277,WFRC_8219,Arterial,2.0,D2,Daily,1974.12,0.094239,0.040403,0.400000,1878.0,Arterial,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,MaxSeason,1.057,2705.92,WD1,7-Sunday,0.773,2091.67616
4364278,WFRC_8219,Arterial,2.0,D2,Daily,1974.12,0.094239,0.040403,0.400000,1878.0,Arterial,StatewideArterial,2040,6400,2560.0,0,1000000,SNS,MaxSeason,1.057,2705.92,WD1,W1-Weekday,1.012,2738.39104


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.09037,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M01-Jan,0.885,1639.905,WD2,1-Monday,1.083,1776.017115
1,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.09037,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M01-Jan,0.885,1639.905,WD2,2-Tuesday,1.116,1830.133980
2,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.09037,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M01-Jan,0.885,1639.905,WD2,3-Wednesday,1.129,1851.452745
3,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.09037,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M01-Jan,0.885,1639.905,WD2,4-Thursday,1.137,1864.571985
4,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.09037,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M01-Jan,0.885,1639.905,WD2,5-Friday,1.146,1879.331130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4466335,WFRC_8219,Arterial,2.0,D2,Daily,5245.32,0.156581,0.058498,0.40000,1878.0,Arterial,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,MaxSeason,1.057,5707.800,WD1,5-Friday,1.142,6518.307600
4466336,WFRC_8219,Arterial,2.0,D2,Daily,5245.32,0.156581,0.058498,0.40000,1878.0,Arterial,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,MaxSeason,1.057,5707.800,WD1,6-Saturday,1.036,5913.280800
4466337,WFRC_8219,Arterial,2.0,D2,Daily,5245.32,0.156581,0.058498,0.40000,1878.0,Arterial,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,MaxSeason,1.057,5707.800,WD1,7-Sunday,0.773,4412.129400
4466338,WFRC_8219,Arterial,2.0,D2,Daily,5245.32,0.156581,0.058498,0.40000,1878.0,Arterial,StatewideArterial,2050,13500,5400.0,0,1000000,SNS,MaxSeason,1.057,5707.800,WD1,W1-Weekday,1.012,5776.293600


...

,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M01-Jan,0.885,1745.220,WD2,1-Monday,1.083,1890.073260
1,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M01-Jan,0.885,1745.220,WD2,2-Tuesday,1.116,1947.665520
2,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M01-Jan,0.885,1745.220,WD2,3-Wednesday,1.129,1970.353380
3,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M01-Jan,0.885,1745.220,WD2,4-Thursday,1.137,1984.315140
4,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M01-Jan,0.885,1745.220,WD2,5-Friday,1.146,2000.022120
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4210375,WFRC_8218,CD Road,3.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,2946.0,FreewayI215,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,MaxSeason,1.015,13981.625,WDB,5-Friday,1.154,16134.795250
4210376,WFRC_8218,CD Road,3.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,2946.0,FreewayI215,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,MaxSeason,1.015,13981.625,WDB,6-Saturday,0.788,11017.520500
4210377,WFRC_8218,CD Road,3.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,2946.0,FreewayI215,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,MaxSeason,1.015,13981.625,WDB,7-Sunday,0.576,8053.416000
4210378,WFRC_8218,CD Road,3.0,D2,Daily,15402.88,0.134403,0.053395,0.400000,2946.0,FreewayI215,FreewayI215,2040,34437.0,13775.0,0,1000000,SN8,MaxSeason,1.015,13981.625,WDB,W1-Weekday,1.123,15701.364875


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1366.5,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M01-Jan,0.885,1776.195,WD2,1-Monday,1.083,1923.619185
1,0013_000.0,Arterial,3.0,D1,AM,1366.5,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M01-Jan,0.885,1776.195,WD2,2-Tuesday,1.116,1982.233620
2,0013_000.0,Arterial,3.0,D1,AM,1366.5,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M01-Jan,0.885,1776.195,WD2,3-Wednesday,1.129,2005.324155
3,0013_000.0,Arterial,3.0,D1,AM,1366.5,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M01-Jan,0.885,1776.195,WD2,4-Thursday,1.137,2019.533715
4,0013_000.0,Arterial,3.0,D1,AM,1366.5,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M01-Jan,0.885,1776.195,WD2,5-Friday,1.146,2035.519470
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4210375,WFRC_8218,CD Road,3.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,2946.0,FreewayI215,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,MaxSeason,1.015,15245.300,WDB,5-Friday,1.154,17593.076200
4210376,WFRC_8218,CD Road,3.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,2946.0,FreewayI215,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,MaxSeason,1.015,15245.300,WDB,6-Saturday,0.788,12013.296400
4210377,WFRC_8218,CD Road,3.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,2946.0,FreewayI215,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,MaxSeason,1.015,15245.300,WDB,7-Sunday,0.576,8781.292800
4210378,WFRC_8218,CD Road,3.0,D2,Daily,16987.6,0.133780,0.053354,0.400000,2946.0,FreewayI215,FreewayI215,2050,37550.0,15020.0,0,1000000,SN8,MaxSeason,1.015,15245.300,WDB,W1-Weekday,1.123,17120.471900


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M01-Jan,0.885,1547.865,WD2,1-Monday,1.083,1676.337795
1,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M01-Jan,0.885,1547.865,WD2,2-Tuesday,1.116,1727.417340
2,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M01-Jan,0.885,1547.865,WD2,3-Wednesday,1.129,1747.539585
3,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M01-Jan,0.885,1547.865,WD2,4-Thursday,1.137,1759.922505
4,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M01-Jan,0.885,1547.865,WD2,5-Friday,1.146,1773.853290
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4299475,WFRC_8219,Arterial,2.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,1878.0,Arterial,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,MaxSeason,1.057,2538.914,WD1,5-Friday,1.142,2899.439788
4299476,WFRC_8219,Arterial,2.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,1878.0,Arterial,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,MaxSeason,1.057,2538.914,WD1,6-Saturday,1.036,2630.314904
4299477,WFRC_8219,Arterial,2.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,1878.0,Arterial,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,MaxSeason,1.057,2538.914,WD1,7-Sunday,0.773,1962.580522
4299478,WFRC_8219,Arterial,2.0,D2,Daily,1807.68,0.087803,0.038834,0.400000,1878.0,Arterial,StatewideArterial,2030,6005.4,2402.0,0,1000000,SNS,MaxSeason,1.057,2538.914,WD1,W1-Weekday,1.012,2569.380968


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.09153,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M01-Jan,0.885,1659.375,WD2,1-Monday,1.083,1797.103125
1,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.09153,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M01-Jan,0.885,1659.375,WD2,2-Tuesday,1.116,1851.862500
2,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.09153,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M01-Jan,0.885,1659.375,WD2,3-Wednesday,1.129,1873.434375
3,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.09153,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M01-Jan,0.885,1659.375,WD2,4-Thursday,1.137,1886.709375
4,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.09153,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M01-Jan,0.885,1659.375,WD2,5-Friday,1.146,1901.643750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4364275,WFRC_8219,Arterial,2.0,D2,Daily,1982.12,0.092648,0.037374,0.40000,1878.0,Arterial,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,MaxSeason,1.057,2714.376,WD1,5-Friday,1.142,3099.817392
4364276,WFRC_8219,Arterial,2.0,D2,Daily,1982.12,0.092648,0.037374,0.40000,1878.0,Arterial,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,MaxSeason,1.057,2714.376,WD1,6-Saturday,1.036,2812.093536
4364277,WFRC_8219,Arterial,2.0,D2,Daily,1982.12,0.092648,0.037374,0.40000,1878.0,Arterial,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,MaxSeason,1.057,2714.376,WD1,7-Sunday,0.773,2098.212648
4364278,WFRC_8219,Arterial,2.0,D2,Daily,1982.12,0.092648,0.037374,0.40000,1878.0,Arterial,StatewideArterial,2040,6420.0,2568.0,0,1000000,SNS,MaxSeason,1.057,2714.376,WD1,W1-Weekday,1.012,2746.948512


,SEGID,FUNCGROUP,AREATYPE,Dir,Period,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT
0,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M01-Jan,0.885,1625.745,WD2,1-Monday,1.083,1760.681835
1,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M01-Jan,0.885,1625.745,WD2,2-Tuesday,1.116,1814.331420
2,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M01-Jan,0.885,1625.745,WD2,3-Wednesday,1.129,1835.466105
3,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M01-Jan,0.885,1625.745,WD2,4-Thursday,1.137,1848.472065
4,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M01-Jan,0.885,1625.745,WD2,5-Friday,1.146,1863.103770
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4466335,WFRC_8219,Arterial,2.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,1878.0,Arterial,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,MaxSeason,1.057,5813.500,WD1,5-Friday,1.142,6639.017000
4466336,WFRC_8219,Arterial,2.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,1878.0,Arterial,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,MaxSeason,1.057,5813.500,WD1,6-Saturday,1.036,6022.786000
4466337,WFRC_8219,Arterial,2.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,1878.0,Arterial,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,MaxSeason,1.057,5813.500,WD1,7-Sunday,0.773,4493.835500
4466338,WFRC_8219,Arterial,2.0,D2,Daily,5345.56,0.149462,0.051632,0.400000,1878.0,Arterial,StatewideArterial,2050,13750.6,5500.0,0,1000000,SNS,MaxSeason,1.057,5813.500,WD1,W1-Weekday,1.012,5883.262000


In [180]:
#add tod factor

dfs_PeakHourForecasts = []

#for (idx, row) in dfs_PeriodForecasts.iterrows():

for df_PeriodForecasts_Season_DOW in dfs_PeriodForecasts_Season_DOW: 
    
    df_PeriodForecasts_Season_DOW = df_PeriodForecasts_Season_DOW.rename(columns={namePeriod: namePeriod + '_TDM'})
    #joing forecasts data with season factors
    
    df_TODFactors_Period = df_TODFactors[df_TODFactors['HourPercentOf'] == 'Period']
    df_TODFactors_Daily  = df_TODFactors[df_TODFactors['HourPercentOf'] == 'Daily']
    
    #get peak hour as a percent of period
    df_PeakHourForecasts_Period = pd.DataFrame.merge(df_PeriodForecasts_Season_DOW,df_TODFactors_Period,left_on=('SiteGroupSeason','SeasonGroup','DOWGroup',namePeriod+'_TDM'),right_on=('SiteGroupSeason','SeasonGroup','DOWGroup',namePeriod))
    
    #get peak hour as a percent of daily
    df_PeakHourForecasts_Daily = pd.DataFrame.merge(df_PeriodForecasts_Season_DOW,df_TODFactors_Daily,left_on=('SiteGroupSeason','SeasonGroup','DOWGroup',namePeriod+'_TDM'),right_on=('SiteGroupSeason','SeasonGroup','DOWGroup','HourPercentOf'))
 
    df_PeakHourForecasts = pd.concat([df_PeakHourForecasts_Period, df_PeakHourForecasts_Daily])

    #display(df_PeriodForecasts_Season_DOW_TOD)

    df_PeakHourForecasts['ForecastPeakHour'] = df_PeakHourForecasts['ForecastASeasonDOWPeriodT'] * df_PeakHourForecasts['VolMaxHourPercent']

    #add truck percentage adjustments
    #PCE for medium and heavy trucks from HCM 2000 Exhibit 23-8
    Emd = 1.2 #same as RVs
    Ehv = 1.5
    df_PeakHourForecasts['FactorTrucks'] = 1 + (df_PeakHourForecasts[namePercentTrucksMD] * (Emd-1)) + (df_PeakHourForecasts[namePercentTrucksHV] * (Ehv-1))
    
    #calculate Average Season DOW
    df_PeakHourForecasts['ForecastPeakHourPCE'] = df_PeakHourForecasts['ForecastPeakHour'] * df_PeakHourForecasts['FactorTrucks']
    
    display(df_PeakHourForecasts)
    
    dfs_PeakHourForecasts.append(df_PeakHourForecasts)

,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M01-Jan,0.885,1479.720,WD2,1-Monday,1.083,1602.536760,AM,Period,0.408955,655.365266,1.061055,695.378478
1,0013_000.0,Arterial,3.0,D2,AM,782.40,0.207183,0.090874,0.065183,3716.0,Arterial,StatewideArterial,2019,19000,1238.0,0,1000000,SNT,M01-Jan,0.885,1095.630,WD2,1-Monday,1.083,1186.567290,AM,Period,0.408955,485.252512,1.086874,527.408204
2,0013_000.6,Arterial,3.0,D1,AM,1174.90,0.119329,0.056260,0.081416,3716.0,Arterial,StatewideArterial,2019,19000,1547.0,0,1000000,SNT,M01-Jan,0.885,1369.095,WD2,1-Monday,1.083,1482.729885,AM,Period,0.408955,606.369657,1.051996,637.898402
3,0013_000.6,Arterial,3.0,D2,AM,1186.30,0.125516,0.057658,0.082206,3716.0,Arterial,StatewideArterial,2019,19000,1562.0,0,1000000,SNT,M01-Jan,0.885,1382.370,WD2,1-Monday,1.083,1497.106710,AM,Period,0.408955,612.249130,1.053932,645.269192
4,0013_001.3,Arterial,3.0,D1,AM,1419.90,0.112261,0.060638,0.084827,3716.0,Arterial,StatewideArterial,2019,15500,1315.0,0,1000000,SNT,M01-Jan,0.885,1163.775,WD2,1-Monday,1.083,1260.368325,AM,Period,0.408955,515.433807,1.052771,542.633930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2537467,1084_001.0,Arterial,1.0,D1,Daily,548.52,0.218442,0.123824,0.600000,1144.0,Arterial,RuralRecLowVol,2019,350,210.0,0,1000000,SNL,S04-Fal,0.926,194.460,WD0,W1-Weekday,0.938,182.403480,PM,Daily,0.094372,17.213811,1.105601,19.031598
2537468,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,AM,Daily,0.051266,6.234092,1.105601,6.892415
2537469,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,EV,Daily,0.064197,7.806536,1.105601,8.630910
2537470,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,MD,Daily,0.077305,9.400466,1.105601,10.393160


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M01-Jan,0.885,1538.130,WD2,1-Monday,1.083,1665.794790,AM,Period,0.408955,681.234948,1.062608,723.885757
1,0013_000.0,Arterial,3.0,D2,AM,823.10,0.209209,0.095007,0.065398,3716.0,Arterial,StatewideArterial,2024,19500,1275.0,0,1000000,SNT,M01-Jan,0.885,1128.375,WD2,1-Monday,1.083,1222.030125,AM,Period,0.408955,499.755212,1.089345,544.405921
2,0013_000.6,Arterial,3.0,D1,AM,1213.30,0.122311,0.057364,0.086046,3716.0,Arterial,StatewideArterial,2024,19500,1678.0,0,1000000,SNT,M01-Jan,0.885,1485.030,WD2,1-Monday,1.083,1608.287490,AM,Period,0.408955,657.717056,1.053144,692.670979
3,0013_000.6,Arterial,3.0,D2,AM,1105.50,0.140299,0.064858,0.078401,3716.0,Arterial,StatewideArterial,2024,19500,1529.0,0,1000000,SNT,M01-Jan,0.885,1353.165,WD2,1-Monday,1.083,1465.477695,AM,Period,0.408955,599.314290,1.060488,635.565892
4,0013_001.8,Arterial,3.0,D1,AM,844.20,0.120232,0.072613,0.072599,3680.0,Arterial,StatewideArterial,2024,17000,1234.0,0,1000000,SNT,M01-Jan,0.885,1092.090,WD2,1-Monday,1.083,1182.733470,AM,Period,0.408955,483.684652,1.060353,512.876471
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2562043,1084_001.0,Arterial,1.0,D1,Daily,530.52,0.245646,0.139222,0.600000,1144.0,Arterial,RuralRecLowVol,2024,400,240.0,0,1000000,SNL,S04-Fal,0.926,222.240,WD0,W1-Weekday,0.938,208.461120,PM,Daily,0.094372,19.672927,1.118740,22.008892
2562044,1084_001.0,Arterial,1.0,D2,Daily,353.68,0.245646,0.139222,0.400000,1144.0,Arterial,RuralRecLowVol,2024,400,160.0,0,1000000,SNL,S04-Fal,0.926,148.160,WD0,W1-Weekday,0.938,138.974080,AM,Daily,0.051266,7.124677,1.118740,7.970661
2562045,1084_001.0,Arterial,1.0,D2,Daily,353.68,0.245646,0.139222,0.400000,1144.0,Arterial,RuralRecLowVol,2024,400,160.0,0,1000000,SNL,S04-Fal,0.926,148.160,WD0,W1-Weekday,0.938,138.974080,EV,Daily,0.064197,8.921755,1.118740,9.981125
2562046,1084_001.0,Arterial,1.0,D2,Daily,353.68,0.245646,0.139222,0.400000,1144.0,Arterial,RuralRecLowVol,2024,400,160.0,0,1000000,SNL,S04-Fal,0.926,148.160,WD0,W1-Weekday,0.938,138.974080,MD,Daily,0.077305,10.743390,1.118740,12.019061


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M01-Jan,0.885,1554.060,WD2,1-Monday,1.083,1683.046980,AM,Period,0.408955,688.290316,1.063784,732.192242
1,0013_000.0,Arterial,3.0,D2,AM,841.00,0.208442,0.094293,0.065917,3716.0,Arterial,StatewideArterial,2030,20000,1318.0,0,1000000,SNT,M01-Jan,0.885,1166.430,WD2,1-Monday,1.083,1263.243690,AM,Period,0.408955,516.609702,1.088835,562.502580
2,0013_000.6,Arterial,3.0,D1,AM,1203.80,0.125685,0.058232,0.085248,3716.0,Arterial,StatewideArterial,2030,19500,1662.0,0,1000000,SNT,M01-Jan,0.885,1470.870,WD2,1-Monday,1.083,1592.952210,AM,Period,0.408955,651.445618,1.054253,686.788626
3,0013_000.6,Arterial,3.0,D2,AM,1111.40,0.141623,0.065323,0.078704,3716.0,Arterial,StatewideArterial,2030,19500,1535.0,0,1000000,SNT,M01-Jan,0.885,1358.475,WD2,1-Monday,1.083,1471.228425,AM,Period,0.408955,601.666079,1.060986,638.359373
4,0013_001.8,Arterial,3.0,D1,AM,845.70,0.120137,0.074258,0.071745,3680.0,Arterial,StatewideArterial,2030,17000,1220.0,0,1000000,SNT,M01-Jan,0.885,1079.700,WD2,1-Monday,1.083,1169.315100,AM,Period,0.408955,478.197144,1.061156,507.441978
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2586619,1084_001.0,Arterial,1.0,D1,Daily,467.76,0.273345,0.161878,0.600000,1144.0,Arterial,RuralRecLowVol,2030,500,300.0,0,1000000,SNL,S04-Fal,0.926,277.800,WD0,W1-Weekday,0.938,260.576400,PM,Daily,0.094372,24.591158,1.135608,27.925916
2586620,1084_001.0,Arterial,1.0,D2,Daily,311.84,0.273345,0.161878,0.400000,1144.0,Arterial,RuralRecLowVol,2030,500,200.0,0,1000000,SNL,S04-Fal,0.926,185.200,WD0,W1-Weekday,0.938,173.717600,AM,Daily,0.051266,8.905846,1.135608,10.113550
2586621,1084_001.0,Arterial,1.0,D2,Daily,311.84,0.273345,0.161878,0.400000,1144.0,Arterial,RuralRecLowVol,2030,500,200.0,0,1000000,SNL,S04-Fal,0.926,185.200,WD0,W1-Weekday,0.938,173.717600,EV,Daily,0.064197,11.152194,1.135608,12.664521
2586622,1084_001.0,Arterial,1.0,D2,Daily,311.84,0.273345,0.161878,0.400000,1144.0,Arterial,RuralRecLowVol,2030,500,200.0,0,1000000,SNL,S04-Fal,0.926,185.200,WD0,W1-Weekday,0.938,173.717600,MD,Daily,0.077305,13.429237,1.135608,15.250349


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M01-Jan,0.885,1667.340,WD2,1-Monday,1.083,1805.729220,AM,Period,0.408955,738.461819,1.057934,781.244227
1,0013_000.0,Arterial,3.0,D2,AM,846.80,0.205598,0.086679,0.063880,3716.0,Arterial,StatewideArterial,2040,20500,1310.0,0,1000000,SNT,M01-Jan,0.885,1159.350,WD2,1-Monday,1.083,1255.576050,AM,Period,0.408955,513.473983,1.084459,556.841554
2,0013_000.6,Arterial,3.0,D1,AM,1314.50,0.118600,0.051350,0.091021,3716.0,Arterial,StatewideArterial,2040,20000,1820.0,0,1000000,SNT,M01-Jan,0.885,1610.700,WD2,1-Monday,1.083,1744.388100,AM,Period,0.408955,713.376068,1.049395,748.613426
3,0013_000.6,Arterial,3.0,D2,AM,1076.00,0.144610,0.063290,0.074506,3716.0,Arterial,StatewideArterial,2040,20000,1490.0,0,1000000,SNT,M01-Jan,0.885,1318.650,WD2,1-Monday,1.083,1428.097950,AM,Period,0.408955,584.027660,1.060567,619.400413
4,0013_001.8,Arterial,3.0,D1,AM,894.00,0.124720,0.076286,0.073113,3680.0,Arterial,StatewideArterial,2040,17500,1279.0,0,1000000,SNT,M01-Jan,0.885,1131.915,WD2,1-Monday,1.083,1225.863945,AM,Period,0.408955,501.323072,1.063087,532.950165
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2603003,1084_001.0,Arterial,1.0,D1,Daily,535.56,0.285010,0.222832,0.600000,1144.0,Arterial,RuralRecLowVol,2040,700,420.0,0,1000000,SNL,S04-Fal,0.926,388.920,WD0,W1-Weekday,0.938,364.806960,PM,Daily,0.094372,34.427622,1.168418,40.225857
2603004,1084_001.0,Arterial,1.0,D2,Daily,357.04,0.285010,0.222832,0.400000,1144.0,Arterial,RuralRecLowVol,2040,700,280.0,0,1000000,SNL,S04-Fal,0.926,259.280,WD0,W1-Weekday,0.938,243.204640,AM,Daily,0.051266,12.468184,1.168418,14.568052
2603005,1084_001.0,Arterial,1.0,D2,Daily,357.04,0.285010,0.222832,0.400000,1144.0,Arterial,RuralRecLowVol,2040,700,280.0,0,1000000,SNL,S04-Fal,0.926,259.280,WD0,W1-Weekday,0.938,243.204640,EV,Daily,0.064197,15.613072,1.168418,18.242596
2603006,1084_001.0,Arterial,1.0,D2,Daily,357.04,0.285010,0.222832,0.400000,1144.0,Arterial,RuralRecLowVol,2040,700,280.0,0,1000000,SNL,S04-Fal,0.926,259.280,WD0,W1-Weekday,0.938,243.204640,MD,Daily,0.077305,18.800932,1.168418,21.967349


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.090370,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M01-Jan,0.885,1639.905,WD2,1-Monday,1.083,1776.017115,AM,Period,0.408955,726.310908,1.054994,766.253298
1,0013_000.0,Arterial,3.0,D2,AM,867.60,0.210350,0.090134,0.063558,3716.0,Arterial,StatewideArterial,2050,20500,1303.0,0,1000000,SNT,M01-Jan,0.885,1153.155,WD2,1-Monday,1.083,1248.866865,AM,Period,0.408955,510.730229,1.087137,555.233692
2,0013_000.6,Arterial,3.0,D1,AM,1366.50,0.115843,0.051884,0.093377,3716.0,Arterial,StatewideArterial,2050,20000,1868.0,0,1000000,SNT,M01-Jan,0.885,1653.180,WD2,1-Monday,1.083,1790.393940,AM,Period,0.408955,732.190381,1.049111,768.148886
3,0013_000.6,Arterial,3.0,D2,AM,1046.00,0.156883,0.069981,0.071476,3716.0,Arterial,StatewideArterial,2050,20000,1430.0,0,1000000,SNT,M01-Jan,0.885,1265.550,WD2,1-Monday,1.083,1370.590650,AM,Period,0.408955,560.509767,1.066367,597.709182
4,0013_001.8,Arterial,3.0,D1,AM,890.70,0.126417,0.077916,0.073503,3680.0,Arterial,StatewideArterial,2050,17500,1286.0,0,1000000,SNT,M01-Jan,0.885,1138.110,WD2,1-Monday,1.083,1232.573130,AM,Period,0.408955,504.066826,1.064242,536.448889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2641915,1084_001.0,Arterial,1.0,D1,Daily,388.56,0.208616,0.174336,0.600000,1144.0,Arterial,RuralRecLowVol,2050,800,480.0,0,1000000,SNL,S04-Fal,0.926,444.480,WD0,W1-Weekday,0.938,416.922240,PM,Daily,0.094372,39.345854,1.128891,44.417191
2641916,1084_001.0,Arterial,1.0,D2,Daily,259.04,0.208616,0.174336,0.400000,1144.0,Arterial,RuralRecLowVol,2050,800,320.0,0,1000000,SNL,S04-Fal,0.926,296.320,WD0,W1-Weekday,0.938,277.948160,AM,Daily,0.051266,14.249353,1.128891,16.085971
2641917,1084_001.0,Arterial,1.0,D2,Daily,259.04,0.208616,0.174336,0.400000,1144.0,Arterial,RuralRecLowVol,2050,800,320.0,0,1000000,SNL,S04-Fal,0.926,296.320,WD0,W1-Weekday,0.938,277.948160,EV,Daily,0.064197,17.843510,1.128891,20.143384
2641918,1084_001.0,Arterial,1.0,D2,Daily,259.04,0.208616,0.174336,0.400000,1144.0,Arterial,RuralRecLowVol,2050,800,320.0,0,1000000,SNL,S04-Fal,0.926,296.320,WD0,W1-Weekday,0.938,277.948160,MD,Daily,0.077305,21.486780,1.128891,24.256238


...

,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M01-Jan,0.885,1745.220,WD2,1-Monday,1.083,1890.073260,AM,Period,0.408955,772.954728,1.058905,818.485915
1,0013_000.0,Arterial,3.0,D2,AM,902.30,0.209797,0.094093,0.064265,3716.0,Arterial,StatewideArterial,2040,21284.2,1368.0,0,1000000,SNT,M01-Jan,0.885,1210.680,WD2,1-Monday,1.083,1311.166440,AM,Period,0.408955,536.207945,1.089006,583.933602
2,0013_000.6,Arterial,3.0,D1,AM,1424.50,0.117866,0.055177,0.090718,3716.0,Arterial,StatewideArterial,2040,21260.7,1929.0,0,1000000,SNT,M01-Jan,0.885,1707.165,WD2,1-Monday,1.083,1848.859695,AM,Period,0.408955,756.100239,1.051162,794.783696
3,0013_000.6,Arterial,3.0,D2,AM,1167.20,0.148475,0.069225,0.074332,3716.0,Arterial,StatewideArterial,2040,21260.7,1580.0,0,1000000,SNT,M01-Jan,0.885,1398.300,WD2,1-Monday,1.083,1514.358900,AM,Period,0.408955,619.304498,1.064308,659.130574
4,0013_001.8,Arterial,3.0,D1,AM,960.60,0.124089,0.072247,0.073287,3680.0,Arterial,StatewideArterial,2040,18379.7,1347.0,0,1000000,SNT,M01-Jan,0.885,1192.095,WD2,1-Monday,1.083,1291.038885,AM,Period,0.408955,527.976683,1.060941,560.152151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2558971,1084_001.0,Arterial,1.0,D1,Daily,525.54,0.285992,0.206645,0.600000,1144.0,Arterial,RuralRecLowVol,2040,683.3,410.0,0,1000000,SNL,S04-Fal,0.926,379.660,WD0,W1-Weekday,0.938,356.121080,PM,Daily,0.094372,33.607917,1.160521,39.002680
2558972,1084_001.0,Arterial,1.0,D2,Daily,350.36,0.285992,0.206645,0.400000,1144.0,Arterial,RuralRecLowVol,2040,683.3,273.0,0,1000000,SNL,S04-Fal,0.926,252.798,WD0,W1-Weekday,0.938,237.124524,AM,Daily,0.051266,12.156479,1.160521,14.107845
2558973,1084_001.0,Arterial,1.0,D2,Daily,350.36,0.285992,0.206645,0.400000,1144.0,Arterial,RuralRecLowVol,2040,683.3,273.0,0,1000000,SNL,S04-Fal,0.926,252.798,WD0,W1-Weekday,0.938,237.124524,EV,Daily,0.064197,15.222745,1.160521,17.666309
2558974,1084_001.0,Arterial,1.0,D2,Daily,350.36,0.285992,0.206645,0.400000,1144.0,Arterial,RuralRecLowVol,2040,683.3,273.0,0,1000000,SNL,S04-Fal,0.926,252.798,WD0,W1-Weekday,0.938,237.124524,MD,Daily,0.077305,18.330909,1.160521,21.273397


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1366.50,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M01-Jan,0.885,1776.195,WD2,1-Monday,1.083,1923.619185,AM,Period,0.408955,786.673499,1.054790,829.775032
1,0013_000.0,Arterial,3.0,D2,AM,945.50,0.208778,0.093284,0.064714,3716.0,Arterial,StatewideArterial,2050,21460.0,1389.0,0,1000000,SNT,M01-Jan,0.885,1229.265,WD2,1-Monday,1.083,1331.293995,AM,Period,0.408955,544.439208,1.088398,592.566367
2,0013_000.6,Arterial,3.0,D1,AM,1509.20,0.115359,0.053538,0.091884,3716.0,Arterial,StatewideArterial,2050,21790.9,2002.0,0,1000000,SNT,M01-Jan,0.885,1771.770,WD2,1-Monday,1.083,1918.826910,AM,Period,0.408955,784.713674,1.049841,823.824569
3,0013_000.6,Arterial,3.0,D2,AM,1178.10,0.153467,0.071386,0.071726,3716.0,Arterial,StatewideArterial,2050,21790.9,1563.0,0,1000000,SNT,M01-Jan,0.885,1383.255,WD2,1-Monday,1.083,1498.065165,AM,Period,0.408955,612.641095,1.066387,653.312227
4,0013_001.8,Arterial,3.0,D1,AM,1017.40,0.122862,0.071948,0.073615,3680.0,Arterial,StatewideArterial,2050,19202.7,1414.0,0,1000000,SNT,M01-Jan,0.885,1251.390,WD2,1-Monday,1.083,1355.255370,AM,Period,0.408955,554.238329,1.060546,587.795515
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2544635,1084_001.0,Arterial,1.0,D1,Daily,446.82,0.212837,0.181550,0.600000,1144.0,Arterial,RuralRecLowVol,2050,897.1,538.0,0,1000000,SNL,S04-Fal,0.926,498.188,WD0,W1-Weekday,0.938,467.300344,PM,Daily,0.094372,44.100144,1.133342,49.980558
2544636,1084_001.0,Arterial,1.0,D2,Daily,297.88,0.212837,0.181550,0.400000,1144.0,Arterial,RuralRecLowVol,2050,897.1,359.0,0,1000000,SNL,S04-Fal,0.926,332.434,WD0,W1-Weekday,0.938,311.823092,AM,Daily,0.051266,15.985993,1.133342,18.117602
2544637,1084_001.0,Arterial,1.0,D2,Daily,297.88,0.212837,0.181550,0.400000,1144.0,Arterial,RuralRecLowVol,2050,897.1,359.0,0,1000000,SNL,S04-Fal,0.926,332.434,WD0,W1-Weekday,0.938,311.823092,EV,Daily,0.064197,20.018188,1.133342,22.687459
2544638,1084_001.0,Arterial,1.0,D2,Daily,297.88,0.212837,0.181550,0.400000,1144.0,Arterial,RuralRecLowVol,2050,897.1,359.0,0,1000000,SNL,S04-Fal,0.926,332.434,WD0,W1-Weekday,0.938,311.823092,MD,Daily,0.077305,24.105481,1.133342,27.319761


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M01-Jan,0.885,1547.865,WD2,1-Monday,1.083,1676.337795,AM,Period,0.408955,685.546562,1.063967,729.398838
1,0013_000.0,Arterial,3.0,D2,AM,831.20,0.208855,0.094562,0.065875,3716.0,Arterial,StatewideArterial,2030,19859.5,1308.0,0,1000000,SNT,M01-Jan,0.885,1157.580,WD2,1-Monday,1.083,1253.659140,AM,Period,0.408955,512.690053,1.089052,558.346114
2,0013_000.6,Arterial,3.0,D1,AM,1193.50,0.126183,0.058483,0.085016,3716.0,Arterial,StatewideArterial,2030,19417.3,1651.0,0,1000000,SNT,M01-Jan,0.885,1461.135,WD2,1-Monday,1.083,1582.409205,AM,Period,0.408955,647.134004,1.054478,682.388845
3,0013_000.6,Arterial,3.0,D2,AM,1108.60,0.140718,0.065127,0.078969,3716.0,Arterial,StatewideArterial,2030,19417.3,1533.0,0,1000000,SNT,M01-Jan,0.885,1356.705,WD2,1-Monday,1.083,1469.311515,AM,Period,0.408955,600.882149,1.060707,637.360021
4,0013_001.8,Arterial,3.0,D1,AM,840.70,0.120971,0.074700,0.071485,3680.0,Arterial,StatewideArterial,2030,16972.9,1213.0,0,1000000,SNT,M01-Jan,0.885,1073.505,WD2,1-Monday,1.083,1162.605915,AM,Period,0.408955,475.453390,1.061544,504.714670
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2587643,1084_001.0,Arterial,1.0,D1,Daily,489.24,0.261467,0.155016,0.600000,1144.0,Arterial,RuralRecLowVol,2030,535.8,321.0,0,1000000,SNL,S04-Fal,0.926,297.246,WD0,W1-Weekday,0.938,278.816748,PM,Daily,0.094372,26.312540,1.129801,29.727942
2587644,1084_001.0,Arterial,1.0,D2,Daily,326.16,0.261467,0.155016,0.400000,1144.0,Arterial,RuralRecLowVol,2030,535.8,214.0,0,1000000,SNL,S04-Fal,0.926,198.164,WD0,W1-Weekday,0.938,185.877832,AM,Daily,0.051266,9.529255,1.129801,10.766165
2587645,1084_001.0,Arterial,1.0,D2,Daily,326.16,0.261467,0.155016,0.400000,1144.0,Arterial,RuralRecLowVol,2030,535.8,214.0,0,1000000,SNL,S04-Fal,0.926,198.164,WD0,W1-Weekday,0.938,185.877832,EV,Daily,0.064197,11.932848,1.129801,13.481747
2587646,1084_001.0,Arterial,1.0,D2,Daily,326.16,0.261467,0.155016,0.400000,1144.0,Arterial,RuralRecLowVol,2030,535.8,214.0,0,1000000,SNL,S04-Fal,0.926,198.164,WD0,W1-Weekday,0.938,185.877832,MD,Daily,0.077305,14.369284,1.129801,16.234436


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.091530,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M01-Jan,0.885,1659.375,WD2,1-Monday,1.083,1797.103125,AM,Period,0.408955,734.934135,1.058216,777.719404
1,0013_000.0,Arterial,3.0,D2,AM,849.50,0.204944,0.086404,0.064144,3716.0,Arterial,StatewideArterial,2040,20487.5,1314.0,0,1000000,SNT,M01-Jan,0.885,1162.890,WD2,1-Monday,1.083,1259.409870,AM,Period,0.408955,515.041842,1.084191,558.403576
2,0013_000.6,Arterial,3.0,D1,AM,1309.20,0.119004,0.051635,0.090739,3716.0,Arterial,StatewideArterial,2040,19986.4,1814.0,0,1000000,SNT,M01-Jan,0.885,1605.390,WD2,1-Monday,1.083,1738.637370,AM,Period,0.408955,711.024278,1.049618,746.303943
3,0013_000.6,Arterial,3.0,D2,AM,1079.70,0.144114,0.063166,0.074833,3716.0,Arterial,StatewideArterial,2040,19986.4,1496.0,0,1000000,SNT,M01-Jan,0.885,1323.960,WD2,1-Monday,1.083,1433.848680,AM,Period,0.408955,586.379449,1.060406,621.800091
4,0013_001.8,Arterial,3.0,D1,AM,891.70,0.124818,0.076371,0.072980,3680.0,Arterial,StatewideArterial,2040,17490.7,1276.0,0,1000000,SNT,M01-Jan,0.885,1129.260,WD2,1-Monday,1.083,1222.988580,AM,Period,0.408955,500.147177,1.063149,531.730992
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2597883,1084_001.0,Arterial,1.0,D1,Daily,553.08,0.287047,0.225754,0.600000,1144.0,Arterial,RuralRecLowVol,2040,729.2,438.0,0,1000000,SNL,S04-Fal,0.926,405.588,WD0,W1-Weekday,0.938,380.441544,PM,Daily,0.094372,35.903091,1.170286,42.016899
2597884,1084_001.0,Arterial,1.0,D2,Daily,368.72,0.287047,0.225754,0.400000,1144.0,Arterial,RuralRecLowVol,2040,729.2,292.0,0,1000000,SNL,S04-Fal,0.926,270.392,WD0,W1-Weekday,0.938,253.627696,AM,Daily,0.051266,13.002535,1.170286,15.216690
2597885,1084_001.0,Arterial,1.0,D2,Daily,368.72,0.287047,0.225754,0.400000,1144.0,Arterial,RuralRecLowVol,2040,729.2,292.0,0,1000000,SNL,S04-Fal,0.926,270.392,WD0,W1-Weekday,0.938,253.627696,EV,Daily,0.064197,16.282203,1.170286,19.054841
2597886,1084_001.0,Arterial,1.0,D2,Daily,368.72,0.287047,0.225754,0.400000,1144.0,Arterial,RuralRecLowVol,2040,729.2,292.0,0,1000000,SNL,S04-Fal,0.926,270.392,WD0,W1-Weekday,0.938,253.627696,MD,Daily,0.077305,19.606686,1.170286,22.945438


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M01-Jan,0.885,1625.745,WD2,1-Monday,1.083,1760.681835,AM,Period,0.408955,720.039470,1.054536,759.307396
1,0013_000.0,Arterial,3.0,D2,AM,874.30,0.210111,0.088757,0.063997,3716.0,Arterial,StatewideArterial,2050,20511.0,1313.0,0,1000000,SNT,M01-Jan,0.885,1162.005,WD2,1-Monday,1.083,1258.451415,AM,Period,0.408955,514.649877,1.086401,559.115909
2,0013_000.6,Arterial,3.0,D1,AM,1363.50,0.116392,0.051779,0.093181,3716.0,Arterial,StatewideArterial,2050,19998.6,1863.0,0,1000000,SNT,M01-Jan,0.885,1648.755,WD2,1-Monday,1.083,1785.601665,AM,Period,0.408955,730.230557,1.049168,766.134229
3,0013_000.6,Arterial,3.0,D2,AM,1047.70,0.157297,0.069295,0.071599,3716.0,Arterial,StatewideArterial,2050,19998.6,1432.0,0,1000000,SNT,M01-Jan,0.885,1267.320,WD2,1-Monday,1.083,1372.507560,AM,Period,0.408955,561.293697,1.066107,598.398977
4,0013_001.8,Arterial,3.0,D1,AM,889.40,0.126715,0.078030,0.073434,3680.0,Arterial,StatewideArterial,2050,17493.6,1285.0,0,1000000,SNT,M01-Jan,0.885,1137.225,WD2,1-Monday,1.083,1231.614675,AM,Period,0.408955,503.674861,1.064358,536.090365
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2642939,1084_001.0,Arterial,1.0,D1,Daily,343.56,0.202934,0.162592,0.600000,1144.0,Arterial,RuralRecLowVol,2050,725.0,435.0,0,1000000,SNL,S04-Fal,0.926,402.810,WD0,W1-Weekday,0.938,377.835780,PM,Daily,0.094372,35.657180,1.121883,40.003171
2642940,1084_001.0,Arterial,1.0,D2,Daily,229.04,0.202934,0.162592,0.400000,1144.0,Arterial,RuralRecLowVol,2050,725.0,290.0,0,1000000,SNL,S04-Fal,0.926,268.540,WD0,W1-Weekday,0.938,251.890520,AM,Daily,0.051266,12.913476,1.121883,14.487405
2642941,1084_001.0,Arterial,1.0,D2,Daily,229.04,0.202934,0.162592,0.400000,1144.0,Arterial,RuralRecLowVol,2050,725.0,290.0,0,1000000,SNL,S04-Fal,0.926,268.540,WD0,W1-Weekday,0.938,251.890520,EV,Daily,0.064197,16.170681,1.121883,18.141607
2642942,1084_001.0,Arterial,1.0,D2,Daily,229.04,0.202934,0.162592,0.400000,1144.0,Arterial,RuralRecLowVol,2050,725.0,290.0,0,1000000,SNL,S04-Fal,0.926,268.540,WD0,W1-Weekday,0.938,251.890520,MD,Daily,0.077305,19.472394,1.121883,21.845741


In [181]:
dfs_PeakHourForecasts[0].columns

Index(['SEGID', 'FUNCGROUP', 'AREATYPE', 'Dir', 'Period_TDM', 'VOL',
       'PercentTrucksMD', 'PercentTrucksHV', 'VolPercentSegDY', 'Cap1Hr',
       'DOWFACFC', 'SsnGrp', 'FYEAR', 'ForecastAADT', 'ForecastAAPeriodT',
       'Vol_From', 'Vol_To', 'SiteGroupSeason', 'SeasonGroup', 'SeasonFactor',
       'ForecastASeasonPeriodT', 'SiteGroupDOW', 'DOWGroup', 'DOWFactor',
       'ForecastASeasonDOWPeriodT', 'Period', 'HourPercentOf',
       'VolMaxHourPercent', 'ForecastPeakHour', 'FactorTrucks',
       'ForecastPeakHourPCE'],
      dtype='object')

In [182]:
dfs_PeakHourForecasts_wCapacity = []

for df_PeakHourForecasts in dfs_PeakHourForecasts:

    #calculate peak hour
    df_PeakHourForecasts_wCapacity = pd.DataFrame.merge(df_PeakHourForecasts,df_PeriodDuration,on='Period',how='left')
    df_PeakHourForecasts_wCapacity['VCPeakHourPCE']  = df_PeakHourForecasts_wCapacity['ForecastPeakHourPCE'] / df_PeakHourForecasts_wCapacity[nameCap1Hr]
    
    display(df_PeakHourForecasts_wCapacity)
    
    dfs_PeakHourForecasts_wCapacity.append(df_PeakHourForecasts_wCapacity)

,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M01-Jan,0.885,1479.720,WD2,1-Monday,1.083,1602.536760,AM,Period,0.408955,655.365266,1.061055,695.378478,3,0.187131
1,0013_000.0,Arterial,3.0,D2,AM,782.40,0.207183,0.090874,0.065183,3716.0,Arterial,StatewideArterial,2019,19000,1238.0,0,1000000,SNT,M01-Jan,0.885,1095.630,WD2,1-Monday,1.083,1186.567290,AM,Period,0.408955,485.252512,1.086874,527.408204,3,0.141929
2,0013_000.6,Arterial,3.0,D1,AM,1174.90,0.119329,0.056260,0.081416,3716.0,Arterial,StatewideArterial,2019,19000,1547.0,0,1000000,SNT,M01-Jan,0.885,1369.095,WD2,1-Monday,1.083,1482.729885,AM,Period,0.408955,606.369657,1.051996,637.898402,3,0.171663
3,0013_000.6,Arterial,3.0,D2,AM,1186.30,0.125516,0.057658,0.082206,3716.0,Arterial,StatewideArterial,2019,19000,1562.0,0,1000000,SNT,M01-Jan,0.885,1382.370,WD2,1-Monday,1.083,1497.106710,AM,Period,0.408955,612.249130,1.053932,645.269192,3,0.173646
4,0013_001.3,Arterial,3.0,D1,AM,1419.90,0.112261,0.060638,0.084827,3716.0,Arterial,StatewideArterial,2019,15500,1315.0,0,1000000,SNT,M01-Jan,0.885,1163.775,WD2,1-Monday,1.083,1260.368325,AM,Period,0.408955,515.433807,1.052771,542.633930,3,0.146026
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074939,1084_001.0,Arterial,1.0,D1,Daily,548.52,0.218442,0.123824,0.600000,1144.0,Arterial,RuralRecLowVol,2019,350,210.0,0,1000000,SNL,S04-Fal,0.926,194.460,WD0,W1-Weekday,0.938,182.403480,PM,Daily,0.094372,17.213811,1.105601,19.031598,3,0.016636
5074940,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,AM,Daily,0.051266,6.234092,1.105601,6.892415,3,0.006025
5074941,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,EV,Daily,0.064197,7.806536,1.105601,8.630910,12,0.007545
5074942,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,MD,Daily,0.077305,9.400466,1.105601,10.393160,6,0.009085


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M01-Jan,0.885,1538.130,WD2,1-Monday,1.083,1665.794790,AM,Period,0.408955,681.234948,1.062608,723.885757,3,0.194802
1,0013_000.0,Arterial,3.0,D2,AM,823.10,0.209209,0.095007,0.065398,3716.0,Arterial,StatewideArterial,2024,19500,1275.0,0,1000000,SNT,M01-Jan,0.885,1128.375,WD2,1-Monday,1.083,1222.030125,AM,Period,0.408955,499.755212,1.089345,544.405921,3,0.146503
2,0013_000.6,Arterial,3.0,D1,AM,1213.30,0.122311,0.057364,0.086046,3716.0,Arterial,StatewideArterial,2024,19500,1678.0,0,1000000,SNT,M01-Jan,0.885,1485.030,WD2,1-Monday,1.083,1608.287490,AM,Period,0.408955,657.717056,1.053144,692.670979,3,0.186402
3,0013_000.6,Arterial,3.0,D2,AM,1105.50,0.140299,0.064858,0.078401,3716.0,Arterial,StatewideArterial,2024,19500,1529.0,0,1000000,SNT,M01-Jan,0.885,1353.165,WD2,1-Monday,1.083,1465.477695,AM,Period,0.408955,599.314290,1.060488,635.565892,3,0.171035
4,0013_001.8,Arterial,3.0,D1,AM,844.20,0.120232,0.072613,0.072599,3680.0,Arterial,StatewideArterial,2024,17000,1234.0,0,1000000,SNT,M01-Jan,0.885,1092.090,WD2,1-Monday,1.083,1182.733470,AM,Period,0.408955,483.684652,1.060353,512.876471,3,0.139369
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5124091,1084_001.0,Arterial,1.0,D1,Daily,530.52,0.245646,0.139222,0.600000,1144.0,Arterial,RuralRecLowVol,2024,400,240.0,0,1000000,SNL,S04-Fal,0.926,222.240,WD0,W1-Weekday,0.938,208.461120,PM,Daily,0.094372,19.672927,1.118740,22.008892,3,0.019239
5124092,1084_001.0,Arterial,1.0,D2,Daily,353.68,0.245646,0.139222,0.400000,1144.0,Arterial,RuralRecLowVol,2024,400,160.0,0,1000000,SNL,S04-Fal,0.926,148.160,WD0,W1-Weekday,0.938,138.974080,AM,Daily,0.051266,7.124677,1.118740,7.970661,3,0.006967
5124093,1084_001.0,Arterial,1.0,D2,Daily,353.68,0.245646,0.139222,0.400000,1144.0,Arterial,RuralRecLowVol,2024,400,160.0,0,1000000,SNL,S04-Fal,0.926,148.160,WD0,W1-Weekday,0.938,138.974080,EV,Daily,0.064197,8.921755,1.118740,9.981125,12,0.008725
5124094,1084_001.0,Arterial,1.0,D2,Daily,353.68,0.245646,0.139222,0.400000,1144.0,Arterial,RuralRecLowVol,2024,400,160.0,0,1000000,SNL,S04-Fal,0.926,148.160,WD0,W1-Weekday,0.938,138.974080,MD,Daily,0.077305,10.743390,1.118740,12.019061,6,0.010506


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M01-Jan,0.885,1554.060,WD2,1-Monday,1.083,1683.046980,AM,Period,0.408955,688.290316,1.063784,732.192242,3,0.197038
1,0013_000.0,Arterial,3.0,D2,AM,841.00,0.208442,0.094293,0.065917,3716.0,Arterial,StatewideArterial,2030,20000,1318.0,0,1000000,SNT,M01-Jan,0.885,1166.430,WD2,1-Monday,1.083,1263.243690,AM,Period,0.408955,516.609702,1.088835,562.502580,3,0.151373
2,0013_000.6,Arterial,3.0,D1,AM,1203.80,0.125685,0.058232,0.085248,3716.0,Arterial,StatewideArterial,2030,19500,1662.0,0,1000000,SNT,M01-Jan,0.885,1470.870,WD2,1-Monday,1.083,1592.952210,AM,Period,0.408955,651.445618,1.054253,686.788626,3,0.184819
3,0013_000.6,Arterial,3.0,D2,AM,1111.40,0.141623,0.065323,0.078704,3716.0,Arterial,StatewideArterial,2030,19500,1535.0,0,1000000,SNT,M01-Jan,0.885,1358.475,WD2,1-Monday,1.083,1471.228425,AM,Period,0.408955,601.666079,1.060986,638.359373,3,0.171787
4,0013_001.8,Arterial,3.0,D1,AM,845.70,0.120137,0.074258,0.071745,3680.0,Arterial,StatewideArterial,2030,17000,1220.0,0,1000000,SNT,M01-Jan,0.885,1079.700,WD2,1-Monday,1.083,1169.315100,AM,Period,0.408955,478.197144,1.061156,507.441978,3,0.137892
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5173243,1084_001.0,Arterial,1.0,D1,Daily,467.76,0.273345,0.161878,0.600000,1144.0,Arterial,RuralRecLowVol,2030,500,300.0,0,1000000,SNL,S04-Fal,0.926,277.800,WD0,W1-Weekday,0.938,260.576400,PM,Daily,0.094372,24.591158,1.135608,27.925916,3,0.024411
5173244,1084_001.0,Arterial,1.0,D2,Daily,311.84,0.273345,0.161878,0.400000,1144.0,Arterial,RuralRecLowVol,2030,500,200.0,0,1000000,SNL,S04-Fal,0.926,185.200,WD0,W1-Weekday,0.938,173.717600,AM,Daily,0.051266,8.905846,1.135608,10.113550,3,0.008841
5173245,1084_001.0,Arterial,1.0,D2,Daily,311.84,0.273345,0.161878,0.400000,1144.0,Arterial,RuralRecLowVol,2030,500,200.0,0,1000000,SNL,S04-Fal,0.926,185.200,WD0,W1-Weekday,0.938,173.717600,EV,Daily,0.064197,11.152194,1.135608,12.664521,12,0.011070
5173246,1084_001.0,Arterial,1.0,D2,Daily,311.84,0.273345,0.161878,0.400000,1144.0,Arterial,RuralRecLowVol,2030,500,200.0,0,1000000,SNL,S04-Fal,0.926,185.200,WD0,W1-Weekday,0.938,173.717600,MD,Daily,0.077305,13.429237,1.135608,15.250349,6,0.013331


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M01-Jan,0.885,1667.340,WD2,1-Monday,1.083,1805.729220,AM,Period,0.408955,738.461819,1.057934,781.244227,3,0.210238
1,0013_000.0,Arterial,3.0,D2,AM,846.80,0.205598,0.086679,0.063880,3716.0,Arterial,StatewideArterial,2040,20500,1310.0,0,1000000,SNT,M01-Jan,0.885,1159.350,WD2,1-Monday,1.083,1255.576050,AM,Period,0.408955,513.473983,1.084459,556.841554,3,0.149850
2,0013_000.6,Arterial,3.0,D1,AM,1314.50,0.118600,0.051350,0.091021,3716.0,Arterial,StatewideArterial,2040,20000,1820.0,0,1000000,SNT,M01-Jan,0.885,1610.700,WD2,1-Monday,1.083,1744.388100,AM,Period,0.408955,713.376068,1.049395,748.613426,3,0.201457
3,0013_000.6,Arterial,3.0,D2,AM,1076.00,0.144610,0.063290,0.074506,3716.0,Arterial,StatewideArterial,2040,20000,1490.0,0,1000000,SNT,M01-Jan,0.885,1318.650,WD2,1-Monday,1.083,1428.097950,AM,Period,0.408955,584.027660,1.060567,619.400413,3,0.166685
4,0013_001.8,Arterial,3.0,D1,AM,894.00,0.124720,0.076286,0.073113,3680.0,Arterial,StatewideArterial,2040,17500,1279.0,0,1000000,SNT,M01-Jan,0.885,1131.915,WD2,1-Monday,1.083,1225.863945,AM,Period,0.408955,501.323072,1.063087,532.950165,3,0.144823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5206011,1084_001.0,Arterial,1.0,D1,Daily,535.56,0.285010,0.222832,0.600000,1144.0,Arterial,RuralRecLowVol,2040,700,420.0,0,1000000,SNL,S04-Fal,0.926,388.920,WD0,W1-Weekday,0.938,364.806960,PM,Daily,0.094372,34.427622,1.168418,40.225857,3,0.035162
5206012,1084_001.0,Arterial,1.0,D2,Daily,357.04,0.285010,0.222832,0.400000,1144.0,Arterial,RuralRecLowVol,2040,700,280.0,0,1000000,SNL,S04-Fal,0.926,259.280,WD0,W1-Weekday,0.938,243.204640,AM,Daily,0.051266,12.468184,1.168418,14.568052,3,0.012734
5206013,1084_001.0,Arterial,1.0,D2,Daily,357.04,0.285010,0.222832,0.400000,1144.0,Arterial,RuralRecLowVol,2040,700,280.0,0,1000000,SNL,S04-Fal,0.926,259.280,WD0,W1-Weekday,0.938,243.204640,EV,Daily,0.064197,15.613072,1.168418,18.242596,12,0.015946
5206014,1084_001.0,Arterial,1.0,D2,Daily,357.04,0.285010,0.222832,0.400000,1144.0,Arterial,RuralRecLowVol,2040,700,280.0,0,1000000,SNL,S04-Fal,0.926,259.280,WD0,W1-Weekday,0.938,243.204640,MD,Daily,0.077305,18.800932,1.168418,21.967349,6,0.019202


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.090370,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M01-Jan,0.885,1639.905,WD2,1-Monday,1.083,1776.017115,AM,Period,0.408955,726.310908,1.054994,766.253298,3,0.206204
1,0013_000.0,Arterial,3.0,D2,AM,867.60,0.210350,0.090134,0.063558,3716.0,Arterial,StatewideArterial,2050,20500,1303.0,0,1000000,SNT,M01-Jan,0.885,1153.155,WD2,1-Monday,1.083,1248.866865,AM,Period,0.408955,510.730229,1.087137,555.233692,3,0.149417
2,0013_000.6,Arterial,3.0,D1,AM,1366.50,0.115843,0.051884,0.093377,3716.0,Arterial,StatewideArterial,2050,20000,1868.0,0,1000000,SNT,M01-Jan,0.885,1653.180,WD2,1-Monday,1.083,1790.393940,AM,Period,0.408955,732.190381,1.049111,768.148886,3,0.206714
3,0013_000.6,Arterial,3.0,D2,AM,1046.00,0.156883,0.069981,0.071476,3716.0,Arterial,StatewideArterial,2050,20000,1430.0,0,1000000,SNT,M01-Jan,0.885,1265.550,WD2,1-Monday,1.083,1370.590650,AM,Period,0.408955,560.509767,1.066367,597.709182,3,0.160847
4,0013_001.8,Arterial,3.0,D1,AM,890.70,0.126417,0.077916,0.073503,3680.0,Arterial,StatewideArterial,2050,17500,1286.0,0,1000000,SNT,M01-Jan,0.885,1138.110,WD2,1-Monday,1.083,1232.573130,AM,Period,0.408955,504.066826,1.064242,536.448889,3,0.145774
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5283835,1084_001.0,Arterial,1.0,D1,Daily,388.56,0.208616,0.174336,0.600000,1144.0,Arterial,RuralRecLowVol,2050,800,480.0,0,1000000,SNL,S04-Fal,0.926,444.480,WD0,W1-Weekday,0.938,416.922240,PM,Daily,0.094372,39.345854,1.128891,44.417191,3,0.038826
5283836,1084_001.0,Arterial,1.0,D2,Daily,259.04,0.208616,0.174336,0.400000,1144.0,Arterial,RuralRecLowVol,2050,800,320.0,0,1000000,SNL,S04-Fal,0.926,296.320,WD0,W1-Weekday,0.938,277.948160,AM,Daily,0.051266,14.249353,1.128891,16.085971,3,0.014061
5283837,1084_001.0,Arterial,1.0,D2,Daily,259.04,0.208616,0.174336,0.400000,1144.0,Arterial,RuralRecLowVol,2050,800,320.0,0,1000000,SNL,S04-Fal,0.926,296.320,WD0,W1-Weekday,0.938,277.948160,EV,Daily,0.064197,17.843510,1.128891,20.143384,12,0.017608
5283838,1084_001.0,Arterial,1.0,D2,Daily,259.04,0.208616,0.174336,0.400000,1144.0,Arterial,RuralRecLowVol,2050,800,320.0,0,1000000,SNL,S04-Fal,0.926,296.320,WD0,W1-Weekday,0.938,277.948160,MD,Daily,0.077305,21.486780,1.128891,24.256238,6,0.021203


...

,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M01-Jan,0.885,1745.220,WD2,1-Monday,1.083,1890.073260,AM,Period,0.408955,772.954728,1.058905,818.485915,3,0.220260
1,0013_000.0,Arterial,3.0,D2,AM,902.30,0.209797,0.094093,0.064265,3716.0,Arterial,StatewideArterial,2040,21284.2,1368.0,0,1000000,SNT,M01-Jan,0.885,1210.680,WD2,1-Monday,1.083,1311.166440,AM,Period,0.408955,536.207945,1.089006,583.933602,3,0.157140
2,0013_000.6,Arterial,3.0,D1,AM,1424.50,0.117866,0.055177,0.090718,3716.0,Arterial,StatewideArterial,2040,21260.7,1929.0,0,1000000,SNT,M01-Jan,0.885,1707.165,WD2,1-Monday,1.083,1848.859695,AM,Period,0.408955,756.100239,1.051162,794.783696,3,0.213882
3,0013_000.6,Arterial,3.0,D2,AM,1167.20,0.148475,0.069225,0.074332,3716.0,Arterial,StatewideArterial,2040,21260.7,1580.0,0,1000000,SNT,M01-Jan,0.885,1398.300,WD2,1-Monday,1.083,1514.358900,AM,Period,0.408955,619.304498,1.064308,659.130574,3,0.177376
4,0013_001.8,Arterial,3.0,D1,AM,960.60,0.124089,0.072247,0.073287,3680.0,Arterial,StatewideArterial,2040,18379.7,1347.0,0,1000000,SNT,M01-Jan,0.885,1192.095,WD2,1-Monday,1.083,1291.038885,AM,Period,0.408955,527.976683,1.060941,560.152151,3,0.152215
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5117947,1084_001.0,Arterial,1.0,D1,Daily,525.54,0.285992,0.206645,0.600000,1144.0,Arterial,RuralRecLowVol,2040,683.3,410.0,0,1000000,SNL,S04-Fal,0.926,379.660,WD0,W1-Weekday,0.938,356.121080,PM,Daily,0.094372,33.607917,1.160521,39.002680,3,0.034093
5117948,1084_001.0,Arterial,1.0,D2,Daily,350.36,0.285992,0.206645,0.400000,1144.0,Arterial,RuralRecLowVol,2040,683.3,273.0,0,1000000,SNL,S04-Fal,0.926,252.798,WD0,W1-Weekday,0.938,237.124524,AM,Daily,0.051266,12.156479,1.160521,14.107845,3,0.012332
5117949,1084_001.0,Arterial,1.0,D2,Daily,350.36,0.285992,0.206645,0.400000,1144.0,Arterial,RuralRecLowVol,2040,683.3,273.0,0,1000000,SNL,S04-Fal,0.926,252.798,WD0,W1-Weekday,0.938,237.124524,EV,Daily,0.064197,15.222745,1.160521,17.666309,12,0.015443
5117950,1084_001.0,Arterial,1.0,D2,Daily,350.36,0.285992,0.206645,0.400000,1144.0,Arterial,RuralRecLowVol,2040,683.3,273.0,0,1000000,SNL,S04-Fal,0.926,252.798,WD0,W1-Weekday,0.938,237.124524,MD,Daily,0.077305,18.330909,1.160521,21.273397,6,0.018596


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1366.50,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M01-Jan,0.885,1776.195,WD2,1-Monday,1.083,1923.619185,AM,Period,0.408955,786.673499,1.054790,829.775032,3,0.223298
1,0013_000.0,Arterial,3.0,D2,AM,945.50,0.208778,0.093284,0.064714,3716.0,Arterial,StatewideArterial,2050,21460.0,1389.0,0,1000000,SNT,M01-Jan,0.885,1229.265,WD2,1-Monday,1.083,1331.293995,AM,Period,0.408955,544.439208,1.088398,592.566367,3,0.159464
2,0013_000.6,Arterial,3.0,D1,AM,1509.20,0.115359,0.053538,0.091884,3716.0,Arterial,StatewideArterial,2050,21790.9,2002.0,0,1000000,SNT,M01-Jan,0.885,1771.770,WD2,1-Monday,1.083,1918.826910,AM,Period,0.408955,784.713674,1.049841,823.824569,3,0.221697
3,0013_000.6,Arterial,3.0,D2,AM,1178.10,0.153467,0.071386,0.071726,3716.0,Arterial,StatewideArterial,2050,21790.9,1563.0,0,1000000,SNT,M01-Jan,0.885,1383.255,WD2,1-Monday,1.083,1498.065165,AM,Period,0.408955,612.641095,1.066387,653.312227,3,0.175811
4,0013_001.8,Arterial,3.0,D1,AM,1017.40,0.122862,0.071948,0.073615,3680.0,Arterial,StatewideArterial,2050,19202.7,1414.0,0,1000000,SNT,M01-Jan,0.885,1251.390,WD2,1-Monday,1.083,1355.255370,AM,Period,0.408955,554.238329,1.060546,587.795515,3,0.159727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089275,1084_001.0,Arterial,1.0,D1,Daily,446.82,0.212837,0.181550,0.600000,1144.0,Arterial,RuralRecLowVol,2050,897.1,538.0,0,1000000,SNL,S04-Fal,0.926,498.188,WD0,W1-Weekday,0.938,467.300344,PM,Daily,0.094372,44.100144,1.133342,49.980558,3,0.043689
5089276,1084_001.0,Arterial,1.0,D2,Daily,297.88,0.212837,0.181550,0.400000,1144.0,Arterial,RuralRecLowVol,2050,897.1,359.0,0,1000000,SNL,S04-Fal,0.926,332.434,WD0,W1-Weekday,0.938,311.823092,AM,Daily,0.051266,15.985993,1.133342,18.117602,3,0.015837
5089277,1084_001.0,Arterial,1.0,D2,Daily,297.88,0.212837,0.181550,0.400000,1144.0,Arterial,RuralRecLowVol,2050,897.1,359.0,0,1000000,SNL,S04-Fal,0.926,332.434,WD0,W1-Weekday,0.938,311.823092,EV,Daily,0.064197,20.018188,1.133342,22.687459,12,0.019832
5089278,1084_001.0,Arterial,1.0,D2,Daily,297.88,0.212837,0.181550,0.400000,1144.0,Arterial,RuralRecLowVol,2050,897.1,359.0,0,1000000,SNL,S04-Fal,0.926,332.434,WD0,W1-Weekday,0.938,311.823092,MD,Daily,0.077305,24.105481,1.133342,27.319761,6,0.023881


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M01-Jan,0.885,1547.865,WD2,1-Monday,1.083,1676.337795,AM,Period,0.408955,685.546562,1.063967,729.398838,3,0.196286
1,0013_000.0,Arterial,3.0,D2,AM,831.20,0.208855,0.094562,0.065875,3716.0,Arterial,StatewideArterial,2030,19859.5,1308.0,0,1000000,SNT,M01-Jan,0.885,1157.580,WD2,1-Monday,1.083,1253.659140,AM,Period,0.408955,512.690053,1.089052,558.346114,3,0.150255
2,0013_000.6,Arterial,3.0,D1,AM,1193.50,0.126183,0.058483,0.085016,3716.0,Arterial,StatewideArterial,2030,19417.3,1651.0,0,1000000,SNT,M01-Jan,0.885,1461.135,WD2,1-Monday,1.083,1582.409205,AM,Period,0.408955,647.134004,1.054478,682.388845,3,0.183635
3,0013_000.6,Arterial,3.0,D2,AM,1108.60,0.140718,0.065127,0.078969,3716.0,Arterial,StatewideArterial,2030,19417.3,1533.0,0,1000000,SNT,M01-Jan,0.885,1356.705,WD2,1-Monday,1.083,1469.311515,AM,Period,0.408955,600.882149,1.060707,637.360021,3,0.171518
4,0013_001.8,Arterial,3.0,D1,AM,840.70,0.120971,0.074700,0.071485,3680.0,Arterial,StatewideArterial,2030,16972.9,1213.0,0,1000000,SNT,M01-Jan,0.885,1073.505,WD2,1-Monday,1.083,1162.605915,AM,Period,0.408955,475.453390,1.061544,504.714670,3,0.137151
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5175291,1084_001.0,Arterial,1.0,D1,Daily,489.24,0.261467,0.155016,0.600000,1144.0,Arterial,RuralRecLowVol,2030,535.8,321.0,0,1000000,SNL,S04-Fal,0.926,297.246,WD0,W1-Weekday,0.938,278.816748,PM,Daily,0.094372,26.312540,1.129801,29.727942,3,0.025986
5175292,1084_001.0,Arterial,1.0,D2,Daily,326.16,0.261467,0.155016,0.400000,1144.0,Arterial,RuralRecLowVol,2030,535.8,214.0,0,1000000,SNL,S04-Fal,0.926,198.164,WD0,W1-Weekday,0.938,185.877832,AM,Daily,0.051266,9.529255,1.129801,10.766165,3,0.009411
5175293,1084_001.0,Arterial,1.0,D2,Daily,326.16,0.261467,0.155016,0.400000,1144.0,Arterial,RuralRecLowVol,2030,535.8,214.0,0,1000000,SNL,S04-Fal,0.926,198.164,WD0,W1-Weekday,0.938,185.877832,EV,Daily,0.064197,11.932848,1.129801,13.481747,12,0.011785
5175294,1084_001.0,Arterial,1.0,D2,Daily,326.16,0.261467,0.155016,0.400000,1144.0,Arterial,RuralRecLowVol,2030,535.8,214.0,0,1000000,SNL,S04-Fal,0.926,198.164,WD0,W1-Weekday,0.938,185.877832,MD,Daily,0.077305,14.369284,1.129801,16.234436,6,0.014191


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.091530,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M01-Jan,0.885,1659.375,WD2,1-Monday,1.083,1797.103125,AM,Period,0.408955,734.934135,1.058216,777.719404,3,0.209289
1,0013_000.0,Arterial,3.0,D2,AM,849.50,0.204944,0.086404,0.064144,3716.0,Arterial,StatewideArterial,2040,20487.5,1314.0,0,1000000,SNT,M01-Jan,0.885,1162.890,WD2,1-Monday,1.083,1259.409870,AM,Period,0.408955,515.041842,1.084191,558.403576,3,0.150270
2,0013_000.6,Arterial,3.0,D1,AM,1309.20,0.119004,0.051635,0.090739,3716.0,Arterial,StatewideArterial,2040,19986.4,1814.0,0,1000000,SNT,M01-Jan,0.885,1605.390,WD2,1-Monday,1.083,1738.637370,AM,Period,0.408955,711.024278,1.049618,746.303943,3,0.200835
3,0013_000.6,Arterial,3.0,D2,AM,1079.70,0.144114,0.063166,0.074833,3716.0,Arterial,StatewideArterial,2040,19986.4,1496.0,0,1000000,SNT,M01-Jan,0.885,1323.960,WD2,1-Monday,1.083,1433.848680,AM,Period,0.408955,586.379449,1.060406,621.800091,3,0.167330
4,0013_001.8,Arterial,3.0,D1,AM,891.70,0.124818,0.076371,0.072980,3680.0,Arterial,StatewideArterial,2040,17490.7,1276.0,0,1000000,SNT,M01-Jan,0.885,1129.260,WD2,1-Monday,1.083,1222.988580,AM,Period,0.408955,500.147177,1.063149,531.730992,3,0.144492
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195771,1084_001.0,Arterial,1.0,D1,Daily,553.08,0.287047,0.225754,0.600000,1144.0,Arterial,RuralRecLowVol,2040,729.2,438.0,0,1000000,SNL,S04-Fal,0.926,405.588,WD0,W1-Weekday,0.938,380.441544,PM,Daily,0.094372,35.903091,1.170286,42.016899,3,0.036728
5195772,1084_001.0,Arterial,1.0,D2,Daily,368.72,0.287047,0.225754,0.400000,1144.0,Arterial,RuralRecLowVol,2040,729.2,292.0,0,1000000,SNL,S04-Fal,0.926,270.392,WD0,W1-Weekday,0.938,253.627696,AM,Daily,0.051266,13.002535,1.170286,15.216690,3,0.013301
5195773,1084_001.0,Arterial,1.0,D2,Daily,368.72,0.287047,0.225754,0.400000,1144.0,Arterial,RuralRecLowVol,2040,729.2,292.0,0,1000000,SNL,S04-Fal,0.926,270.392,WD0,W1-Weekday,0.938,253.627696,EV,Daily,0.064197,16.282203,1.170286,19.054841,12,0.016656
5195774,1084_001.0,Arterial,1.0,D2,Daily,368.72,0.287047,0.225754,0.400000,1144.0,Arterial,RuralRecLowVol,2040,729.2,292.0,0,1000000,SNL,S04-Fal,0.926,270.392,WD0,W1-Weekday,0.938,253.627696,MD,Daily,0.077305,19.606686,1.170286,22.945438,6,0.020057


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE
0,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M01-Jan,0.885,1625.745,WD2,1-Monday,1.083,1760.681835,AM,Period,0.408955,720.039470,1.054536,759.307396,3,0.204335
1,0013_000.0,Arterial,3.0,D2,AM,874.30,0.210111,0.088757,0.063997,3716.0,Arterial,StatewideArterial,2050,20511.0,1313.0,0,1000000,SNT,M01-Jan,0.885,1162.005,WD2,1-Monday,1.083,1258.451415,AM,Period,0.408955,514.649877,1.086401,559.115909,3,0.150462
2,0013_000.6,Arterial,3.0,D1,AM,1363.50,0.116392,0.051779,0.093181,3716.0,Arterial,StatewideArterial,2050,19998.6,1863.0,0,1000000,SNT,M01-Jan,0.885,1648.755,WD2,1-Monday,1.083,1785.601665,AM,Period,0.408955,730.230557,1.049168,766.134229,3,0.206172
3,0013_000.6,Arterial,3.0,D2,AM,1047.70,0.157297,0.069295,0.071599,3716.0,Arterial,StatewideArterial,2050,19998.6,1432.0,0,1000000,SNT,M01-Jan,0.885,1267.320,WD2,1-Monday,1.083,1372.507560,AM,Period,0.408955,561.293697,1.066107,598.398977,3,0.161033
4,0013_001.8,Arterial,3.0,D1,AM,889.40,0.126715,0.078030,0.073434,3680.0,Arterial,StatewideArterial,2050,17493.6,1285.0,0,1000000,SNT,M01-Jan,0.885,1137.225,WD2,1-Monday,1.083,1231.614675,AM,Period,0.408955,503.674861,1.064358,536.090365,3,0.145677
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285883,1084_001.0,Arterial,1.0,D1,Daily,343.56,0.202934,0.162592,0.600000,1144.0,Arterial,RuralRecLowVol,2050,725.0,435.0,0,1000000,SNL,S04-Fal,0.926,402.810,WD0,W1-Weekday,0.938,377.835780,PM,Daily,0.094372,35.657180,1.121883,40.003171,3,0.034968
5285884,1084_001.0,Arterial,1.0,D2,Daily,229.04,0.202934,0.162592,0.400000,1144.0,Arterial,RuralRecLowVol,2050,725.0,290.0,0,1000000,SNL,S04-Fal,0.926,268.540,WD0,W1-Weekday,0.938,251.890520,AM,Daily,0.051266,12.913476,1.121883,14.487405,3,0.012664
5285885,1084_001.0,Arterial,1.0,D2,Daily,229.04,0.202934,0.162592,0.400000,1144.0,Arterial,RuralRecLowVol,2050,725.0,290.0,0,1000000,SNL,S04-Fal,0.926,268.540,WD0,W1-Weekday,0.938,251.890520,EV,Daily,0.064197,16.170681,1.121883,18.141607,12,0.015858
5285886,1084_001.0,Arterial,1.0,D2,Daily,229.04,0.202934,0.162592,0.400000,1144.0,Arterial,RuralRecLowVol,2050,725.0,290.0,0,1000000,SNL,S04-Fal,0.926,268.540,WD0,W1-Weekday,0.938,251.890520,MD,Daily,0.077305,19.472394,1.121883,21.845741,6,0.019096


In [183]:
#15-Min Analysis

dfs_Peak_VC = []

for df_PeakHourForecasts_wCapacity in dfs_PeakHourForecasts_wCapacity:

    df_Peak_VC = df_PeakHourForecasts_wCapacity
    
    #PHF conditions (from Chad's spreadsheet for Urban (0.92,0.95,0.97,0.98). HCM2000 Exhibit 9.2 says default PHF for Urban Areas is 0.92, Rural is 0.88. Rural estimated at (0.88,0.91,0.93,0.95)
    conditions = [
        (df_Peak_VC[nameAT] >= 3) & (df_Peak_VC['VCPeakHourPCE'] <= 0.85),
        (df_Peak_VC[nameAT] >= 3) & (df_Peak_VC['VCPeakHourPCE'] >  0.85) & (df_Peak_VC['VCPeakHourPCE'] <= 0.95),
        (df_Peak_VC[nameAT] >= 3) & (df_Peak_VC['VCPeakHourPCE'] >  0.95) & (df_Peak_VC['VCPeakHourPCE'] <= 1.00),
        (df_Peak_VC[nameAT] >= 3) & (df_Peak_VC['VCPeakHourPCE'] >  1.00),
        (df_Peak_VC[nameAT] <= 2) & (df_Peak_VC['VCPeakHourPCE'] <= 0.85),
        (df_Peak_VC[nameAT] <= 2) & (df_Peak_VC['VCPeakHourPCE'] >  0.85) & (df_Peak_VC['VCPeakHourPCE'] <= 0.95),
        (df_Peak_VC[nameAT] <= 2) & (df_Peak_VC['VCPeakHourPCE'] >  0.95) & (df_Peak_VC['VCPeakHourPCE'] <= 1.00),
        (df_Peak_VC[nameAT] <= 2) & (df_Peak_VC['VCPeakHourPCE'] >  1.00)
    ]
    choices = [0.92,0.95,0.97,0.98,0.88,0.91,0.93,0.95]

    df_Peak_VC['PHF'] = np.select(conditions, choices, default=0)


    #15-min volume and v/c calcs
    df_Peak_VC['ForecastPeak15MinPCE'] = df_Peak_VC['ForecastPeakHourPCE']  / df_Peak_VC['PHF']
    df_Peak_VC['VCPeak15MinPCE']  = df_Peak_VC['ForecastPeak15MinPCE'] / df_Peak_VC[nameCap1Hr]

    #effective K-factor
    df_Peak_VC['kFactor_effective']  = df_Peak_VC['ForecastPeakHour']  / df_Peak_VC['ForecastAADT']

    display(df_Peak_VC)
    
    dfs_Peak_VC.append(df_Peak_VC)
    

,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
0,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M01-Jan,0.885,1479.720,WD2,1-Monday,1.083,1602.536760,AM,Period,0.408955,655.365266,1.061055,695.378478,3,0.187131,0.92,755.846171,0.203403,0.034493
1,0013_000.0,Arterial,3.0,D2,AM,782.40,0.207183,0.090874,0.065183,3716.0,Arterial,StatewideArterial,2019,19000,1238.0,0,1000000,SNT,M01-Jan,0.885,1095.630,WD2,1-Monday,1.083,1186.567290,AM,Period,0.408955,485.252512,1.086874,527.408204,3,0.141929,0.92,573.269787,0.154271,0.025540
2,0013_000.6,Arterial,3.0,D1,AM,1174.90,0.119329,0.056260,0.081416,3716.0,Arterial,StatewideArterial,2019,19000,1547.0,0,1000000,SNT,M01-Jan,0.885,1369.095,WD2,1-Monday,1.083,1482.729885,AM,Period,0.408955,606.369657,1.051996,637.898402,3,0.171663,0.92,693.367829,0.186590,0.031914
3,0013_000.6,Arterial,3.0,D2,AM,1186.30,0.125516,0.057658,0.082206,3716.0,Arterial,StatewideArterial,2019,19000,1562.0,0,1000000,SNT,M01-Jan,0.885,1382.370,WD2,1-Monday,1.083,1497.106710,AM,Period,0.408955,612.249130,1.053932,645.269192,3,0.173646,0.92,701.379557,0.188746,0.032224
4,0013_001.3,Arterial,3.0,D1,AM,1419.90,0.112261,0.060638,0.084827,3716.0,Arterial,StatewideArterial,2019,15500,1315.0,0,1000000,SNT,M01-Jan,0.885,1163.775,WD2,1-Monday,1.083,1260.368325,AM,Period,0.408955,515.433807,1.052771,542.633930,3,0.146026,0.92,589.819490,0.158724,0.033254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074939,1084_001.0,Arterial,1.0,D1,Daily,548.52,0.218442,0.123824,0.600000,1144.0,Arterial,RuralRecLowVol,2019,350,210.0,0,1000000,SNL,S04-Fal,0.926,194.460,WD0,W1-Weekday,0.938,182.403480,PM,Daily,0.094372,17.213811,1.105601,19.031598,3,0.016636,0.88,21.626816,0.018905,0.049182
5074940,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,AM,Daily,0.051266,6.234092,1.105601,6.892415,3,0.006025,0.88,7.832290,0.006846,0.017812
5074941,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,EV,Daily,0.064197,7.806536,1.105601,8.630910,12,0.007545,0.88,9.807852,0.008573,0.022304
5074942,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,MD,Daily,0.077305,9.400466,1.105601,10.393160,6,0.009085,0.88,11.810409,0.010324,0.026858


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
0,0013_000.0,Arterial,3.0,D1,AM,1121.90,0.145913,0.066851,0.089138,3716.0,Arterial,StatewideArterial,2024,19500,1738.0,0,1000000,SNT,M01-Jan,0.885,1538.130,WD2,1-Monday,1.083,1665.794790,AM,Period,0.408955,681.234948,1.062608,723.885757,3,0.194802,0.92,786.832345,0.211742,0.034935
1,0013_000.0,Arterial,3.0,D2,AM,823.10,0.209209,0.095007,0.065398,3716.0,Arterial,StatewideArterial,2024,19500,1275.0,0,1000000,SNT,M01-Jan,0.885,1128.375,WD2,1-Monday,1.083,1222.030125,AM,Period,0.408955,499.755212,1.089345,544.405921,3,0.146503,0.92,591.745566,0.159243,0.025628
2,0013_000.6,Arterial,3.0,D1,AM,1213.30,0.122311,0.057364,0.086046,3716.0,Arterial,StatewideArterial,2024,19500,1678.0,0,1000000,SNT,M01-Jan,0.885,1485.030,WD2,1-Monday,1.083,1608.287490,AM,Period,0.408955,657.717056,1.053144,692.670979,3,0.186402,0.92,752.903239,0.202611,0.033729
3,0013_000.6,Arterial,3.0,D2,AM,1105.50,0.140299,0.064858,0.078401,3716.0,Arterial,StatewideArterial,2024,19500,1529.0,0,1000000,SNT,M01-Jan,0.885,1353.165,WD2,1-Monday,1.083,1465.477695,AM,Period,0.408955,599.314290,1.060488,635.565892,3,0.171035,0.92,690.832491,0.185908,0.030734
4,0013_001.8,Arterial,3.0,D1,AM,844.20,0.120232,0.072613,0.072599,3680.0,Arterial,StatewideArterial,2024,17000,1234.0,0,1000000,SNT,M01-Jan,0.885,1092.090,WD2,1-Monday,1.083,1182.733470,AM,Period,0.408955,483.684652,1.060353,512.876471,3,0.139369,0.92,557.474425,0.151488,0.028452
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5124091,1084_001.0,Arterial,1.0,D1,Daily,530.52,0.245646,0.139222,0.600000,1144.0,Arterial,RuralRecLowVol,2024,400,240.0,0,1000000,SNL,S04-Fal,0.926,222.240,WD0,W1-Weekday,0.938,208.461120,PM,Daily,0.094372,19.672927,1.118740,22.008892,3,0.019239,0.88,25.010105,0.021862,0.049182
5124092,1084_001.0,Arterial,1.0,D2,Daily,353.68,0.245646,0.139222,0.400000,1144.0,Arterial,RuralRecLowVol,2024,400,160.0,0,1000000,SNL,S04-Fal,0.926,148.160,WD0,W1-Weekday,0.938,138.974080,AM,Daily,0.051266,7.124677,1.118740,7.970661,3,0.006967,0.88,9.057570,0.007917,0.017812
5124093,1084_001.0,Arterial,1.0,D2,Daily,353.68,0.245646,0.139222,0.400000,1144.0,Arterial,RuralRecLowVol,2024,400,160.0,0,1000000,SNL,S04-Fal,0.926,148.160,WD0,W1-Weekday,0.938,138.974080,EV,Daily,0.064197,8.921755,1.118740,9.981125,12,0.008725,0.88,11.342188,0.009914,0.022304
5124094,1084_001.0,Arterial,1.0,D2,Daily,353.68,0.245646,0.139222,0.400000,1144.0,Arterial,RuralRecLowVol,2024,400,160.0,0,1000000,SNL,S04-Fal,0.926,148.160,WD0,W1-Weekday,0.938,138.974080,MD,Daily,0.077305,10.743390,1.118740,12.019061,6,0.010506,0.88,13.658024,0.011939,0.026858


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
0,0013_000.0,Arterial,3.0,D1,AM,1120.50,0.150022,0.067559,0.087824,3716.0,Arterial,StatewideArterial,2030,20000,1756.0,0,1000000,SNT,M01-Jan,0.885,1554.060,WD2,1-Monday,1.083,1683.046980,AM,Period,0.408955,688.290316,1.063784,732.192242,3,0.197038,0.92,795.861133,0.214171,0.034415
1,0013_000.0,Arterial,3.0,D2,AM,841.00,0.208442,0.094293,0.065917,3716.0,Arterial,StatewideArterial,2030,20000,1318.0,0,1000000,SNT,M01-Jan,0.885,1166.430,WD2,1-Monday,1.083,1263.243690,AM,Period,0.408955,516.609702,1.088835,562.502580,3,0.151373,0.92,611.415848,0.164536,0.025830
2,0013_000.6,Arterial,3.0,D1,AM,1203.80,0.125685,0.058232,0.085248,3716.0,Arterial,StatewideArterial,2030,19500,1662.0,0,1000000,SNT,M01-Jan,0.885,1470.870,WD2,1-Monday,1.083,1592.952210,AM,Period,0.408955,651.445618,1.054253,686.788626,3,0.184819,0.92,746.509376,0.200891,0.033407
3,0013_000.6,Arterial,3.0,D2,AM,1111.40,0.141623,0.065323,0.078704,3716.0,Arterial,StatewideArterial,2030,19500,1535.0,0,1000000,SNT,M01-Jan,0.885,1358.475,WD2,1-Monday,1.083,1471.228425,AM,Period,0.408955,601.666079,1.060986,638.359373,3,0.171787,0.92,693.868883,0.186725,0.030855
4,0013_001.8,Arterial,3.0,D1,AM,845.70,0.120137,0.074258,0.071745,3680.0,Arterial,StatewideArterial,2030,17000,1220.0,0,1000000,SNT,M01-Jan,0.885,1079.700,WD2,1-Monday,1.083,1169.315100,AM,Period,0.408955,478.197144,1.061156,507.441978,3,0.137892,0.92,551.567368,0.149882,0.028129
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5173243,1084_001.0,Arterial,1.0,D1,Daily,467.76,0.273345,0.161878,0.600000,1144.0,Arterial,RuralRecLowVol,2030,500,300.0,0,1000000,SNL,S04-Fal,0.926,277.800,WD0,W1-Weekday,0.938,260.576400,PM,Daily,0.094372,24.591158,1.135608,27.925916,3,0.024411,0.88,31.733996,0.027740,0.049182
5173244,1084_001.0,Arterial,1.0,D2,Daily,311.84,0.273345,0.161878,0.400000,1144.0,Arterial,RuralRecLowVol,2030,500,200.0,0,1000000,SNL,S04-Fal,0.926,185.200,WD0,W1-Weekday,0.938,173.717600,AM,Daily,0.051266,8.905846,1.135608,10.113550,3,0.008841,0.88,11.492670,0.010046,0.017812
5173245,1084_001.0,Arterial,1.0,D2,Daily,311.84,0.273345,0.161878,0.400000,1144.0,Arterial,RuralRecLowVol,2030,500,200.0,0,1000000,SNL,S04-Fal,0.926,185.200,WD0,W1-Weekday,0.938,173.717600,EV,Daily,0.064197,11.152194,1.135608,12.664521,12,0.011070,0.88,14.391501,0.012580,0.022304
5173246,1084_001.0,Arterial,1.0,D2,Daily,311.84,0.273345,0.161878,0.400000,1144.0,Arterial,RuralRecLowVol,2030,500,200.0,0,1000000,SNL,S04-Fal,0.926,185.200,WD0,W1-Weekday,0.938,173.717600,MD,Daily,0.077305,13.429237,1.135608,15.250349,6,0.013331,0.88,17.329942,0.015149,0.026858


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
0,0013_000.0,Arterial,3.0,D1,AM,1218.10,0.141696,0.059191,0.091889,3716.0,Arterial,StatewideArterial,2040,20500,1884.0,0,1000000,SNT,M01-Jan,0.885,1667.340,WD2,1-Monday,1.083,1805.729220,AM,Period,0.408955,738.461819,1.057934,781.244227,3,0.210238,0.92,849.178507,0.228520,0.036023
1,0013_000.0,Arterial,3.0,D2,AM,846.80,0.205598,0.086679,0.063880,3716.0,Arterial,StatewideArterial,2040,20500,1310.0,0,1000000,SNT,M01-Jan,0.885,1159.350,WD2,1-Monday,1.083,1255.576050,AM,Period,0.408955,513.473983,1.084459,556.841554,3,0.149850,0.92,605.262558,0.162880,0.025048
2,0013_000.6,Arterial,3.0,D1,AM,1314.50,0.118600,0.051350,0.091021,3716.0,Arterial,StatewideArterial,2040,20000,1820.0,0,1000000,SNT,M01-Jan,0.885,1610.700,WD2,1-Monday,1.083,1744.388100,AM,Period,0.408955,713.376068,1.049395,748.613426,3,0.201457,0.92,813.710246,0.218975,0.035669
3,0013_000.6,Arterial,3.0,D2,AM,1076.00,0.144610,0.063290,0.074506,3716.0,Arterial,StatewideArterial,2040,20000,1490.0,0,1000000,SNT,M01-Jan,0.885,1318.650,WD2,1-Monday,1.083,1428.097950,AM,Period,0.408955,584.027660,1.060567,619.400413,3,0.166685,0.92,673.261318,0.181179,0.029201
4,0013_001.8,Arterial,3.0,D1,AM,894.00,0.124720,0.076286,0.073113,3680.0,Arterial,StatewideArterial,2040,17500,1279.0,0,1000000,SNT,M01-Jan,0.885,1131.915,WD2,1-Monday,1.083,1225.863945,AM,Period,0.408955,501.323072,1.063087,532.950165,3,0.144823,0.92,579.293657,0.157417,0.028647
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5206011,1084_001.0,Arterial,1.0,D1,Daily,535.56,0.285010,0.222832,0.600000,1144.0,Arterial,RuralRecLowVol,2040,700,420.0,0,1000000,SNL,S04-Fal,0.926,388.920,WD0,W1-Weekday,0.938,364.806960,PM,Daily,0.094372,34.427622,1.168418,40.225857,3,0.035162,0.88,45.711201,0.039957,0.049182
5206012,1084_001.0,Arterial,1.0,D2,Daily,357.04,0.285010,0.222832,0.400000,1144.0,Arterial,RuralRecLowVol,2040,700,280.0,0,1000000,SNL,S04-Fal,0.926,259.280,WD0,W1-Weekday,0.938,243.204640,AM,Daily,0.051266,12.468184,1.168418,14.568052,3,0.012734,0.88,16.554605,0.014471,0.017812
5206013,1084_001.0,Arterial,1.0,D2,Daily,357.04,0.285010,0.222832,0.400000,1144.0,Arterial,RuralRecLowVol,2040,700,280.0,0,1000000,SNL,S04-Fal,0.926,259.280,WD0,W1-Weekday,0.938,243.204640,EV,Daily,0.064197,15.613072,1.168418,18.242596,12,0.015946,0.88,20.730222,0.018121,0.022304
5206014,1084_001.0,Arterial,1.0,D2,Daily,357.04,0.285010,0.222832,0.400000,1144.0,Arterial,RuralRecLowVol,2040,700,280.0,0,1000000,SNL,S04-Fal,0.926,259.280,WD0,W1-Weekday,0.938,243.204640,MD,Daily,0.077305,18.800932,1.168418,21.967349,6,0.019202,0.88,24.962897,0.021821,0.026858


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
0,0013_000.0,Arterial,3.0,D1,AM,1233.60,0.123784,0.060473,0.090370,3716.0,Arterial,StatewideArterial,2050,20500,1853.0,0,1000000,SNT,M01-Jan,0.885,1639.905,WD2,1-Monday,1.083,1776.017115,AM,Period,0.408955,726.310908,1.054994,766.253298,3,0.206204,0.92,832.884020,0.224135,0.035430
1,0013_000.0,Arterial,3.0,D2,AM,867.60,0.210350,0.090134,0.063558,3716.0,Arterial,StatewideArterial,2050,20500,1303.0,0,1000000,SNT,M01-Jan,0.885,1153.155,WD2,1-Monday,1.083,1248.866865,AM,Period,0.408955,510.730229,1.087137,555.233692,3,0.149417,0.92,603.514883,0.162410,0.024914
2,0013_000.6,Arterial,3.0,D1,AM,1366.50,0.115843,0.051884,0.093377,3716.0,Arterial,StatewideArterial,2050,20000,1868.0,0,1000000,SNT,M01-Jan,0.885,1653.180,WD2,1-Monday,1.083,1790.393940,AM,Period,0.408955,732.190381,1.049111,768.148886,3,0.206714,0.92,834.944441,0.224689,0.036610
3,0013_000.6,Arterial,3.0,D2,AM,1046.00,0.156883,0.069981,0.071476,3716.0,Arterial,StatewideArterial,2050,20000,1430.0,0,1000000,SNT,M01-Jan,0.885,1265.550,WD2,1-Monday,1.083,1370.590650,AM,Period,0.408955,560.509767,1.066367,597.709182,3,0.160847,0.92,649.683894,0.174834,0.028025
4,0013_001.8,Arterial,3.0,D1,AM,890.70,0.126417,0.077916,0.073503,3680.0,Arterial,StatewideArterial,2050,17500,1286.0,0,1000000,SNT,M01-Jan,0.885,1138.110,WD2,1-Monday,1.083,1232.573130,AM,Period,0.408955,504.066826,1.064242,536.448889,3,0.145774,0.92,583.096618,0.158450,0.028804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5283835,1084_001.0,Arterial,1.0,D1,Daily,388.56,0.208616,0.174336,0.600000,1144.0,Arterial,RuralRecLowVol,2050,800,480.0,0,1000000,SNL,S04-Fal,0.926,444.480,WD0,W1-Weekday,0.938,416.922240,PM,Daily,0.094372,39.345854,1.128891,44.417191,3,0.038826,0.88,50.474081,0.044121,0.049182
5283836,1084_001.0,Arterial,1.0,D2,Daily,259.04,0.208616,0.174336,0.400000,1144.0,Arterial,RuralRecLowVol,2050,800,320.0,0,1000000,SNL,S04-Fal,0.926,296.320,WD0,W1-Weekday,0.938,277.948160,AM,Daily,0.051266,14.249353,1.128891,16.085971,3,0.014061,0.88,18.279512,0.015979,0.017812
5283837,1084_001.0,Arterial,1.0,D2,Daily,259.04,0.208616,0.174336,0.400000,1144.0,Arterial,RuralRecLowVol,2050,800,320.0,0,1000000,SNL,S04-Fal,0.926,296.320,WD0,W1-Weekday,0.938,277.948160,EV,Daily,0.064197,17.843510,1.128891,20.143384,12,0.017608,0.88,22.890209,0.020009,0.022304
5283838,1084_001.0,Arterial,1.0,D2,Daily,259.04,0.208616,0.174336,0.400000,1144.0,Arterial,RuralRecLowVol,2050,800,320.0,0,1000000,SNL,S04-Fal,0.926,296.320,WD0,W1-Weekday,0.938,277.948160,MD,Daily,0.077305,21.486780,1.128891,24.256238,6,0.021203,0.88,27.563907,0.024094,0.026858


...

,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
0,0013_000.0,Arterial,3.0,D1,AM,1300.90,0.137136,0.062956,0.092654,3716.0,Arterial,StatewideArterial,2040,21284.2,1972.0,0,1000000,SNT,M01-Jan,0.885,1745.220,WD2,1-Monday,1.083,1890.073260,AM,Period,0.408955,772.954728,1.058905,818.485915,3,0.220260,0.92,889.658603,0.239413,0.036316
1,0013_000.0,Arterial,3.0,D2,AM,902.30,0.209797,0.094093,0.064265,3716.0,Arterial,StatewideArterial,2040,21284.2,1368.0,0,1000000,SNT,M01-Jan,0.885,1210.680,WD2,1-Monday,1.083,1311.166440,AM,Period,0.408955,536.207945,1.089006,583.933602,3,0.157140,0.92,634.710437,0.170805,0.025193
2,0013_000.6,Arterial,3.0,D1,AM,1424.50,0.117866,0.055177,0.090718,3716.0,Arterial,StatewideArterial,2040,21260.7,1929.0,0,1000000,SNT,M01-Jan,0.885,1707.165,WD2,1-Monday,1.083,1848.859695,AM,Period,0.408955,756.100239,1.051162,794.783696,3,0.213882,0.92,863.895322,0.232480,0.035563
3,0013_000.6,Arterial,3.0,D2,AM,1167.20,0.148475,0.069225,0.074332,3716.0,Arterial,StatewideArterial,2040,21260.7,1580.0,0,1000000,SNT,M01-Jan,0.885,1398.300,WD2,1-Monday,1.083,1514.358900,AM,Period,0.408955,619.304498,1.064308,659.130574,3,0.177376,0.92,716.446276,0.192800,0.029129
4,0013_001.8,Arterial,3.0,D1,AM,960.60,0.124089,0.072247,0.073287,3680.0,Arterial,StatewideArterial,2040,18379.7,1347.0,0,1000000,SNT,M01-Jan,0.885,1192.095,WD2,1-Monday,1.083,1291.038885,AM,Period,0.408955,527.976683,1.060941,560.152151,3,0.152215,0.92,608.861034,0.165451,0.028726
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5117947,1084_001.0,Arterial,1.0,D1,Daily,525.54,0.285992,0.206645,0.600000,1144.0,Arterial,RuralRecLowVol,2040,683.3,410.0,0,1000000,SNL,S04-Fal,0.926,379.660,WD0,W1-Weekday,0.938,356.121080,PM,Daily,0.094372,33.607917,1.160521,39.002680,3,0.034093,0.88,44.321227,0.038742,0.049185
5117948,1084_001.0,Arterial,1.0,D2,Daily,350.36,0.285992,0.206645,0.400000,1144.0,Arterial,RuralRecLowVol,2040,683.3,273.0,0,1000000,SNL,S04-Fal,0.926,252.798,WD0,W1-Weekday,0.938,237.124524,AM,Daily,0.051266,12.156479,1.160521,14.107845,3,0.012332,0.88,16.031642,0.014014,0.017791
5117949,1084_001.0,Arterial,1.0,D2,Daily,350.36,0.285992,0.206645,0.400000,1144.0,Arterial,RuralRecLowVol,2040,683.3,273.0,0,1000000,SNL,S04-Fal,0.926,252.798,WD0,W1-Weekday,0.938,237.124524,EV,Daily,0.064197,15.222745,1.160521,17.666309,12,0.015443,0.88,20.075351,0.017548,0.022278
5117950,1084_001.0,Arterial,1.0,D2,Daily,350.36,0.285992,0.206645,0.400000,1144.0,Arterial,RuralRecLowVol,2040,683.3,273.0,0,1000000,SNL,S04-Fal,0.926,252.798,WD0,W1-Weekday,0.938,237.124524,MD,Daily,0.077305,18.330909,1.160521,21.273397,6,0.018596,0.88,24.174315,0.021131,0.026827


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
0,0013_000.0,Arterial,3.0,D1,AM,1366.50,0.119722,0.061690,0.093529,3716.0,Arterial,StatewideArterial,2050,21460.0,2007.0,0,1000000,SNT,M01-Jan,0.885,1776.195,WD2,1-Monday,1.083,1923.619185,AM,Period,0.408955,786.673499,1.054790,829.775032,3,0.223298,0.92,901.929382,0.242715,0.036658
1,0013_000.0,Arterial,3.0,D2,AM,945.50,0.208778,0.093284,0.064714,3716.0,Arterial,StatewideArterial,2050,21460.0,1389.0,0,1000000,SNT,M01-Jan,0.885,1229.265,WD2,1-Monday,1.083,1331.293995,AM,Period,0.408955,544.439208,1.088398,592.566367,3,0.159464,0.92,644.093877,0.173330,0.025370
2,0013_000.6,Arterial,3.0,D1,AM,1509.20,0.115359,0.053538,0.091884,3716.0,Arterial,StatewideArterial,2050,21790.9,2002.0,0,1000000,SNT,M01-Jan,0.885,1771.770,WD2,1-Monday,1.083,1918.826910,AM,Period,0.408955,784.713674,1.049841,823.824569,3,0.221697,0.92,895.461488,0.240975,0.036011
3,0013_000.6,Arterial,3.0,D2,AM,1178.10,0.153467,0.071386,0.071726,3716.0,Arterial,StatewideArterial,2050,21790.9,1563.0,0,1000000,SNT,M01-Jan,0.885,1383.255,WD2,1-Monday,1.083,1498.065165,AM,Period,0.408955,612.641095,1.066387,653.312227,3,0.175811,0.92,710.121986,0.191098,0.028115
4,0013_001.8,Arterial,3.0,D1,AM,1017.40,0.122862,0.071948,0.073615,3680.0,Arterial,StatewideArterial,2050,19202.7,1414.0,0,1000000,SNT,M01-Jan,0.885,1251.390,WD2,1-Monday,1.083,1355.255370,AM,Period,0.408955,554.238329,1.060546,587.795515,3,0.159727,0.92,638.908169,0.173616,0.028863
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5089275,1084_001.0,Arterial,1.0,D1,Daily,446.82,0.212837,0.181550,0.600000,1144.0,Arterial,RuralRecLowVol,2050,897.1,538.0,0,1000000,SNL,S04-Fal,0.926,498.188,WD0,W1-Weekday,0.938,467.300344,PM,Daily,0.094372,44.100144,1.133342,49.980558,3,0.043689,0.88,56.796089,0.049647,0.049159
5089276,1084_001.0,Arterial,1.0,D2,Daily,297.88,0.212837,0.181550,0.400000,1144.0,Arterial,RuralRecLowVol,2050,897.1,359.0,0,1000000,SNL,S04-Fal,0.926,332.434,WD0,W1-Weekday,0.938,311.823092,AM,Daily,0.051266,15.985993,1.133342,18.117602,3,0.015837,0.88,20.588184,0.017997,0.017820
5089277,1084_001.0,Arterial,1.0,D2,Daily,297.88,0.212837,0.181550,0.400000,1144.0,Arterial,RuralRecLowVol,2050,897.1,359.0,0,1000000,SNL,S04-Fal,0.926,332.434,WD0,W1-Weekday,0.938,311.823092,EV,Daily,0.064197,20.018188,1.133342,22.687459,12,0.019832,0.88,25.781204,0.022536,0.022314
5089278,1084_001.0,Arterial,1.0,D2,Daily,297.88,0.212837,0.181550,0.400000,1144.0,Arterial,RuralRecLowVol,2050,897.1,359.0,0,1000000,SNL,S04-Fal,0.926,332.434,WD0,W1-Weekday,0.938,311.823092,MD,Daily,0.077305,24.105481,1.133342,27.319761,6,0.023881,0.88,31.045183,0.027137,0.026870


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
0,0013_000.0,Arterial,3.0,D1,AM,1111.20,0.150648,0.067675,0.088065,3716.0,Arterial,StatewideArterial,2030,19859.5,1749.0,0,1000000,SNT,M01-Jan,0.885,1547.865,WD2,1-Monday,1.083,1676.337795,AM,Period,0.408955,685.546562,1.063967,729.398838,3,0.196286,0.92,792.824824,0.213354,0.034520
1,0013_000.0,Arterial,3.0,D2,AM,831.20,0.208855,0.094562,0.065875,3716.0,Arterial,StatewideArterial,2030,19859.5,1308.0,0,1000000,SNT,M01-Jan,0.885,1157.580,WD2,1-Monday,1.083,1253.659140,AM,Period,0.408955,512.690053,1.089052,558.346114,3,0.150255,0.92,606.897950,0.163320,0.025816
2,0013_000.6,Arterial,3.0,D1,AM,1193.50,0.126183,0.058483,0.085016,3716.0,Arterial,StatewideArterial,2030,19417.3,1651.0,0,1000000,SNT,M01-Jan,0.885,1461.135,WD2,1-Monday,1.083,1582.409205,AM,Period,0.408955,647.134004,1.054478,682.388845,3,0.183635,0.92,741.727006,0.199604,0.033328
3,0013_000.6,Arterial,3.0,D2,AM,1108.60,0.140718,0.065127,0.078969,3716.0,Arterial,StatewideArterial,2030,19417.3,1533.0,0,1000000,SNT,M01-Jan,0.885,1356.705,WD2,1-Monday,1.083,1469.311515,AM,Period,0.408955,600.882149,1.060707,637.360021,3,0.171518,0.92,692.782631,0.186432,0.030946
4,0013_001.8,Arterial,3.0,D1,AM,840.70,0.120971,0.074700,0.071485,3680.0,Arterial,StatewideArterial,2030,16972.9,1213.0,0,1000000,SNT,M01-Jan,0.885,1073.505,WD2,1-Monday,1.083,1162.605915,AM,Period,0.408955,475.453390,1.061544,504.714670,3,0.137151,0.92,548.602903,0.149077,0.028013
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5175291,1084_001.0,Arterial,1.0,D1,Daily,489.24,0.261467,0.155016,0.600000,1144.0,Arterial,RuralRecLowVol,2030,535.8,321.0,0,1000000,SNL,S04-Fal,0.926,297.246,WD0,W1-Weekday,0.938,278.816748,PM,Daily,0.094372,26.312540,1.129801,29.727942,3,0.025986,0.88,33.781752,0.029530,0.049109
5175292,1084_001.0,Arterial,1.0,D2,Daily,326.16,0.261467,0.155016,0.400000,1144.0,Arterial,RuralRecLowVol,2030,535.8,214.0,0,1000000,SNL,S04-Fal,0.926,198.164,WD0,W1-Weekday,0.938,185.877832,AM,Daily,0.051266,9.529255,1.129801,10.766165,3,0.009411,0.88,12.234278,0.010694,0.017785
5175293,1084_001.0,Arterial,1.0,D2,Daily,326.16,0.261467,0.155016,0.400000,1144.0,Arterial,RuralRecLowVol,2030,535.8,214.0,0,1000000,SNL,S04-Fal,0.926,198.164,WD0,W1-Weekday,0.938,185.877832,EV,Daily,0.064197,11.932848,1.129801,13.481747,12,0.011785,0.88,15.320167,0.013392,0.022271
5175294,1084_001.0,Arterial,1.0,D2,Daily,326.16,0.261467,0.155016,0.400000,1144.0,Arterial,RuralRecLowVol,2030,535.8,214.0,0,1000000,SNL,S04-Fal,0.926,198.164,WD0,W1-Weekday,0.938,185.877832,MD,Daily,0.077305,14.369284,1.129801,16.234436,6,0.014191,0.88,18.448223,0.016126,0.026818


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
0,0013_000.0,Arterial,3.0,D1,AM,1212.20,0.142386,0.059479,0.091530,3716.0,Arterial,StatewideArterial,2040,20487.5,1875.0,0,1000000,SNT,M01-Jan,0.885,1659.375,WD2,1-Monday,1.083,1797.103125,AM,Period,0.408955,734.934135,1.058216,777.719404,3,0.209289,0.92,845.347178,0.227488,0.035872
1,0013_000.0,Arterial,3.0,D2,AM,849.50,0.204944,0.086404,0.064144,3716.0,Arterial,StatewideArterial,2040,20487.5,1314.0,0,1000000,SNT,M01-Jan,0.885,1162.890,WD2,1-Monday,1.083,1259.409870,AM,Period,0.408955,515.041842,1.084191,558.403576,3,0.150270,0.92,606.960408,0.163337,0.025139
2,0013_000.6,Arterial,3.0,D1,AM,1309.20,0.119004,0.051635,0.090739,3716.0,Arterial,StatewideArterial,2040,19986.4,1814.0,0,1000000,SNT,M01-Jan,0.885,1605.390,WD2,1-Monday,1.083,1738.637370,AM,Period,0.408955,711.024278,1.049618,746.303943,3,0.200835,0.92,811.199938,0.218299,0.035575
3,0013_000.6,Arterial,3.0,D2,AM,1079.70,0.144114,0.063166,0.074833,3716.0,Arterial,StatewideArterial,2040,19986.4,1496.0,0,1000000,SNT,M01-Jan,0.885,1323.960,WD2,1-Monday,1.083,1433.848680,AM,Period,0.408955,586.379449,1.060406,621.800091,3,0.167330,0.92,675.869665,0.181881,0.029339
4,0013_001.8,Arterial,3.0,D1,AM,891.70,0.124818,0.076371,0.072980,3680.0,Arterial,StatewideArterial,2040,17490.7,1276.0,0,1000000,SNT,M01-Jan,0.885,1129.260,WD2,1-Monday,1.083,1222.988580,AM,Period,0.408955,500.147177,1.063149,531.730992,3,0.144492,0.92,577.968469,0.157057,0.028595
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5195771,1084_001.0,Arterial,1.0,D1,Daily,553.08,0.287047,0.225754,0.600000,1144.0,Arterial,RuralRecLowVol,2040,729.2,438.0,0,1000000,SNL,S04-Fal,0.926,405.588,WD0,W1-Weekday,0.938,380.441544,PM,Daily,0.094372,35.903091,1.170286,42.016899,3,0.036728,0.88,47.746477,0.041736,0.049236
5195772,1084_001.0,Arterial,1.0,D2,Daily,368.72,0.287047,0.225754,0.400000,1144.0,Arterial,RuralRecLowVol,2040,729.2,292.0,0,1000000,SNL,S04-Fal,0.926,270.392,WD0,W1-Weekday,0.938,253.627696,AM,Daily,0.051266,13.002535,1.170286,15.216690,3,0.013301,0.88,17.291693,0.015115,0.017831
5195773,1084_001.0,Arterial,1.0,D2,Daily,368.72,0.287047,0.225754,0.400000,1144.0,Arterial,RuralRecLowVol,2040,729.2,292.0,0,1000000,SNL,S04-Fal,0.926,270.392,WD0,W1-Weekday,0.938,253.627696,EV,Daily,0.064197,16.282203,1.170286,19.054841,12,0.016656,0.88,21.653228,0.018928,0.022329
5195774,1084_001.0,Arterial,1.0,D2,Daily,368.72,0.287047,0.225754,0.400000,1144.0,Arterial,RuralRecLowVol,2040,729.2,292.0,0,1000000,SNL,S04-Fal,0.926,270.392,WD0,W1-Weekday,0.938,253.627696,MD,Daily,0.077305,19.606686,1.170286,22.945438,6,0.020057,0.88,26.074362,0.022792,0.026888


,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
0,0013_000.0,Arterial,3.0,D1,AM,1223.60,0.121281,0.060559,0.089566,3716.0,Arterial,StatewideArterial,2050,20511.0,1837.0,0,1000000,SNT,M01-Jan,0.885,1625.745,WD2,1-Monday,1.083,1760.681835,AM,Period,0.408955,720.039470,1.054536,759.307396,3,0.204335,0.92,825.334126,0.222103,0.035105
1,0013_000.0,Arterial,3.0,D2,AM,874.30,0.210111,0.088757,0.063997,3716.0,Arterial,StatewideArterial,2050,20511.0,1313.0,0,1000000,SNT,M01-Jan,0.885,1162.005,WD2,1-Monday,1.083,1258.451415,AM,Period,0.408955,514.649877,1.086401,559.115909,3,0.150462,0.92,607.734684,0.163545,0.025091
2,0013_000.6,Arterial,3.0,D1,AM,1363.50,0.116392,0.051779,0.093181,3716.0,Arterial,StatewideArterial,2050,19998.6,1863.0,0,1000000,SNT,M01-Jan,0.885,1648.755,WD2,1-Monday,1.083,1785.601665,AM,Period,0.408955,730.230557,1.049168,766.134229,3,0.206172,0.92,832.754597,0.224100,0.036514
3,0013_000.6,Arterial,3.0,D2,AM,1047.70,0.157297,0.069295,0.071599,3716.0,Arterial,StatewideArterial,2050,19998.6,1432.0,0,1000000,SNT,M01-Jan,0.885,1267.320,WD2,1-Monday,1.083,1372.507560,AM,Period,0.408955,561.293697,1.066107,598.398977,3,0.161033,0.92,650.433670,0.175036,0.028067
4,0013_001.8,Arterial,3.0,D1,AM,889.40,0.126715,0.078030,0.073434,3680.0,Arterial,StatewideArterial,2050,17493.6,1285.0,0,1000000,SNT,M01-Jan,0.885,1137.225,WD2,1-Monday,1.083,1231.614675,AM,Period,0.408955,503.674861,1.064358,536.090365,3,0.145677,0.92,582.706918,0.158344,0.028792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5285883,1084_001.0,Arterial,1.0,D1,Daily,343.56,0.202934,0.162592,0.600000,1144.0,Arterial,RuralRecLowVol,2050,725.0,435.0,0,1000000,SNL,S04-Fal,0.926,402.810,WD0,W1-Weekday,0.938,377.835780,PM,Daily,0.094372,35.657180,1.121883,40.003171,3,0.034968,0.88,45.458149,0.039736,0.049182
5285884,1084_001.0,Arterial,1.0,D2,Daily,229.04,0.202934,0.162592,0.400000,1144.0,Arterial,RuralRecLowVol,2050,725.0,290.0,0,1000000,SNL,S04-Fal,0.926,268.540,WD0,W1-Weekday,0.938,251.890520,AM,Daily,0.051266,12.913476,1.121883,14.487405,3,0.012664,0.88,16.462960,0.014391,0.017812
5285885,1084_001.0,Arterial,1.0,D2,Daily,229.04,0.202934,0.162592,0.400000,1144.0,Arterial,RuralRecLowVol,2050,725.0,290.0,0,1000000,SNL,S04-Fal,0.926,268.540,WD0,W1-Weekday,0.938,251.890520,EV,Daily,0.064197,16.170681,1.121883,18.141607,12,0.015858,0.88,20.615462,0.018021,0.022304
5285886,1084_001.0,Arterial,1.0,D2,Daily,229.04,0.202934,0.162592,0.400000,1144.0,Arterial,RuralRecLowVol,2050,725.0,290.0,0,1000000,SNL,S04-Fal,0.926,268.540,WD0,W1-Weekday,0.938,251.890520,MD,Daily,0.077305,19.472394,1.121883,21.845741,6,0.019096,0.88,24.824705,0.021700,0.026858


In [184]:
dfs_Peak_VC[0]

,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
0,0013_000.0,Arterial,3.0,D1,AM,1056.10,0.143831,0.064577,0.087986,3716.0,Arterial,StatewideArterial,2019,19000,1672.0,0,1000000,SNT,M01-Jan,0.885,1479.720,WD2,1-Monday,1.083,1602.536760,AM,Period,0.408955,655.365266,1.061055,695.378478,3,0.187131,0.92,755.846171,0.203403,0.034493
1,0013_000.0,Arterial,3.0,D2,AM,782.40,0.207183,0.090874,0.065183,3716.0,Arterial,StatewideArterial,2019,19000,1238.0,0,1000000,SNT,M01-Jan,0.885,1095.630,WD2,1-Monday,1.083,1186.567290,AM,Period,0.408955,485.252512,1.086874,527.408204,3,0.141929,0.92,573.269787,0.154271,0.025540
2,0013_000.6,Arterial,3.0,D1,AM,1174.90,0.119329,0.056260,0.081416,3716.0,Arterial,StatewideArterial,2019,19000,1547.0,0,1000000,SNT,M01-Jan,0.885,1369.095,WD2,1-Monday,1.083,1482.729885,AM,Period,0.408955,606.369657,1.051996,637.898402,3,0.171663,0.92,693.367829,0.186590,0.031914
3,0013_000.6,Arterial,3.0,D2,AM,1186.30,0.125516,0.057658,0.082206,3716.0,Arterial,StatewideArterial,2019,19000,1562.0,0,1000000,SNT,M01-Jan,0.885,1382.370,WD2,1-Monday,1.083,1497.106710,AM,Period,0.408955,612.249130,1.053932,645.269192,3,0.173646,0.92,701.379557,0.188746,0.032224
4,0013_001.3,Arterial,3.0,D1,AM,1419.90,0.112261,0.060638,0.084827,3716.0,Arterial,StatewideArterial,2019,15500,1315.0,0,1000000,SNT,M01-Jan,0.885,1163.775,WD2,1-Monday,1.083,1260.368325,AM,Period,0.408955,515.433807,1.052771,542.633930,3,0.146026,0.92,589.819490,0.158724,0.033254
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5074939,1084_001.0,Arterial,1.0,D1,Daily,548.52,0.218442,0.123824,0.600000,1144.0,Arterial,RuralRecLowVol,2019,350,210.0,0,1000000,SNL,S04-Fal,0.926,194.460,WD0,W1-Weekday,0.938,182.403480,PM,Daily,0.094372,17.213811,1.105601,19.031598,3,0.016636,0.88,21.626816,0.018905,0.049182
5074940,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,AM,Daily,0.051266,6.234092,1.105601,6.892415,3,0.006025,0.88,7.832290,0.006846,0.017812
5074941,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,EV,Daily,0.064197,7.806536,1.105601,8.630910,12,0.007545,0.88,9.807852,0.008573,0.022304
5074942,1084_001.0,Arterial,1.0,D2,Daily,365.68,0.218442,0.123824,0.400000,1144.0,Arterial,RuralRecLowVol,2019,350,140.0,0,1000000,SNL,S04-Fal,0.926,129.640,WD0,W1-Weekday,0.938,121.602320,MD,Daily,0.077305,9.400466,1.105601,10.393160,6,0.009085,0.88,11.810409,0.010324,0.026858


In [185]:
df_Peak_VC.columns

Index(['SEGID', 'FUNCGROUP', 'AREATYPE', 'Dir', 'Period_TDM', 'VOL',
       'PercentTrucksMD', 'PercentTrucksHV', 'VolPercentSegDY', 'Cap1Hr',
       'DOWFACFC', 'SsnGrp', 'FYEAR', 'ForecastAADT', 'ForecastAAPeriodT',
       'Vol_From', 'Vol_To', 'SiteGroupSeason', 'SeasonGroup', 'SeasonFactor',
       'ForecastASeasonPeriodT', 'SiteGroupDOW', 'DOWGroup', 'DOWFactor',
       'ForecastASeasonDOWPeriodT', 'Period', 'HourPercentOf',
       'VolMaxHourPercent', 'ForecastPeakHour', 'FactorTrucks',
       'ForecastPeakHourPCE', 'PeriodHours', 'VCPeakHourPCE', 'PHF',
       'ForecastPeak15MinPCE', 'VCPeak15MinPCE', 'kFactor_effective'],
      dtype='object')

In [186]:
df = dfs_Peak_VC[0]

df = df[(df[nameSegID]=='0015_295.6') & (df['SeasonGroup']=='S03-Sum') & (df['DOWGroup']=='W1-Weekday') & (df['HourPercentOf']=='Period')]

pd.set_option('display.max_columns', None)

display(df)

df2 = df.groupby('SEGID').agg({nameVol:[np.sum],namePercentSegTotal:[np.sum],'ForecastAAPeriodT':[np.sum]})

display(df2)

,SEGID,FUNCGROUP,AREATYPE,Dir,Period_TDM,VOL,PercentTrucksMD,PercentTrucksHV,VolPercentSegDY,Cap1Hr,DOWFACFC,SsnGrp,FYEAR,ForecastAADT,ForecastAAPeriodT,Vol_From,Vol_To,SiteGroupSeason,SeasonGroup,SeasonFactor,ForecastASeasonPeriodT,SiteGroupDOW,DOWGroup,DOWFactor,ForecastASeasonDOWPeriodT,Period,HourPercentOf,VolMaxHourPercent,ForecastPeakHour,FactorTrucks,ForecastPeakHourPCE,PeriodHours,VCPeakHourPCE,PHF,ForecastPeak15MinPCE,VCPeak15MinPCE,kFactor_effective
1465236,0015_295.6,CD Road,4.0,D1,AM,9076.7,0.073044,0.035564,0.026847,7955.0,FreewayI15,FreewayI15,2019,277500,7450.0,0,1000000,SN7,S03-Sum,1.033,7695.850,WDA,W1-Weekday,1.057,8134.513450,AM,Period,0.366372,2980.261029,1.032391,3076.793551,3,0.386775,0.92,3344.340816,0.420407,0.010740
1465237,0015_295.6,CD Road,4.0,D2,AM,7403.7,0.103759,0.049975,0.021898,7955.0,FreewayI15,FreewayI15,2019,277500,6077.0,0,1000000,SN7,S03-Sum,1.033,6277.541,WDA,W1-Weekday,1.057,6635.360837,AM,Period,0.366372,2431.012923,1.045739,2542.205734,3,0.319573,0.92,2763.267102,0.347362,0.008760
1465238,0015_295.6,Freeway,4.0,D1,AM,22987.0,0.072341,0.034454,0.067990,14320.0,FreewayI15,FreewayI15,2019,277500,18867.0,0,1000000,SN7,S03-Sum,1.033,19489.611,WDA,W1-Weekday,1.057,20600.518827,AM,Period,0.366372,7547.461052,1.031695,7786.680140,3,0.543763,0.92,8463.782761,0.591046,0.027198
1465239,0015_295.6,Freeway,4.0,D2,AM,17697.9,0.137502,0.060521,0.052346,14320.0,FreewayI15,FreewayI15,2019,277500,14526.0,0,1000000,SN7,S03-Sum,1.033,15005.358,WDA,W1-Weekday,1.057,15860.663406,AM,Period,0.366372,5810.908954,1.057761,6146.553391,3,0.429229,0.92,6681.036295,0.466553,0.020940
1465240,0015_295.6,Managed,4.0,D1,AM,3683.5,0.000000,0.000000,0.010895,3476.0,FreewayI15,FreewayI15,2019,277500,3023.0,0,1000000,SN7,S03-Sum,1.033,3122.759,WDA,W1-Weekday,1.057,3300.756263,AM,Period,0.366372,1209.305918,1.000000,1209.305918,3,0.347902,0.92,1314.462955,0.378154,0.004358
1465241,0015_295.6,Managed,4.0,D2,AM,757.9,0.000000,0.000000,0.002242,3476.0,FreewayI15,FreewayI15,2019,277500,622.0,0,1000000,SN7,S03-Sum,1.033,642.526,WDA,W1-Weekday,1.057,679.149982,AM,Period,0.366372,248.821793,1.000000,248.821793,3,0.071583,0.92,270.458471,0.077807,0.000897
1481620,0015_295.6,CD Road,4.0,D1,EV,11744.5,0.143216,0.000000,0.034737,7955.0,FreewayI15,FreewayI15,2019,277500,9640.0,0,1000000,SN7,S03-Sum,1.033,9958.120,WDA,W1-Weekday,1.057,10525.732840,EV,Period,0.225733,2376.004450,1.028643,2444.060808,12,0.307236,0.92,2656.587835,0.333952,0.008562
1481621,0015_295.6,CD Road,4.0,D2,EV,11345.9,0.127720,0.000000,0.033558,7955.0,FreewayI15,FreewayI15,2019,277500,9312.0,0,1000000,SN7,S03-Sum,1.033,9619.296,WDA,W1-Weekday,1.057,10167.595872,EV,Period,0.225733,2295.161145,1.025544,2353.788808,12,0.295888,0.92,2558.466096,0.321617,0.008271
1481622,0015_295.6,Freeway,4.0,D1,EV,30226.5,0.168501,0.000000,0.089402,14320.0,FreewayI15,FreewayI15,2019,277500,24809.0,0,1000000,SN7,S03-Sum,1.033,25627.697,WDA,W1-Weekday,1.057,27088.475729,EV,Period,0.225733,6114.760829,1.033700,6320.829675,12,0.441399,0.92,6870.467038,0.479781,0.022035
1481623,0015_295.6,Freeway,4.0,D2,EV,33429.7,0.148646,0.000000,0.098877,14320.0,FreewayI15,FreewayI15,2019,277500,27438.0,0,1000000,SN7,S03-Sum,1.033,28343.454,WDA,W1-Weekday,1.057,29959.030878,EV,Period,0.225733,6762.739636,1.029729,6963.790832,12,0.486298,0.92,7569.337861,0.528585,0.024370


,VOL,VolPercentSegDY,ForecastAAPeriodT
,sum,sum,sum
SEGID,,,
0015_295.6,338095.4,1.0,277501.0


In [187]:
dfs_Peak_VC_MaxDirFG = []

for df_Peak_VC in dfs_Peak_VC:

    df_Peak_VC_MaxDirFG = df_Peak_VC.groupby([nameSegID,namePeriod,'SeasonGroup','DOWGroup','HourPercentOf'], as_index=False).agg({nameDir:[np.size],'ForecastPeakHourPCE':[np.sum],'ForecastPeak15MinPCE':[np.sum],'VCPeakHourPCE':[np.max],'VCPeak15MinPCE':[np.max]})

    df_Peak_VC_MaxDirFG.columns = ([nameSegID,namePeriod,'SeasonGroup','DOWGroup','HourPercentOf','numRecords','ForecastPeakHourPCE','ForecastPeak15MinPCE','VCPeakHourPCE','VCPeak15MinPCE'])
    
    dfs_Peak_VC_MaxDirFG.append(df_Peak_VC_MaxDirFG)
    
    display(df_Peak_VC_MaxDirFG)
                                            
    #df_Peak_VC_wGroups = pd.DataFrame.merge(df_VCGroups,df_Peak_VC,on=('SeasonGroup','DOWGroup','Period','HourPercentOf'))

,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0013_000.0,AM,M01-Jan,1-Monday,Daily,2,1479.518528,1608.172313,0.238889,0.259662
1,0013_000.0,AM,M01-Jan,1-Monday,Period,2,1222.786681,1329.115958,0.187131,0.203403
2,0013_000.0,AM,M01-Jan,2-Tuesday,Daily,2,1517.817176,1649.801278,0.245073,0.266383
3,0013_000.0,AM,M01-Jan,2-Tuesday,Period,2,1260.086610,1369.659359,0.192839,0.209608
4,0013_000.0,AM,M01-Jan,3-Wednesday,Daily,2,1520.136988,1652.322813,0.245447,0.266791
...,...,...,...,...,...,...,...,...,...,...
2476027,WFRC_8218,PM,S04-Fal,6-Saturday,Period,2,1891.084884,2055.527048,0.331973,0.360841
2476028,WFRC_8218,PM,S04-Fal,7-Sunday,Daily,2,1255.918532,1365.128839,0.255788,0.278030
2476029,WFRC_8218,PM,S04-Fal,7-Sunday,Period,2,1407.590047,1529.989181,0.247097,0.268584
2476030,WFRC_8218,PM,S04-Fal,W1-Weekday,Daily,2,2800.647402,3044.181959,0.570397,0.619996


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0013_000.0,AM,M01-Jan,1-Monday,Daily,2,1520.906716,1653.159474,0.245572,0.266926
1,0013_000.0,AM,M01-Jan,1-Monday,Period,2,1268.291678,1378.577911,0.194802,0.211742
2,0013_000.0,AM,M01-Jan,2-Tuesday,Daily,2,1560.276733,1695.952971,0.251928,0.273835
3,0013_000.0,AM,M01-Jan,2-Tuesday,Period,2,1306.979693,1420.630101,0.200745,0.218201
4,0013_000.0,AM,M01-Jan,3-Wednesday,Daily,2,1562.661440,1698.545043,0.252313,0.274254
...,...,...,...,...,...,...,...,...,...,...
2492411,WFRC_8218,PM,S04-Fal,6-Saturday,Period,2,2032.448233,2209.182862,0.351137,0.381670
2492412,WFRC_8218,PM,S04-Fal,7-Sunday,Daily,2,1356.545500,1474.505978,0.276282,0.300307
2492413,WFRC_8218,PM,S04-Fal,7-Sunday,Period,2,1512.810941,1644.359718,0.261361,0.284089
2492414,WFRC_8218,PM,S04-Fal,W1-Weekday,Daily,2,3025.041461,3288.088544,0.616098,0.669672


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0013_000.0,AM,M01-Jan,1-Monday,Daily,2,1559.472157,1695.078431,0.251799,0.273694
1,0013_000.0,AM,M01-Jan,1-Monday,Period,2,1294.694822,1407.276981,0.197038,0.214171
2,0013_000.0,AM,M01-Jan,2-Tuesday,Daily,2,1599.840475,1738.957038,0.258317,0.280779
3,0013_000.0,AM,M01-Jan,2-Tuesday,Period,2,1334.188240,1450.204609,0.203048,0.220705
4,0013_000.0,AM,M01-Jan,3-Wednesday,Daily,2,1602.285650,1741.614837,0.258711,0.281208
...,...,...,...,...,...,...,...,...,...,...
2511867,WFRC_8219,PM,S04-Fal,6-Saturday,Period,2,697.278170,792.361556,0.188082,0.213730
2511868,WFRC_8219,PM,S04-Fal,7-Sunday,Daily,2,466.443023,530.048890,0.149023,0.169345
2511869,WFRC_8219,PM,S04-Fal,7-Sunday,Period,2,531.249101,603.692160,0.143298,0.162839
2511870,WFRC_8219,PM,S04-Fal,W1-Weekday,Daily,2,639.061308,726.206032,0.204173,0.232015


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0013_000.0,AM,M01-Jan,1-Monday,Daily,2,1591.903900,1730.330326,0.257035,0.279386
1,0013_000.0,AM,M01-Jan,1-Monday,Period,2,1338.085781,1454.441066,0.210238,0.228520
2,0013_000.0,AM,M01-Jan,2-Tuesday,Daily,2,1633.111742,1775.121459,0.263689,0.286618
3,0013_000.0,AM,M01-Jan,2-Tuesday,Period,2,1378.902797,1498.807388,0.216651,0.235490
4,0013_000.0,AM,M01-Jan,3-Wednesday,Daily,2,1635.607769,1777.834531,0.264092,0.287056
...,...,...,...,...,...,...,...,...,...,...
2518011,WFRC_8219,PM,S04-Fal,6-Saturday,Period,2,685.173513,778.606265,0.187648,0.213236
2518012,WFRC_8219,PM,S04-Fal,7-Sunday,Daily,2,482.000379,547.727703,0.153994,0.174993
2518013,WFRC_8219,PM,S04-Fal,7-Sunday,Period,2,522.026687,593.212144,0.142967,0.162462
2518014,WFRC_8219,PM,S04-Fal,W1-Weekday,Daily,2,660.376031,750.427308,0.210983,0.239753


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0013_000.0,AM,M01-Jan,1-Monday,Daily,2,1590.226113,1728.506645,0.256764,0.279091
1,0013_000.0,AM,M01-Jan,1-Monday,Period,2,1321.486990,1436.398903,0.206204,0.224135
2,0013_000.0,AM,M01-Jan,2-Tuesday,Daily,2,1631.390525,1773.250570,0.263411,0.286316
3,0013_000.0,AM,M01-Jan,2-Tuesday,Period,2,1361.797677,1480.214866,0.212494,0.230972
4,0013_000.0,AM,M01-Jan,3-Wednesday,Daily,2,1633.883921,1775.960783,0.263813,0.286754
...,...,...,...,...,...,...,...,...,...,...
2555899,WFRC_8219,PM,S04-Fal,6-Saturday,Period,2,1795.092801,2039.878183,0.500222,0.568434
2555900,WFRC_8219,PM,S04-Fal,7-Sunday,Daily,2,1037.773089,1179.287601,0.331557,0.376769
2555901,WFRC_8219,PM,S04-Fal,7-Sunday,Period,2,1367.662832,1554.162309,0.381114,0.433084
2555902,WFRC_8219,PM,S04-Fal,W1-Weekday,Daily,2,1421.825591,1615.710899,0.454257,0.516202


...

,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0013_000.0,AM,M01-Jan,1-Monday,Daily,2,1655.010650,1798.924620,0.267225,0.290461
1,0013_000.0,AM,M01-Jan,1-Monday,Period,2,1402.419517,1524.369040,0.220260,0.239413
2,0013_000.0,AM,M01-Jan,2-Tuesday,Daily,2,1697.852067,1845.491377,0.274142,0.297980
3,0013_000.0,AM,M01-Jan,2-Tuesday,Period,2,1445.198972,1570.868448,0.226979,0.246716
4,0013_000.0,AM,M01-Jan,3-Wednesday,Daily,2,1700.447042,1848.312002,0.274561,0.298436
...,...,...,...,...,...,...,...,...,...,...
2489339,WFRC_8218,PM,S04-Fal,6-Saturday,Period,2,2452.906499,2666.202716,0.423865,0.460723
2489340,WFRC_8218,PM,S04-Fal,7-Sunday,Daily,2,1700.454968,1848.320618,0.346321,0.376436
2489341,WFRC_8218,PM,S04-Fal,7-Sunday,Period,2,1825.770383,1984.533025,0.315495,0.342929
2489342,WFRC_8218,PM,S04-Fal,W1-Weekday,Daily,2,3791.945631,4121.680034,0.772283,0.839438


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0013_000.0,AM,M01-Jan,1-Monday,Daily,2,1662.771350,1807.360163,0.268478,0.291823
1,0013_000.0,AM,M01-Jan,1-Monday,Period,2,1422.341398,1546.023259,0.223298,0.242715
2,0013_000.0,AM,M01-Jan,2-Tuesday,Daily,2,1705.813659,1854.145282,0.275427,0.299378
3,0013_000.0,AM,M01-Jan,2-Tuesday,Period,2,1465.728551,1593.183208,0.230109,0.250119
4,0013_000.0,AM,M01-Jan,3-Wednesday,Daily,2,1708.420802,1856.979133,0.275848,0.299835
...,...,...,...,...,...,...,...,...,...,...
2475003,WFRC_8218,PM,S04-Fal,6-Saturday,Period,2,2496.943394,2714.068907,0.428794,0.466081
2475004,WFRC_8218,PM,S04-Fal,7-Sunday,Daily,2,1853.915389,2015.125423,0.377580,0.410413
2475005,WFRC_8218,PM,S04-Fal,7-Sunday,Period,2,1858.548338,2020.161237,0.319164,0.346918
2475006,WFRC_8218,PM,S04-Fal,W1-Weekday,Daily,2,4134.156147,4493.647986,0.841987,0.915203


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0013_000.0,AM,M01-Jan,1-Monday,Daily,2,1548.387974,1683.030407,0.250009,0.271749
1,0013_000.0,AM,M01-Jan,1-Monday,Period,2,1287.744952,1399.722774,0.196286,0.213354
2,0013_000.0,AM,M01-Jan,2-Tuesday,Daily,2,1588.469368,1726.597139,0.256481,0.278783
3,0013_000.0,AM,M01-Jan,2-Tuesday,Period,2,1327.026370,1442.419967,0.202274,0.219863
4,0013_000.0,AM,M01-Jan,3-Wednesday,Daily,2,1590.897164,1729.236048,0.256873,0.279209
...,...,...,...,...,...,...,...,...,...,...
2508795,WFRC_8219,PM,S04-Fal,6-Saturday,Period,2,642.941821,730.615706,0.178336,0.202655
2508796,WFRC_8219,PM,S04-Fal,7-Sunday,Daily,2,451.350280,512.898045,0.144201,0.163865
2508797,WFRC_8219,PM,S04-Fal,7-Sunday,Period,2,489.850793,556.648629,0.135873,0.154401
2508798,WFRC_8219,PM,S04-Fal,W1-Weekday,Daily,2,618.383137,702.708110,0.197566,0.224507


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0013_000.0,AM,M01-Jan,1-Monday,Daily,2,1591.009416,1729.358061,0.256886,0.279224
1,0013_000.0,AM,M01-Jan,1-Monday,Period,2,1336.122979,1452.307586,0.209289,0.227488
2,0013_000.0,AM,M01-Jan,2-Tuesday,Daily,2,1632.194104,1774.124026,0.263536,0.286452
3,0013_000.0,AM,M01-Jan,2-Tuesday,Period,2,1376.880122,1496.608829,0.215674,0.234428
4,0013_000.0,AM,M01-Jan,3-Wednesday,Daily,2,1634.688728,1776.835574,0.263939,0.286890
...,...,...,...,...,...,...,...,...,...,...
2512891,WFRC_8219,PM,S04-Fal,6-Saturday,Period,2,696.550910,791.535125,0.189341,0.215160
2512892,WFRC_8219,PM,S04-Fal,7-Sunday,Daily,2,482.653865,548.470301,0.154203,0.175230
2512893,WFRC_8219,PM,S04-Fal,7-Sunday,Period,2,530.695009,603.062510,0.144257,0.163928
2512894,WFRC_8219,PM,S04-Fal,W1-Weekday,Daily,2,661.271355,751.444722,0.211269,0.240078


,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
0,0013_000.0,AM,M01-Jan,1-Monday,Daily,2,1590.993385,1729.340636,0.256896,0.279235
1,0013_000.0,AM,M01-Jan,1-Monday,Period,2,1318.423305,1433.068810,0.204335,0.222103
2,0013_000.0,AM,M01-Jan,2-Tuesday,Daily,2,1632.177658,1774.106150,0.263546,0.286463
3,0013_000.0,AM,M01-Jan,2-Tuesday,Period,2,1358.640537,1476.783192,0.210568,0.228878
4,0013_000.0,AM,M01-Jan,3-Wednesday,Daily,2,1634.672257,1776.817671,0.263949,0.286901
...,...,...,...,...,...,...,...,...,...,...
2556923,WFRC_8219,PM,S04-Fal,6-Saturday,Period,2,1662.290718,1888.966724,0.460816,0.523655
2556924,WFRC_8219,PM,S04-Fal,7-Sunday,Daily,2,1052.150538,1195.625612,0.336150,0.381989
2556925,WFRC_8219,PM,S04-Fal,7-Sunday,Period,2,1266.482284,1439.184413,0.351091,0.398967
2556926,WFRC_8219,PM,S04-Fal,W1-Weekday,Daily,2,1441.523756,1638.095177,0.460551,0.523353


In [188]:
df = dfs_Peak_VC_MaxDirFG[0]

df = df[(df[nameSegID]=='0015_295.6') & (df['SeasonGroup']=='S03-Sum') & (df['DOWGroup']=='5-Friday') & (df['HourPercentOf']=='Period')]

pd.set_option('display.max_columns', None)

display(df)

,SEGID,Period,SeasonGroup,DOWGroup,HourPercentOf,numRecords,ForecastPeakHourPCE,ForecastPeak15MinPCE,VCPeakHourPCE,VCPeak15MinPCE
18665,0015_295.6,AM,S03-Sum,5-Friday,Period,6,23010.250643,25011.142003,0.595521,0.647306
18921,0015_295.6,EV,S03-Sum,5-Friday,Period,6,19007.105491,20659.897273,0.489456,0.532017
19177,0015_295.6,MD,S03-Sum,5-Friday,Period,6,22024.043252,23939.177448,0.522109,0.567510
19433,0015_295.6,PM,S03-Sum,5-Friday,Period,6,25310.707781,27511.638893,0.583843,0.634612


# Visualize

In [189]:
df_VCGroups

,SeasonCode,DOWPkCode,SeasonGroup,SeasonName,DOWGroup,DOWPkName,HourPercentOf,Period,VCGroupCode
VCGroupID,,,,,,,,,
0,Win,WkAM,S01-Win,Winter,W1-Weekday,Weekday AM Peak,Period,AM,WinWkAM
1,Spr,WkAM,S01-Win,Spring,W1-Weekday,Weekday AM Peak,Period,AM,SprWkAM
2,Sum,WkAM,S01-Win,Summer,W1-Weekday,Weekday AM Peak,Period,AM,SumWkAM
3,Fal,WkAM,S01-Win,Fall,W1-Weekday,Weekday AM Peak,Period,AM,FalWkAM
4,Jan,WkAM,M01-Jan,January,W1-Weekday,Weekday AM Peak,Period,AM,JanWkAM
5,Feb,WkAM,M02-Feb,February,W1-Weekday,Weekday AM Peak,Period,AM,FebWkAM
6,Mar,WkAM,M03-Mar,March,W1-Weekday,Weekday AM Peak,Period,AM,MarWkAM
7,Apr,WkAM,M04-Apr,April,W1-Weekday,Weekday AM Peak,Period,AM,AprWkAM
8,May,WkAM,M05-May,May,W1-Weekday,Weekday AM Peak,Period,AM,MayWkAM


In [190]:
dfs_Peak_VC_MaxDirFG_wVCGroups = []

for df_Peak_VC_MaxDirFG in dfs_Peak_VC_MaxDirFG:
    
    df_Peak_VC_MaxDirFG_wVCGroups = pd.DataFrame.merge(df_Peak_VC_MaxDirFG,df_VCGroups,on=('SeasonGroup','DOWGroup','HourPercentOf','Period'))

    df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups[['VCGroupCode','SEGID','VCPeak15MinPCE']]
    
    df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups.round({'VCPeak15MinPCE': 2})
    
    df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups.pivot(index="SEGID", columns="VCGroupCode",values='VCPeak15MinPCE')
    
    df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups.reset_index()
    
    dfs_Peak_VC_MaxDirFG_wVCGroups.append(df_Peak_VC_MaxDirFG_wVCGroups)

    display(df_Peak_VC_MaxDirFG_wVCGroups)
    

VCGroupCode,SEGID,AprFrPM,AprSaMD,AprSuMD,AprWkAM,AprWkPM,AugFrPM,AugSaMD,AugSuMD,AugWkAM,AugWkPM,DecFrPM,DecSaMD,DecSuMD,DecWkAM,DecWkPM,FalFrPM,FalSaMD,FalSuMD,FalWkAM,FalWkPM,FebFrPM,FebSaMD,FebSuMD,FebWkAM,FebWkPM,JanFrPM,JanSaMD,JanSuMD,JanWkAM,JanWkPM,JulFrPM,JulSaMD,JulSuMD,JulWkAM,JulWkPM,JunFrPM,JunSaMD,JunSuMD,JunWkAM,JunWkPM,MarFrPM,MarSaMD,MarSuMD,MarWkAM,MarWkPM,MayFrPM,MaySaMD,MaySuMD,MayWkAM,MayWkPM,NovFrPM,NovSaMD,NovSuMD,NovWkAM,NovWkPM,OctFrPM,OctSaMD,OctSuMD,OctWkAM,OctWkPM,SepFrPM,SepSaMD,SepSuMD,SepWkAM,SepWkPM,SprFrPM,SprSaMD,SprSuMD,SprWkAM,SprWkPM,SumFrPM,SumSaMD,SumSuMD,SumWkAM,SumWkPM,WinFrPM,WinSaMD,WinSuMD,WinWkAM,WinWkPM
0,0013_000.0,0.35,0.22,0.16,0.24,0.35,0.35,0.22,0.15,0.24,0.36,0.34,0.21,0.15,0.22,0.33,0.34,0.21,0.15,0.22,0.32,0.34,0.21,0.15,0.23,0.33,0.32,0.20,0.14,0.21,0.31,0.34,0.21,0.15,0.23,0.35,0.35,0.22,0.15,0.24,0.36,0.35,0.22,0.16,0.24,0.34,0.35,0.22,0.16,0.24,0.35,0.35,0.22,0.15,0.23,0.33,0.36,0.23,0.16,0.24,0.35,0.36,0.23,0.16,0.24,0.36,0.34,0.21,0.15,0.22,0.32,0.34,0.21,0.15,0.22,0.32,0.34,0.21,0.15,0.22,0.32
1,0013_000.6,0.35,0.22,0.16,0.22,0.31,0.35,0.21,0.15,0.22,0.32,0.33,0.21,0.14,0.21,0.29,0.33,0.21,0.14,0.20,0.29,0.34,0.21,0.15,0.21,0.30,0.32,0.20,0.14,0.19,0.28,0.34,0.21,0.15,0.21,0.32,0.35,0.21,0.15,0.22,0.33,0.35,0.22,0.16,0.22,0.31,0.35,0.22,0.16,0.22,0.32,0.34,0.22,0.15,0.21,0.30,0.36,0.22,0.16,0.22,0.32,0.36,0.22,0.15,0.23,0.32,0.33,0.21,0.14,0.20,0.29,0.33,0.21,0.14,0.20,0.29,0.33,0.21,0.14,0.20,0.29
2,0013_001.3,0.28,0.18,0.13,0.19,0.25,0.29,0.17,0.12,0.19,0.26,0.27,0.17,0.12,0.18,0.24,0.27,0.17,0.12,0.17,0.24,0.27,0.17,0.12,0.18,0.24,0.26,0.16,0.11,0.16,0.22,0.27,0.17,0.12,0.18,0.26,0.28,0.17,0.12,0.18,0.26,0.28,0.18,0.13,0.18,0.25,0.28,0.18,0.13,0.19,0.26,0.28,0.18,0.12,0.18,0.25,0.29,0.18,0.13,0.19,0.26,0.29,0.18,0.12,0.19,0.26,0.27,0.17,0.12,0.17,0.24,0.27,0.17,0.12,0.17,0.24,0.27,0.17,0.12,0.17,0.24
3,0013_001.6,0.29,0.19,0.13,0.22,0.25,0.29,0.18,0.13,0.22,0.26,0.28,0.17,0.12,0.21,0.24,0.28,0.17,0.12,0.20,0.23,0.28,0.17,0.12,0.21,0.24,0.27,0.16,0.11,0.19,0.22,0.28,0.17,0.12,0.21,0.25,0.29,0.18,0.13,0.22,0.26,0.29,0.19,0.13,0.22,0.25,0.29,0.18,0.13,0.22,0.25,0.29,0.18,0.13,0.21,0.24,0.30,0.19,0.13,0.22,0.26,0.30,0.19,0.13,0.22,0.26,0.28,0.17,0.12,0.20,0.23,0.28,0.17,0.12,0.20,0.23,0.28,0.17,0.12,0.20,0.23
4,0013_001.8,0.30,0.19,0.14,0.21,0.26,0.30,0.18,0.13,0.21,0.27,0.29,0.18,0.13,0.20,0.24,0.29,0.18,0.12,0.19,0.24,0.29,0.18,0.13,0.20,0.25,0.28,0.17,0.12,0.18,0.23,0.29,0.18,0.13,0.20,0.26,0.30,0.18,0.13,0.21,0.27,0.30,0.19,0.14,0.21,0.26,0.30,0.19,0.14,0.21,0.26,0.30,0.19,0.13,0.20,0.25,0.31,0.19,0.14,0.21,0.26,0.31,0.19,0.13,0.21,0.27,0.29,0.18,0.12,0.19,0.24,0.29,0.18,0.12,0.19,0.24,0.29,0.18,0.12,0.19,0.24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2413,WFRC_8208,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2414,WFRC_8209,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.01,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02,0.02,0.01,0.01,0.02,0.02
2415,WFRC_8213,0.03,0.02,0.02,0.02,0.03,0.03,0.02,0.02,0.02,0.03,0.03,0.02,0.02,0.02,0.03,0.03,0.02,0.02,0.02,0.03,0.03

VCGroupCode,SEGID,AprFrPM,AprSaMD,AprSuMD,AprWkAM,AprWkPM,AugFrPM,AugSaMD,AugSuMD,AugWkAM,AugWkPM,DecFrPM,DecSaMD,DecSuMD,DecWkAM,DecWkPM,FalFrPM,FalSaMD,FalSuMD,FalWkAM,FalWkPM,FebFrPM,FebSaMD,FebSuMD,FebWkAM,FebWkPM,JanFrPM,JanSaMD,JanSuMD,JanWkAM,JanWkPM,JulFrPM,JulSaMD,JulSuMD,JulWkAM,JulWkPM,JunFrPM,JunSaMD,JunSuMD,JunWkAM,JunWkPM,MarFrPM,MarSaMD,MarSuMD,MarWkAM,MarWkPM,MayFrPM,MaySaMD,MaySuMD,MayWkAM,MayWkPM,NovFrPM,NovSaMD,NovSuMD,NovWkAM,NovWkPM,OctFrPM,OctSaMD,OctSuMD,OctWkAM,OctWkPM,SepFrPM,SepSaMD,SepSuMD,SepWkAM,SepWkPM,SprFrPM,SprSaMD,SprSuMD,SprWkAM,SprWkPM,SumFrPM,SumSaMD,SumSuMD,SumWkAM,SumWkPM,WinFrPM,WinSaMD,WinSuMD,WinWkAM,WinWkPM
0,0013_000.0,0.36,0.23,0.16,0.25,0.36,0.36,0.22,0.16,0.25,0.37,0.35,0.22,0.15,0.23,0.34,0.35,0.21,0.15,0.23,0.33,0.35,0.22,0.15,0.23,0.34,0.33,0.20,0.14,0.22,0.32,0.35,0.21,0.15,0.24,0.36,0.36,0.22,0.16,0.25,0.37,0.36,0.23,0.16,0.25,0.36,0.36,0.23,0.16,0.25,0.36,0.36,0.23,0.16,0.24,0.35,0.37,0.23,0.16,0.25,0.36,0.37,0.23,0.16,0.25,0.37,0.35,0.21,0.15,0.23,0.33,0.35,0.21,0.15,0.23,0.33,0.35,0.21,0.15,0.23,0.33
1,0013_000.6,0.36,0.23,0.16,0.24,0.34,0.36,0.22,0.16,0.24,0.35,0.34,0.21,0.15,0.22,0.32,0.34,0.21,0.15,0.22,0.31,0.35,0.21,0.15,0.22,0.32,0.33,0.20,0.14,0.21,0.30,0.35,0.21,0.15,0.23,0.34,0.36,0.22,0.16,0.24,0.35,0.36,0.23,0.16,0.24,0.33,0.36,0.23,0.16,0.24,0.34,0.36,0.23,0.16,0.23,0.32,0.37,0.23,0.16,0.24,0.34,0.37,0.23,0.16,0.24,0.35,0.34,0.21,0.15,0.22,0.31,0.34,0.21,0.15,0.22,0.31,0.34,0.21,0.15,0.22,0.31
2,0013_001.3,0.29,0.22,0.15,0.21,0.27,0.29,0.21,0.14,0.20,0.27,0.29,0.21,0.15,0.20,0.26,0.29,0.21,0.15,0.20,0.26,0.29,0.22,0.15,0.20,0.26,0.28,0.21,0.14,0.19,0.25,0.28,0.20,0.13,0.20,0.27,0.29,0.21,0.14,0.20,0.28,0.30,0.22,0.15,0.20,0.27,0.29,0.22,0.15,0.20,0.27,0.29,0.22,0.15,0.20,0.26,0.29,0.21,0.15,0.20,0.27,0.29,0.22,0.15,0.21,0.27,0.29,0.21,0.15,0.20,0.26,0.29,0.21,0.15,0.20,0.26,0.29,0.21,0.15,0.20,0.26
3,0013_001.6,0.30,0.22,0.15,0.21,0.27,0.29,0.21,0.15,0.21,0.27,0.29,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.20,0.25,0.30,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.19,0.25,0.29,0.21,0.14,0.20,0.27,0.29,0.21,0.15,0.21,0.27,0.30,0.22,0.16,0.21,0.27,0.30,0.22,0.15,0.21,0.27,0.30,0.22,0.15,0.20,0.26,0.30,0.22,0.15,0.21,0.27,0.30,0.22,0.15,0.21,0.27,0.29,0.22,0.15,0.20,0.25,0.29,0.22,0.15,0.20,0.25,0.29,0.22,0.15,0.20,0.25
4,0013_001.8,0.31,0.20,0.14,0.21,0.28,0.32,0.19,0.14,0.21,0.29,0.30,0.19,0.13,0.19,0.27,0.30,0.19,0.13,0.19,0.26,0.30,0.19,0.13,0.19,0.27,0.29,0.18,0.12,0.18,0.25,0.30,0.19,0.13,0.20,0.29,0.31,0.19,0.14,0.20,0.29,0.31,0.20,0.14,0.20,0.28,0.31,0.20,0.14,0.21,0.29,0.31,0.20,0.14,0.20,0.27,0.32,0.20,0.14,0.21,0.29,0.32,0.20,0.14,0.21,0.29,0.30,0.19,0.13,0.19,0.26,0.30,0.19,0.13,0.19,0.26,0.30,0.19,0.13,0.19,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2429,WFRC_8213,0.04,0.03,0.02,0.02,0.04,0.04,0.03,0.02,0.02,0.04,0.04,0.03,0.02,0.02,0.03,0.04,0.03,0.02,0.02,0.03,0.04,0.03,0.02,0.02,0.03,0.04,0.03,0.02,0.02,0.03,0.04,0.03,0.02,0.02,0.03,0.04,0.03,0.02,0.02,0.04,0.04,0.03,0.02,0.02,0.04,0.04,0.03,0.02,0.02,0.04,0.04,0.03,0.02,0.02,0.03,0.04,0.03,0.02,0.02,0.03,0.04,0.03,0.02,0.02,0.04,0.04,0.03,0.02,0.02,0.03,0.04,0.03,0.02,0.02,0.03,0.04,0.03,0.02,0.02,0.03
2430,WFRC_8215,0.12,0.09,0.07,0.08,0.12,0.12,0.09,0.07,0.08,0.13,0.11,0.08,0.06,0.07,0.11,0.10,0.08,0.06,0.07,0.11,0.10,0.08,0.06,0.07,0.11,0.10,0.07,0.06,0.07,0.10,0.11,0.09,0.07,0.08,0.13,0.12,0.09,0.07,0.08,0.13,0.11,0.08,0.07,0.08,0.12,0.12,0.09,0.07,0.08,0.13,0.11,0.09,0.07,0.08,0.11,0.12,0.09,0.07,0.08,0.12,0.12,0.09,0.07,0.08,0.13,0.10,0.08,0.06,0.07,0.11,0.10,0.08,0.06,0.07,0.11,0.10,0.08,0.06,0.07,0.11
2431,WFRC_8216,0.17,0.05,0.04,0.18,0.23,0.18,0.05,0.04,0.19,0.24,0.16,0.05,0.04,0.17,0.22,0.16,0.05,0.04,0.17,0.22,0.16

VCGroupCode,SEGID,AprFrPM,AprSaMD,AprSuMD,AprWkAM,AprWkPM,AugFrPM,AugSaMD,AugSuMD,AugWkAM,AugWkPM,DecFrPM,DecSaMD,DecSuMD,DecWkAM,DecWkPM,FalFrPM,FalSaMD,FalSuMD,FalWkAM,FalWkPM,FebFrPM,FebSaMD,FebSuMD,FebWkAM,FebWkPM,JanFrPM,JanSaMD,JanSuMD,JanWkAM,JanWkPM,JulFrPM,JulSaMD,JulSuMD,JulWkAM,JulWkPM,JunFrPM,JunSaMD,JunSuMD,JunWkAM,JunWkPM,MarFrPM,MarSaMD,MarSuMD,MarWkAM,MarWkPM,MayFrPM,MaySaMD,MaySuMD,MayWkAM,MayWkPM,NovFrPM,NovSaMD,NovSuMD,NovWkAM,NovWkPM,OctFrPM,OctSaMD,OctSuMD,OctWkAM,OctWkPM,SepFrPM,SepSaMD,SepSuMD,SepWkAM,SepWkPM,SprFrPM,SprSaMD,SprSuMD,SprWkAM,SprWkPM,SumFrPM,SumSaMD,SumSuMD,SumWkAM,SumWkPM,WinFrPM,WinSaMD,WinSuMD,WinWkAM,WinWkPM
0,0013_000.0,0.37,0.24,0.17,0.25,0.37,0.37,0.23,0.16,0.25,0.38,0.36,0.22,0.15,0.24,0.35,0.35,0.22,0.15,0.23,0.34,0.36,0.22,0.16,0.24,0.35,0.34,0.21,0.15,0.22,0.32,0.36,0.22,0.16,0.24,0.37,0.37,0.23,0.16,0.25,0.38,0.37,0.24,0.17,0.25,0.36,0.37,0.23,0.17,0.25,0.37,0.37,0.23,0.16,0.24,0.35,0.38,0.24,0.17,0.25,0.37,0.38,0.24,0.16,0.26,0.38,0.35,0.22,0.15,0.23,0.34,0.35,0.22,0.15,0.23,0.34,0.35,0.22,0.15,0.23,0.34
1,0013_000.6,0.36,0.23,0.16,0.23,0.34,0.36,0.22,0.16,0.23,0.35,0.34,0.21,0.15,0.22,0.32,0.34,0.21,0.15,0.22,0.31,0.35,0.21,0.15,0.22,0.32,0.33,0.20,0.14,0.21,0.30,0.35,0.21,0.15,0.23,0.34,0.36,0.22,0.16,0.23,0.35,0.36,0.23,0.16,0.23,0.34,0.36,0.23,0.16,0.24,0.34,0.36,0.23,0.16,0.23,0.33,0.37,0.23,0.16,0.24,0.34,0.37,0.23,0.16,0.24,0.35,0.34,0.21,0.15,0.22,0.31,0.34,0.21,0.15,0.22,0.31,0.34,0.21,0.15,0.22,0.31
2,0013_001.3,0.30,0.23,0.16,0.21,0.28,0.30,0.22,0.15,0.21,0.28,0.29,0.22,0.15,0.20,0.27,0.29,0.22,0.15,0.20,0.27,0.30,0.22,0.15,0.20,0.27,0.29,0.22,0.15,0.19,0.26,0.29,0.21,0.14,0.20,0.28,0.30,0.21,0.15,0.21,0.29,0.31,0.23,0.16,0.21,0.28,0.30,0.22,0.15,0.21,0.28,0.30,0.22,0.15,0.20,0.27,0.30,0.22,0.15,0.21,0.28,0.30,0.22,0.15,0.21,0.28,0.29,0.22,0.15,0.20,0.27,0.29,0.22,0.15,0.20,0.27,0.29,0.22,0.15,0.20,0.27
3,0013_001.6,0.30,0.22,0.15,0.21,0.27,0.29,0.21,0.15,0.20,0.27,0.29,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.20,0.26,0.30,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.19,0.25,0.29,0.21,0.14,0.20,0.27,0.29,0.21,0.15,0.20,0.28,0.30,0.22,0.16,0.21,0.27,0.30,0.22,0.15,0.21,0.27,0.30,0.22,0.15,0.20,0.26,0.30,0.22,0.15,0.20,0.27,0.30,0.22,0.15,0.21,0.27,0.29,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.20,0.26
4,0013_001.8,0.31,0.20,0.14,0.21,0.28,0.32,0.19,0.14,0.21,0.29,0.30,0.19,0.13,0.20,0.26,0.30,0.19,0.13,0.19,0.26,0.30,0.19,0.13,0.20,0.27,0.29,0.18,0.12,0.18,0.25,0.30,0.19,0.13,0.20,0.29,0.31,0.19,0.14,0.21,0.29,0.31,0.20,0.14,0.21,0.28,0.31,0.20,0.14,0.21,0.29,0.31,0.20,0.14,0.20,0.27,0.32,0.20,0.14,0.21,0.29,0.32,0.20,0.14,0.21,0.29,0.30,0.19,0.13,0.19,0.26,0.30,0.19,0.13,0.19,0.26,0.30,0.19,0.13,0.19,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2448,WFRC_8215,0.15,0.10,0.07,0.12,0.16,0.15,0.09,0.07,0.12,0.16,0.15,0.09,0.06,0.12,0.15,0.14,0.09,0.06,0.11,0.15,0.15,0.09,0.06,0.12,0.15,0.14,0.09,0.06,0.11,0.14,0.15,0.09,0.06,0.12,0.16,0.15,0.09,0.07,0.12,0.16,0.15,0.10,0.07,0.12,0.16,0.15,0.10,0.07,0.12,0.16,0.15,0.10,0.07,0.12,0.15,0.16,0.10,0.07,0.13,0.16,0.16,0.10,0.07,0.13,0.16,0.14,0.09,0.06,0.11,0.15,0.14,0.09,0.06,0.11,0.15,0.14,0.09,0.06,0.11,0.15
2449,WFRC_8216,0.20,0.06,0.05,0.20,0.26,0.21,0.06,0.05,0.21,0.27,0.19,0.06,0.04,0.18,0.25,0.19,0.06,0.04,0.18,0.25,0.19,0.06,0.04,0.19,0.25,0.19,0.06,0.04,0.18,0.25,0.20,0.06,0.04,0.20,0.27,0.21,0.06,0.04,0.21,0.27,0.20,0.06,0.04,0.20,0.26,0.20,0.06,0.05,0.20,0.27,0.20,0.06,0.04,0.19,0.26,0.21,0.07,0.04,0.20,0.27,0.21,0.06,0.05,0.20,0.27,0.19,0.06,0.04,0.18,0.25,0.19,0.06,0.04,0.18,0.25,0.19,0.06,0.04,0.18,0.25
2450,WFRC_8217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

VCGroupCode,SEGID,AprFrPM,AprSaMD,AprSuMD,AprWkAM,AprWkPM,AugFrPM,AugSaMD,AugSuMD,AugWkAM,AugWkPM,DecFrPM,DecSaMD,DecSuMD,DecWkAM,DecWkPM,FalFrPM,FalSaMD,FalSuMD,FalWkAM,FalWkPM,FebFrPM,FebSaMD,FebSuMD,FebWkAM,FebWkPM,JanFrPM,JanSaMD,JanSuMD,JanWkAM,JanWkPM,JulFrPM,JulSaMD,JulSuMD,JulWkAM,JulWkPM,JunFrPM,JunSaMD,JunSuMD,JunWkAM,JunWkPM,MarFrPM,MarSaMD,MarSuMD,MarWkAM,MarWkPM,MayFrPM,MaySaMD,MaySuMD,MayWkAM,MayWkPM,NovFrPM,NovSaMD,NovSuMD,NovWkAM,NovWkPM,OctFrPM,OctSaMD,OctSuMD,OctWkAM,OctWkPM,SepFrPM,SepSaMD,SepSuMD,SepWkAM,SepWkPM,SprFrPM,SprSaMD,SprSuMD,SprWkAM,SprWkPM,SumFrPM,SumSaMD,SumSuMD,SumWkAM,SumWkPM,WinFrPM,WinSaMD,WinSuMD,WinWkAM,WinWkPM
0,0013_000.0,0.38,0.24,0.17,0.27,0.38,0.38,0.23,0.16,0.27,0.40,0.36,0.23,0.16,0.25,0.36,0.36,0.22,0.16,0.25,0.36,0.37,0.23,0.16,0.25,0.36,0.35,0.21,0.15,0.24,0.34,0.37,0.22,0.16,0.26,0.39,0.38,0.23,0.17,0.27,0.40,0.38,0.24,0.17,0.27,0.38,0.38,0.24,0.17,0.27,0.39,0.38,0.24,0.16,0.26,0.37,0.39,0.24,0.17,0.27,0.39,0.39,0.24,0.17,0.27,0.40,0.36,0.22,0.16,0.25,0.36,0.36,0.22,0.16,0.25,0.36,0.36,0.22,0.16,0.25,0.36
1,0013_000.6,0.36,0.23,0.17,0.26,0.35,0.37,0.22,0.16,0.26,0.37,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33,0.36,0.22,0.15,0.24,0.33,0.34,0.21,0.14,0.23,0.31,0.35,0.22,0.15,0.25,0.36,0.37,0.22,0.16,0.25,0.37,0.37,0.23,0.16,0.25,0.35,0.37,0.23,0.16,0.26,0.36,0.36,0.23,0.16,0.25,0.34,0.38,0.24,0.16,0.26,0.36,0.38,0.24,0.16,0.26,0.36,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33
2,0013_001.3,0.31,0.23,0.16,0.22,0.29,0.31,0.22,0.15,0.22,0.29,0.30,0.23,0.16,0.22,0.28,0.30,0.23,0.16,0.21,0.28,0.31,0.23,0.16,0.22,0.28,0.30,0.22,0.15,0.21,0.27,0.30,0.22,0.14,0.22,0.29,0.30,0.22,0.15,0.22,0.30,0.32,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.22,0.30,0.31,0.23,0.16,0.22,0.28,0.31,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.22,0.29,0.30,0.23,0.16,0.21,0.28,0.30,0.23,0.16,0.21,0.28,0.30,0.23,0.16,0.21,0.28
3,0013_001.6,0.31,0.23,0.16,0.21,0.28,0.30,0.22,0.15,0.20,0.28,0.30,0.23,0.16,0.20,0.26,0.30,0.23,0.16,0.20,0.26,0.31,0.23,0.16,0.20,0.27,0.30,0.22,0.15,0.19,0.25,0.29,0.21,0.14,0.20,0.27,0.30,0.22,0.15,0.20,0.28,0.31,0.23,0.16,0.21,0.28,0.31,0.23,0.16,0.20,0.28,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.21,0.28,0.30,0.23,0.16,0.20,0.26,0.30,0.23,0.16,0.20,0.26,0.30,0.23,0.16,0.20,0.26
4,0013_001.8,0.32,0.21,0.15,0.21,0.28,0.33,0.20,0.14,0.21,0.30,0.31,0.19,0.13,0.20,0.27,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.14,0.20,0.27,0.30,0.18,0.13,0.18,0.25,0.31,0.19,0.14,0.20,0.29,0.32,0.20,0.14,0.21,0.30,0.32,0.21,0.15,0.21,0.28,0.32,0.20,0.15,0.21,0.29,0.32,0.20,0.14,0.20,0.27,0.33,0.21,0.15,0.21,0.29,0.33,0.21,0.14,0.21,0.29,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.13,0.19,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2454,WFRC_8215,0.21,0.13,0.09,0.18,0.22,0.21,0.13,0.09,0.18,0.23,0.20,0.13,0.09,0.17,0.21,0.20,0.12,0.09,0.17,0.21,0.20,0.13,0.09,0.17,0.21,0.19,0.12,0.08,0.16,0.20,0.20,0.12,0.09,0.17,0.23,0.21,0.13,0.09,0.18,0.23,0.21,0.13,0.09,0.18,0.22,0.21,0.13,0.09,0.18,0.23,0.21,0.13,0.09,0.17,0.22,0.22,0.14,0.09,0.18,0.23,0.22,0.14,0.09,0.18,0.23,0.20,0.12,0.09,0.17,0.21,0.20,0.12,0.09,0.17,0.21,0.20,0.12,0.09,0.17,0.21
2455,WFRC_8216,0.25,0.08,0.06,0.24,0.29,0.27,0.08,0.06,0.25,0.30,0.24,0.08,0.05,0.22,0.27,0.24,0.07,0.05,0.22,0.27,0.24,0.08,0.05,0.23,0.27,0.24,0.07,0.05,0.22,0.27,0.25,0.08,0.05,0.24,0.29,0.26,0.08,0.06,0.25,0.30,0.25,0.08,0.06,0.24,0.29,0.26,0.08,0.06,0.24,0.29,0.25,0.08,0.06,0.23,0.28,0.26,0.08,0.06,0.24,0.29,0.27,0.08,0.06,0.24,0.29,0.24,0.07,0.05,0.22,0.27,0.24,0.07,0.05,0.22,0.27,0.24,0.07,0.05,0.22,0.27
2456,WFRC_8217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

VCGroupCode,SEGID,AprFrPM,AprSaMD,AprSuMD,AprWkAM,AprWkPM,AugFrPM,AugSaMD,AugSuMD,AugWkAM,AugWkPM,DecFrPM,DecSaMD,DecSuMD,DecWkAM,DecWkPM,FalFrPM,FalSaMD,FalSuMD,FalWkAM,FalWkPM,FebFrPM,FebSaMD,FebSuMD,FebWkAM,FebWkPM,JanFrPM,JanSaMD,JanSuMD,JanWkAM,JanWkPM,JulFrPM,JulSaMD,JulSuMD,JulWkAM,JulWkPM,JunFrPM,JunSaMD,JunSuMD,JunWkAM,JunWkPM,MarFrPM,MarSaMD,MarSuMD,MarWkAM,MarWkPM,MayFrPM,MaySaMD,MaySuMD,MayWkAM,MayWkPM,NovFrPM,NovSaMD,NovSuMD,NovWkAM,NovWkPM,OctFrPM,OctSaMD,OctSuMD,OctWkAM,OctWkPM,SepFrPM,SepSaMD,SepSuMD,SepWkAM,SepWkPM,SprFrPM,SprSaMD,SprSuMD,SprWkAM,SprWkPM,SumFrPM,SumSaMD,SumSuMD,SumWkAM,SumWkPM,WinFrPM,WinSaMD,WinSuMD,WinWkAM,WinWkPM
0,0013_000.0,0.38,0.24,0.17,0.26,0.39,0.38,0.23,0.16,0.26,0.41,0.36,0.23,0.16,0.25,0.37,0.36,0.22,0.16,0.24,0.36,0.37,0.23,0.16,0.25,0.37,0.35,0.21,0.15,0.23,0.35,0.37,0.22,0.16,0.25,0.40,0.38,0.23,0.17,0.26,0.41,0.38,0.24,0.17,0.26,0.39,0.38,0.24,0.17,0.26,0.40,0.38,0.24,0.16,0.25,0.38,0.39,0.24,0.17,0.27,0.40,0.39,0.24,0.17,0.27,0.41,0.36,0.22,0.16,0.24,0.36,0.36,0.22,0.16,0.24,0.36,0.36,0.22,0.16,0.24,0.36
1,0013_000.6,0.36,0.23,0.17,0.26,0.35,0.37,0.22,0.16,0.26,0.37,0.35,0.22,0.15,0.25,0.33,0.35,0.22,0.15,0.24,0.33,0.36,0.22,0.16,0.25,0.34,0.34,0.21,0.14,0.23,0.31,0.35,0.22,0.15,0.25,0.36,0.37,0.22,0.16,0.26,0.37,0.37,0.23,0.17,0.26,0.35,0.37,0.23,0.16,0.26,0.36,0.36,0.23,0.16,0.25,0.34,0.38,0.24,0.16,0.27,0.36,0.38,0.24,0.16,0.27,0.37,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33
2,0013_001.3,0.31,0.23,0.16,0.23,0.29,0.31,0.22,0.15,0.22,0.29,0.30,0.23,0.16,0.22,0.28,0.30,0.23,0.16,0.21,0.27,0.31,0.23,0.16,0.22,0.28,0.30,0.22,0.15,0.21,0.27,0.30,0.22,0.14,0.22,0.29,0.30,0.22,0.15,0.22,0.30,0.32,0.23,0.16,0.23,0.29,0.31,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.22,0.28,0.31,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.23,0.29,0.30,0.23,0.16,0.21,0.27,0.30,0.23,0.16,0.21,0.27,0.30,0.23,0.16,0.21,0.27
3,0013_001.6,0.31,0.23,0.16,0.20,0.27,0.30,0.22,0.15,0.20,0.27,0.30,0.23,0.16,0.19,0.26,0.30,0.23,0.16,0.19,0.26,0.31,0.23,0.16,0.20,0.26,0.30,0.22,0.15,0.19,0.25,0.30,0.21,0.14,0.19,0.27,0.30,0.22,0.15,0.20,0.28,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.20,0.26,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.20,0.27,0.30,0.23,0.16,0.19,0.26,0.30,0.23,0.16,0.19,0.26,0.30,0.23,0.16,0.19,0.26
4,0013_001.8,0.32,0.21,0.15,0.21,0.29,0.33,0.20,0.14,0.21,0.30,0.31,0.19,0.13,0.19,0.27,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.14,0.19,0.27,0.30,0.18,0.13,0.18,0.25,0.31,0.19,0.14,0.20,0.29,0.32,0.20,0.14,0.20,0.30,0.32,0.21,0.15,0.20,0.28,0.32,0.20,0.15,0.21,0.29,0.32,0.20,0.14,0.20,0.28,0.33,0.21,0.15,0.21,0.29,0.33,0.21,0.14,0.21,0.30,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.13,0.19,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2491,WFRC_8215,0.27,0.18,0.12,0.24,0.29,0.28,0.17,0.12,0.24,0.30,0.27,0.17,0.12,0.23,0.27,0.26,0.16,0.11,0.22,0.27,0.27,0.17,0.12,0.23,0.28,0.25,0.16,0.11,0.21,0.26,0.27,0.16,0.12,0.23,0.29,0.28,0.17,0.12,0.24,0.30,0.28,0.18,0.12,0.24,0.29,0.28,0.17,0.12,0.24,0.29,0.27,0.17,0.12,0.23,0.28,0.29,0.18,0.12,0.24,0.30,0.28,0.18,0.12,0.24,0.30,0.26,0.16,0.11,0.22,0.27,0.26,0.16,0.11,0.22,0.27,0.26,0.16,0.11,0.22,0.27
2492,WFRC_8216,0.31,0.10,0.07,0.30,0.38,0.33,0.10,0.07,0.31,0.39,0.29,0.09,0.07,0.28,0.36,0.30,0.09,0.07,0.28,0.36,0.30,0.09,0.07,0.29,0.36,0.29,0.09,0.06,0.27,0.36,0.31,0.09,0.07,0.30,0.39,0.32,0.10,0.07,0.31,0.39,0.31,0.10,0.07,0.30,0.38,0.32,0.10,0.07,0.30,0.38,0.31,0.10,0.07,0.29,0.37,0.33,0.10,0.07,0.30,0.38,0.33,0.10,0.07,0.31,0.39,0.30,0.09,0.07,0.28,0.36,0.30,0.09,0.07,0.28,0.36,0.30,0.09,0.07,0.28,0.36
2493,WFRC_8217,0.89,0.57,0.40,0.59,0.71,0.90,0.55,0.39,0.59,0.74,0.86,0.54,0.37,0.56,0.67,0.85,0.53,0.37,0.55,0.66,0.87

...

VCGroupCode,SEGID,AprFrPM,AprSaMD,AprSuMD,AprWkAM,AprWkPM,AugFrPM,AugSaMD,AugSuMD,AugWkAM,AugWkPM,DecFrPM,DecSaMD,DecSuMD,DecWkAM,DecWkPM,FalFrPM,FalSaMD,FalSuMD,FalWkAM,FalWkPM,FebFrPM,FebSaMD,FebSuMD,FebWkAM,FebWkPM,JanFrPM,JanSaMD,JanSuMD,JanWkAM,JanWkPM,JulFrPM,JulSaMD,JulSuMD,JulWkAM,JulWkPM,JunFrPM,JunSaMD,JunSuMD,JunWkAM,JunWkPM,MarFrPM,MarSaMD,MarSuMD,MarWkAM,MarWkPM,MayFrPM,MaySaMD,MaySuMD,MayWkAM,MayWkPM,NovFrPM,NovSaMD,NovSuMD,NovWkAM,NovWkPM,OctFrPM,OctSaMD,OctSuMD,OctWkAM,OctWkPM,SepFrPM,SepSaMD,SepSuMD,SepWkAM,SepWkPM,SprFrPM,SprSaMD,SprSuMD,SprWkAM,SprWkPM,SumFrPM,SumSaMD,SumSuMD,SumWkAM,SumWkPM,WinFrPM,WinSaMD,WinSuMD,WinWkAM,WinWkPM
0,0013_000.0,0.39,0.25,0.18,0.28,0.39,0.40,0.24,0.17,0.28,0.41,0.38,0.24,0.16,0.26,0.37,0.38,0.23,0.16,0.26,0.37,0.38,0.24,0.17,0.26,0.37,0.36,0.22,0.15,0.25,0.35,0.38,0.23,0.17,0.27,0.40,0.39,0.24,0.17,0.28,0.41,0.40,0.25,0.18,0.28,0.39,0.39,0.25,0.18,0.28,0.40,0.39,0.25,0.17,0.27,0.38,0.41,0.25,0.18,0.28,0.40,0.41,0.25,0.17,0.29,0.41,0.38,0.23,0.16,0.26,0.37,0.38,0.23,0.16,0.26,0.37,0.38,0.23,0.16,0.26,0.37
1,0013_000.6,0.39,0.25,0.18,0.27,0.37,0.39,0.24,0.17,0.27,0.39,0.37,0.23,0.16,0.26,0.35,0.37,0.23,0.16,0.25,0.35,0.38,0.23,0.16,0.26,0.36,0.36,0.22,0.15,0.24,0.33,0.38,0.23,0.16,0.26,0.38,0.39,0.24,0.17,0.27,0.39,0.39,0.25,0.18,0.27,0.37,0.39,0.25,0.17,0.27,0.38,0.39,0.25,0.17,0.26,0.36,0.40,0.25,0.18,0.28,0.38,0.40,0.25,0.17,0.28,0.39,0.37,0.23,0.16,0.25,0.35,0.37,0.23,0.16,0.25,0.35,0.37,0.23,0.16,0.25,0.35
2,0013_001.3,0.34,0.26,0.18,0.24,0.32,0.34,0.25,0.17,0.24,0.32,0.34,0.25,0.17,0.23,0.31,0.33,0.25,0.17,0.23,0.30,0.34,0.25,0.17,0.24,0.31,0.33,0.25,0.17,0.22,0.29,0.33,0.24,0.16,0.23,0.32,0.34,0.24,0.17,0.24,0.33,0.35,0.26,0.18,0.24,0.32,0.34,0.25,0.17,0.24,0.32,0.34,0.25,0.18,0.24,0.31,0.34,0.25,0.17,0.24,0.32,0.34,0.25,0.17,0.24,0.32,0.33,0.25,0.17,0.23,0.30,0.33,0.25,0.17,0.23,0.30,0.33,0.25,0.17,0.23,0.30
3,0013_001.6,0.33,0.24,0.17,0.22,0.28,0.32,0.23,0.16,0.22,0.28,0.32,0.24,0.16,0.21,0.27,0.32,0.24,0.16,0.21,0.27,0.32,0.24,0.17,0.21,0.27,0.31,0.23,0.16,0.20,0.26,0.31,0.23,0.15,0.21,0.28,0.32,0.23,0.16,0.22,0.29,0.33,0.24,0.17,0.22,0.28,0.33,0.24,0.16,0.22,0.28,0.32,0.24,0.17,0.21,0.27,0.32,0.24,0.16,0.22,0.28,0.32,0.24,0.16,0.22,0.28,0.32,0.24,0.16,0.21,0.27,0.32,0.24,0.16,0.21,0.27,0.32,0.24,0.16,0.21,0.27
4,0013_001.8,0.34,0.22,0.15,0.22,0.30,0.34,0.21,0.15,0.22,0.31,0.33,0.20,0.14,0.21,0.28,0.32,0.20,0.14,0.21,0.28,0.33,0.20,0.14,0.21,0.29,0.31,0.19,0.13,0.20,0.27,0.33,0.20,0.14,0.22,0.31,0.34,0.21,0.15,0.22,0.31,0.34,0.22,0.15,0.22,0.30,0.34,0.21,0.15,0.23,0.31,0.34,0.21,0.15,0.22,0.29,0.35,0.22,0.15,0.23,0.31,0.35,0.22,0.15,0.23,0.31,0.32,0.20,0.14,0.21,0.28,0.32,0.20,0.14,0.21,0.28,0.32,0.20,0.14,0.21,0.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2426,WFRC_8213,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.04,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05,0.05,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05,0.04,0.03,0.02,0.02,0.05
2427,WFRC_8215,0.19,0.12,0.09,0.17,0.21,0.20,0.12,0.08,0.17,0.22,0.19,0.12,0.08,0.16,0.20,0.19,0.11,0.08,0.16,0.19,0.19,0.12,0.08,0.16,0.20,0.18,0.11,0.08,0.15,0.18,0.19,0.11,0.08,0.16,0.21,0.19,0.12,0.09,0.17,0.22,0.20,0.12,0.09,0.17,0.21,0.19,0.12,0.09,0.17,0.21,0.19,0.12,0.08,0.16,0.20,0.20,0.13,0.09,0.17,0.21,0.20,0.13,0.09,0.17,0.22,0.19,0.11,0.08,0.16,0.19,0.19,0.11,0.08,0.16,0.19,0.19,0.11,0.08,0.16,0.19
2428,WFRC_8216,0.26,0.08,0.06,0.25,0.33,0.27,0.08,0.06,0.26,0.34,0.24,0.08,0.06,0.23,0.31,0.24,0.08,0.06,0.23,0.31,0.25

VCGroupCode,SEGID,AprFrPM,AprSaMD,AprSuMD,AprWkAM,AprWkPM,AugFrPM,AugSaMD,AugSuMD,AugWkAM,AugWkPM,DecFrPM,DecSaMD,DecSuMD,DecWkAM,DecWkPM,FalFrPM,FalSaMD,FalSuMD,FalWkAM,FalWkPM,FebFrPM,FebSaMD,FebSuMD,FebWkAM,FebWkPM,JanFrPM,JanSaMD,JanSuMD,JanWkAM,JanWkPM,JulFrPM,JulSaMD,JulSuMD,JulWkAM,JulWkPM,JunFrPM,JunSaMD,JunSuMD,JunWkAM,JunWkPM,MarFrPM,MarSaMD,MarSuMD,MarWkAM,MarWkPM,MayFrPM,MaySaMD,MaySuMD,MayWkAM,MayWkPM,NovFrPM,NovSaMD,NovSuMD,NovWkAM,NovWkPM,OctFrPM,OctSaMD,OctSuMD,OctWkAM,OctWkPM,SepFrPM,SepSaMD,SepSuMD,SepWkAM,SepWkPM,SprFrPM,SprSaMD,SprSuMD,SprWkAM,SprWkPM,SumFrPM,SumSaMD,SumSuMD,SumWkAM,SumWkPM,WinFrPM,WinSaMD,WinSuMD,WinWkAM,WinWkPM
0,0013_000.0,0.39,0.25,0.18,0.28,0.40,0.40,0.24,0.17,0.28,0.42,0.38,0.24,0.16,0.27,0.38,0.38,0.23,0.16,0.26,0.38,0.38,0.24,0.17,0.27,0.38,0.36,0.22,0.16,0.25,0.36,0.38,0.23,0.17,0.27,0.41,0.40,0.24,0.17,0.28,0.42,0.40,0.25,0.18,0.28,0.40,0.40,0.25,0.18,0.29,0.41,0.39,0.25,0.17,0.27,0.39,0.41,0.26,0.18,0.29,0.41,0.41,0.25,0.17,0.29,0.42,0.38,0.23,0.16,0.26,0.38,0.38,0.23,0.16,0.26,0.38,0.38,0.23,0.16,0.26,0.38
1,0013_000.6,0.40,0.25,0.18,0.28,0.39,0.40,0.25,0.17,0.28,0.40,0.38,0.24,0.17,0.27,0.37,0.38,0.24,0.16,0.26,0.36,0.39,0.24,0.17,0.27,0.37,0.37,0.22,0.16,0.25,0.34,0.39,0.24,0.17,0.27,0.40,0.40,0.24,0.17,0.28,0.41,0.40,0.25,0.18,0.28,0.39,0.40,0.25,0.18,0.28,0.40,0.40,0.25,0.17,0.27,0.38,0.41,0.26,0.18,0.29,0.40,0.41,0.26,0.18,0.29,0.40,0.38,0.24,0.16,0.26,0.36,0.38,0.24,0.16,0.26,0.36,0.38,0.24,0.16,0.26,0.36
2,0013_001.3,0.36,0.27,0.19,0.25,0.34,0.35,0.26,0.18,0.25,0.34,0.35,0.26,0.18,0.25,0.32,0.35,0.26,0.18,0.24,0.32,0.36,0.27,0.18,0.25,0.32,0.34,0.26,0.18,0.24,0.31,0.34,0.25,0.17,0.24,0.33,0.35,0.25,0.17,0.25,0.34,0.36,0.27,0.19,0.25,0.33,0.36,0.26,0.18,0.25,0.34,0.36,0.26,0.18,0.25,0.32,0.36,0.26,0.18,0.25,0.33,0.36,0.26,0.18,0.25,0.34,0.35,0.26,0.18,0.24,0.32,0.35,0.26,0.18,0.24,0.32,0.35,0.26,0.18,0.24,0.32
3,0013_001.6,0.34,0.25,0.18,0.23,0.30,0.34,0.24,0.17,0.22,0.30,0.33,0.25,0.17,0.22,0.29,0.33,0.25,0.17,0.21,0.29,0.34,0.25,0.17,0.22,0.29,0.33,0.25,0.17,0.21,0.28,0.33,0.24,0.16,0.22,0.30,0.33,0.24,0.17,0.22,0.31,0.35,0.26,0.18,0.22,0.30,0.34,0.25,0.17,0.22,0.30,0.34,0.25,0.18,0.22,0.29,0.34,0.25,0.17,0.22,0.30,0.34,0.25,0.17,0.23,0.30,0.33,0.25,0.17,0.21,0.29,0.33,0.25,0.17,0.21,0.29,0.33,0.25,0.17,0.21,0.29
4,0013_001.8,0.35,0.23,0.16,0.23,0.32,0.36,0.22,0.15,0.23,0.33,0.34,0.21,0.15,0.22,0.30,0.34,0.21,0.15,0.21,0.30,0.34,0.21,0.15,0.22,0.30,0.33,0.20,0.14,0.20,0.28,0.34,0.21,0.15,0.22,0.32,0.35,0.22,0.15,0.23,0.33,0.36,0.23,0.16,0.23,0.32,0.35,0.22,0.16,0.23,0.32,0.35,0.22,0.15,0.22,0.31,0.36,0.23,0.16,0.23,0.32,0.36,0.23,0.16,0.23,0.33,0.34,0.21,0.15,0.21,0.30,0.34,0.21,0.15,0.21,0.30,0.34,0.21,0.15,0.21,0.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2412,WFRC_8213,0.06,0.05,0.03,0.03,0.07,0.06,0.05,0.03,0.03,0.07,0.06,0.05,0.03,0.03,0.07,0.06,0.05,0.03,0.03,0.07,0.06,0.05,0.03,0.03,0.07,0.06,0.05,0.03,0.03,0.07,0.06,0.04,0.03,0.03,0.07,0.06,0.04,0.03,0.03,0.08,0.06,0.05,0.03,0.03,0.07,0.06,0.05,0.03,0.03,0.08,0.06,0.05,0.03,0.03,0.07,0.06,0.05,0.03,0.03,0.07,0.06,0.05,0.03,0.03,0.07,0.06,0.05,0.03,0.03,0.07,0.06,0.05,0.03,0.03,0.07,0.06,0.05,0.03,0.03,0.07
2413,WFRC_8215,0.24,0.16,0.11,0.21,0.27,0.25,0.15,0.11,0.21,0.28,0.24,0.15,0.10,0.20,0.25,0.24,0.15,0.10,0.19,0.25,0.24,0.15,0.10,0.20,0.25,0.23,0.14,0.10,0.18,0.23,0.24,0.15,0.10,0.20,0.27,0.25,0.15,0.11,0.21,0.28,0.25,0.16,0.11,0.21,0.26,0.25,0.16,0.11,0.21,0.27,0.24,0.16,0.11,0.20,0.26,0.25,0.16,0.11,0.21,0.27,0.25,0.16,0.11,0.22,0.28,0.24,0.15,0.10,0.19,0.25,0.24,0.15,0.10,0.19,0.25,0.24,0.15,0.10,0.19,0.25
2414,WFRC_8216,0.31,0.10,0.07,0.31,0.40,0.33,0.10,0.07,0.32,0.41,0.29,0.09,0.07,0.28,0.37,0.29,0.09,0.07,0.28,0.37,0.30

VCGroupCode,SEGID,AprFrPM,AprSaMD,AprSuMD,AprWkAM,AprWkPM,AugFrPM,AugSaMD,AugSuMD,AugWkAM,AugWkPM,DecFrPM,DecSaMD,DecSuMD,DecWkAM,DecWkPM,FalFrPM,FalSaMD,FalSuMD,FalWkAM,FalWkPM,FebFrPM,FebSaMD,FebSuMD,FebWkAM,FebWkPM,JanFrPM,JanSaMD,JanSuMD,JanWkAM,JanWkPM,JulFrPM,JulSaMD,JulSuMD,JulWkAM,JulWkPM,JunFrPM,JunSaMD,JunSuMD,JunWkAM,JunWkPM,MarFrPM,MarSaMD,MarSuMD,MarWkAM,MarWkPM,MayFrPM,MaySaMD,MaySuMD,MayWkAM,MayWkPM,NovFrPM,NovSaMD,NovSuMD,NovWkAM,NovWkPM,OctFrPM,OctSaMD,OctSuMD,OctWkAM,OctWkPM,SepFrPM,SepSaMD,SepSuMD,SepWkAM,SepWkPM,SprFrPM,SprSaMD,SprSuMD,SprWkAM,SprWkPM,SumFrPM,SumSaMD,SumSuMD,SumWkAM,SumWkPM,WinFrPM,WinSaMD,WinSuMD,WinWkAM,WinWkPM
0,0013_000.0,0.37,0.23,0.17,0.25,0.36,0.37,0.23,0.16,0.25,0.38,0.35,0.22,0.15,0.24,0.34,0.35,0.22,0.15,0.23,0.34,0.36,0.22,0.16,0.24,0.34,0.34,0.21,0.14,0.22,0.32,0.36,0.22,0.15,0.24,0.37,0.37,0.23,0.16,0.25,0.38,0.37,0.23,0.17,0.25,0.36,0.37,0.23,0.17,0.25,0.37,0.37,0.23,0.16,0.24,0.35,0.38,0.24,0.17,0.25,0.37,0.38,0.24,0.16,0.26,0.38,0.35,0.22,0.15,0.23,0.34,0.35,0.22,0.15,0.23,0.34,0.35,0.22,0.15,0.23,0.34
1,0013_000.6,0.35,0.23,0.16,0.23,0.34,0.36,0.22,0.15,0.23,0.35,0.34,0.21,0.15,0.22,0.32,0.34,0.21,0.15,0.22,0.31,0.35,0.21,0.15,0.22,0.32,0.33,0.20,0.14,0.21,0.30,0.34,0.21,0.15,0.22,0.34,0.36,0.22,0.16,0.23,0.35,0.36,0.23,0.16,0.23,0.33,0.36,0.22,0.16,0.23,0.34,0.35,0.23,0.15,0.23,0.32,0.37,0.23,0.16,0.24,0.34,0.37,0.23,0.16,0.24,0.35,0.34,0.21,0.15,0.22,0.31,0.34,0.21,0.15,0.22,0.31,0.34,0.21,0.15,0.22,0.31
2,0013_001.3,0.30,0.22,0.16,0.21,0.28,0.30,0.22,0.15,0.20,0.28,0.29,0.22,0.15,0.20,0.27,0.29,0.22,0.15,0.20,0.26,0.30,0.22,0.15,0.20,0.27,0.29,0.22,0.15,0.19,0.26,0.29,0.21,0.14,0.20,0.28,0.29,0.21,0.15,0.20,0.28,0.30,0.22,0.16,0.21,0.28,0.30,0.22,0.15,0.21,0.28,0.30,0.22,0.15,0.20,0.27,0.30,0.22,0.15,0.20,0.28,0.30,0.22,0.15,0.21,0.28,0.29,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.20,0.26
3,0013_001.6,0.30,0.22,0.15,0.21,0.27,0.29,0.21,0.15,0.20,0.27,0.29,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.20,0.26,0.30,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.19,0.25,0.28,0.21,0.14,0.20,0.27,0.29,0.21,0.15,0.20,0.28,0.30,0.22,0.16,0.21,0.27,0.30,0.22,0.15,0.20,0.27,0.30,0.22,0.15,0.20,0.26,0.30,0.22,0.15,0.20,0.27,0.30,0.22,0.15,0.21,0.27,0.29,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.20,0.26,0.29,0.22,0.15,0.20,0.26
4,0013_001.8,0.31,0.20,0.14,0.21,0.28,0.32,0.19,0.14,0.21,0.29,0.30,0.19,0.13,0.20,0.26,0.30,0.18,0.13,0.19,0.26,0.30,0.19,0.13,0.20,0.27,0.29,0.18,0.12,0.18,0.25,0.30,0.18,0.13,0.20,0.28,0.31,0.19,0.14,0.21,0.29,0.31,0.20,0.14,0.21,0.28,0.31,0.20,0.14,0.21,0.28,0.31,0.20,0.14,0.20,0.27,0.32,0.20,0.14,0.21,0.29,0.32,0.20,0.14,0.21,0.29,0.30,0.18,0.13,0.19,0.26,0.30,0.18,0.13,0.19,0.26,0.30,0.18,0.13,0.19,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2445,WFRC_8215,0.13,0.09,0.06,0.10,0.14,0.14,0.08,0.06,0.10,0.14,0.13,0.08,0.06,0.10,0.13,0.13,0.08,0.06,0.10,0.13,0.13,0.08,0.06,0.10,0.13,0.12,0.08,0.05,0.09,0.12,0.13,0.08,0.06,0.10,0.14,0.13,0.08,0.06,0.10,0.14,0.14,0.09,0.06,0.10,0.14,0.13,0.08,0.06,0.11,0.14,0.13,0.09,0.06,0.10,0.13,0.14,0.09,0.06,0.11,0.14,0.14,0.09,0.06,0.11,0.14,0.13,0.08,0.06,0.10,0.13,0.13,0.08,0.06,0.10,0.13,0.13,0.08,0.06,0.10,0.13
2446,WFRC_8216,0.20,0.06,0.05,0.19,0.24,0.21,0.06,0.04,0.20,0.25,0.18,0.06,0.04,0.18,0.22,0.19,0.06,0.04,0.18,0.23,0.19,0.06,0.04,0.18,0.23,0.18,0.06,0.04,0.17,0.22,0.19,0.06,0.04,0.19,0.24,0.20,0.06,0.04,0.20,0.25,0.20,0.06,0.04,0.19,0.24,0.20,0.06,0.04,0.19,0.24,0.19,0.06,0.04,0.19,0.23,0.20,0.06,0.04,0.19,0.24,0.21,0.06,0.04,0.19,0.24,0.19,0.06,0.04,0.18,0.23,0.19,0.06,0.04,0.18,0.23,0.19,0.06,0.04,0.18,0.23
2447,WFRC_8217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

VCGroupCode,SEGID,AprFrPM,AprSaMD,AprSuMD,AprWkAM,AprWkPM,AugFrPM,AugSaMD,AugSuMD,AugWkAM,AugWkPM,DecFrPM,DecSaMD,DecSuMD,DecWkAM,DecWkPM,FalFrPM,FalSaMD,FalSuMD,FalWkAM,FalWkPM,FebFrPM,FebSaMD,FebSuMD,FebWkAM,FebWkPM,JanFrPM,JanSaMD,JanSuMD,JanWkAM,JanWkPM,JulFrPM,JulSaMD,JulSuMD,JulWkAM,JulWkPM,JunFrPM,JunSaMD,JunSuMD,JunWkAM,JunWkPM,MarFrPM,MarSaMD,MarSuMD,MarWkAM,MarWkPM,MayFrPM,MaySaMD,MaySuMD,MayWkAM,MayWkPM,NovFrPM,NovSaMD,NovSuMD,NovWkAM,NovWkPM,OctFrPM,OctSaMD,OctSuMD,OctWkAM,OctWkPM,SepFrPM,SepSaMD,SepSuMD,SepWkAM,SepWkPM,SprFrPM,SprSaMD,SprSuMD,SprWkAM,SprWkPM,SumFrPM,SumSaMD,SumSuMD,SumWkAM,SumWkPM,WinFrPM,WinSaMD,WinSuMD,WinWkAM,WinWkPM
0,0013_000.0,0.38,0.24,0.17,0.27,0.38,0.38,0.23,0.16,0.27,0.40,0.36,0.23,0.16,0.25,0.36,0.36,0.22,0.16,0.25,0.35,0.37,0.23,0.16,0.25,0.36,0.35,0.21,0.15,0.23,0.34,0.37,0.22,0.16,0.26,0.39,0.38,0.23,0.17,0.26,0.40,0.38,0.24,0.17,0.26,0.38,0.38,0.24,0.17,0.27,0.39,0.38,0.24,0.16,0.26,0.37,0.39,0.24,0.17,0.27,0.39,0.39,0.24,0.17,0.27,0.39,0.36,0.22,0.16,0.25,0.35,0.36,0.22,0.16,0.25,0.35,0.36,0.22,0.16,0.25,0.35
1,0013_000.6,0.36,0.23,0.16,0.26,0.35,0.37,0.22,0.16,0.26,0.37,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33,0.36,0.22,0.15,0.24,0.33,0.34,0.21,0.14,0.22,0.31,0.35,0.22,0.15,0.25,0.36,0.37,0.22,0.16,0.25,0.37,0.37,0.23,0.16,0.25,0.35,0.37,0.23,0.16,0.26,0.36,0.36,0.23,0.16,0.25,0.34,0.38,0.24,0.16,0.26,0.36,0.38,0.24,0.16,0.26,0.36,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33
2,0013_001.3,0.31,0.23,0.16,0.22,0.29,0.31,0.22,0.15,0.22,0.29,0.30,0.23,0.16,0.22,0.28,0.30,0.23,0.16,0.21,0.28,0.31,0.23,0.16,0.22,0.28,0.30,0.22,0.15,0.21,0.27,0.30,0.22,0.14,0.22,0.29,0.30,0.22,0.15,0.22,0.30,0.32,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.22,0.28,0.31,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.22,0.29,0.30,0.23,0.16,0.21,0.28,0.30,0.23,0.16,0.21,0.28,0.30,0.23,0.16,0.21,0.28
3,0013_001.6,0.31,0.23,0.16,0.21,0.28,0.30,0.22,0.15,0.20,0.28,0.30,0.23,0.16,0.20,0.27,0.30,0.23,0.16,0.20,0.26,0.31,0.23,0.16,0.20,0.27,0.30,0.22,0.15,0.19,0.25,0.29,0.21,0.14,0.20,0.27,0.30,0.22,0.15,0.20,0.28,0.31,0.23,0.16,0.21,0.28,0.31,0.23,0.16,0.20,0.28,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.21,0.28,0.30,0.23,0.16,0.20,0.26,0.30,0.23,0.16,0.20,0.26,0.30,0.23,0.16,0.20,0.26
4,0013_001.8,0.32,0.21,0.15,0.21,0.28,0.33,0.20,0.14,0.21,0.30,0.31,0.19,0.13,0.20,0.27,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.14,0.20,0.27,0.30,0.18,0.13,0.18,0.25,0.31,0.19,0.14,0.20,0.29,0.32,0.20,0.14,0.21,0.30,0.32,0.21,0.15,0.21,0.28,0.32,0.20,0.14,0.21,0.29,0.32,0.20,0.14,0.20,0.27,0.33,0.21,0.15,0.21,0.29,0.33,0.21,0.14,0.22,0.29,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.13,0.19,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2449,WFRC_8215,0.20,0.13,0.09,0.17,0.21,0.21,0.13,0.09,0.17,0.22,0.20,0.12,0.09,0.16,0.20,0.20,0.12,0.08,0.16,0.20,0.20,0.12,0.09,0.16,0.20,0.19,0.12,0.08,0.15,0.19,0.20,0.12,0.09,0.16,0.22,0.21,0.13,0.09,0.17,0.22,0.21,0.13,0.09,0.17,0.21,0.21,0.13,0.09,0.17,0.22,0.20,0.13,0.09,0.16,0.21,0.21,0.13,0.09,0.17,0.22,0.21,0.13,0.09,0.17,0.22,0.20,0.12,0.08,0.16,0.20,0.20,0.12,0.08,0.16,0.20,0.20,0.12,0.08,0.16,0.20
2450,WFRC_8216,0.28,0.09,0.07,0.26,0.34,0.30,0.09,0.06,0.27,0.35,0.26,0.08,0.06,0.24,0.32,0.26,0.08,0.06,0.24,0.32,0.27,0.08,0.06,0.25,0.32,0.26,0.08,0.06,0.24,0.32,0.28,0.08,0.06,0.27,0.34,0.29,0.09,0.06,0.27,0.35,0.28,0.09,0.06,0.26,0.34,0.28,0.09,0.06,0.26,0.34,0.27,0.09,0.06,0.26,0.33,0.29,0.09,0.06,0.26,0.34,0.30,0.09,0.06,0.27,0.34,0.26,0.08,0.06,0.24,0.32,0.26,0.08,0.06,0.24,0.32,0.26,0.08,0.06,0.24,0.32
2451,WFRC_8217,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,

VCGroupCode,SEGID,AprFrPM,AprSaMD,AprSuMD,AprWkAM,AprWkPM,AugFrPM,AugSaMD,AugSuMD,AugWkAM,AugWkPM,DecFrPM,DecSaMD,DecSuMD,DecWkAM,DecWkPM,FalFrPM,FalSaMD,FalSuMD,FalWkAM,FalWkPM,FebFrPM,FebSaMD,FebSuMD,FebWkAM,FebWkPM,JanFrPM,JanSaMD,JanSuMD,JanWkAM,JanWkPM,JulFrPM,JulSaMD,JulSuMD,JulWkAM,JulWkPM,JunFrPM,JunSaMD,JunSuMD,JunWkAM,JunWkPM,MarFrPM,MarSaMD,MarSuMD,MarWkAM,MarWkPM,MayFrPM,MaySaMD,MaySuMD,MayWkAM,MayWkPM,NovFrPM,NovSaMD,NovSuMD,NovWkAM,NovWkPM,OctFrPM,OctSaMD,OctSuMD,OctWkAM,OctWkPM,SepFrPM,SepSaMD,SepSuMD,SepWkAM,SepWkPM,SprFrPM,SprSaMD,SprSuMD,SprWkAM,SprWkPM,SumFrPM,SumSaMD,SumSuMD,SumWkAM,SumWkPM,WinFrPM,WinSaMD,WinSuMD,WinWkAM,WinWkPM
0,0013_000.0,0.38,0.24,0.17,0.26,0.39,0.38,0.23,0.16,0.26,0.41,0.36,0.23,0.16,0.25,0.37,0.36,0.22,0.16,0.24,0.36,0.37,0.23,0.16,0.25,0.37,0.35,0.21,0.15,0.23,0.35,0.37,0.22,0.16,0.25,0.40,0.38,0.23,0.17,0.26,0.41,0.38,0.24,0.17,0.26,0.39,0.38,0.24,0.17,0.26,0.40,0.38,0.24,0.16,0.25,0.38,0.39,0.24,0.17,0.26,0.40,0.39,0.24,0.17,0.27,0.41,0.36,0.22,0.16,0.24,0.36,0.36,0.22,0.16,0.24,0.36,0.36,0.22,0.16,0.24,0.36
1,0013_000.6,0.36,0.23,0.17,0.26,0.35,0.37,0.22,0.16,0.26,0.37,0.35,0.22,0.15,0.25,0.33,0.35,0.22,0.15,0.24,0.33,0.36,0.22,0.16,0.25,0.33,0.34,0.21,0.14,0.23,0.31,0.35,0.22,0.15,0.25,0.36,0.37,0.22,0.16,0.26,0.37,0.37,0.23,0.17,0.26,0.35,0.37,0.23,0.16,0.26,0.36,0.36,0.23,0.16,0.25,0.34,0.38,0.24,0.16,0.27,0.36,0.38,0.24,0.16,0.27,0.36,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33
2,0013_001.3,0.31,0.23,0.16,0.23,0.29,0.31,0.22,0.15,0.22,0.29,0.30,0.23,0.16,0.22,0.28,0.30,0.23,0.16,0.21,0.27,0.31,0.23,0.16,0.22,0.28,0.30,0.22,0.15,0.21,0.27,0.30,0.22,0.14,0.22,0.29,0.30,0.22,0.15,0.22,0.29,0.32,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.22,0.28,0.31,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.23,0.29,0.30,0.23,0.16,0.21,0.27,0.30,0.23,0.16,0.21,0.27,0.30,0.23,0.16,0.21,0.27
3,0013_001.6,0.31,0.23,0.16,0.20,0.27,0.30,0.22,0.15,0.20,0.27,0.30,0.23,0.16,0.19,0.26,0.30,0.23,0.16,0.19,0.26,0.31,0.23,0.16,0.20,0.26,0.30,0.22,0.15,0.19,0.25,0.30,0.21,0.14,0.19,0.27,0.30,0.22,0.15,0.20,0.28,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.20,0.26,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.20,0.27,0.30,0.23,0.16,0.19,0.26,0.30,0.23,0.16,0.19,0.26,0.30,0.23,0.16,0.19,0.26
4,0013_001.8,0.32,0.21,0.15,0.21,0.28,0.33,0.20,0.14,0.21,0.30,0.31,0.19,0.13,0.19,0.27,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.14,0.19,0.27,0.30,0.18,0.13,0.18,0.25,0.31,0.19,0.14,0.20,0.29,0.32,0.20,0.14,0.20,0.30,0.32,0.21,0.15,0.20,0.28,0.32,0.20,0.15,0.21,0.29,0.32,0.20,0.14,0.20,0.28,0.33,0.21,0.15,0.21,0.29,0.33,0.21,0.14,0.21,0.30,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.13,0.19,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,WFRC_8215,0.27,0.17,0.12,0.22,0.28,0.27,0.16,0.12,0.22,0.29,0.26,0.16,0.11,0.21,0.26,0.26,0.16,0.11,0.21,0.26,0.26,0.16,0.11,0.21,0.26,0.25,0.15,0.11,0.20,0.24,0.26,0.16,0.11,0.21,0.28,0.27,0.16,0.12,0.22,0.29,0.27,0.17,0.12,0.22,0.27,0.27,0.17,0.12,0.22,0.28,0.27,0.17,0.12,0.21,0.27,0.28,0.17,0.12,0.22,0.28,0.28,0.17,0.12,0.23,0.29,0.26,0.16,0.11,0.21,0.26,0.26,0.16,0.11,0.21,0.26,0.26,0.16,0.11,0.21,0.26
2493,WFRC_8216,0.31,0.10,0.07,0.28,0.37,0.33,0.10,0.07,0.29,0.38,0.29,0.09,0.07,0.26,0.35,0.30,0.09,0.07,0.26,0.35,0.30,0.09,0.07,0.26,0.35,0.29,0.09,0.07,0.25,0.35,0.31,0.09,0.07,0.28,0.37,0.32,0.10,0.07,0.29,0.38,0.31,0.10,0.07,0.27,0.37,0.32,0.10,0.07,0.28,0.37,0.31,0.10,0.07,0.27,0.36,0.33,0.10,0.07,0.28,0.37,0.33,0.10,0.07,0.28,0.38,0.30,0.09,0.07,0.26,0.35,0.30,0.09,0.07,0.26,0.35,0.30,0.09,0.07,0.26,0.35
2494,WFRC_8217,0.87,0.56,0.40,0.58,0.72,0.88,0.54,0.38,0.58,0.75,0.84,0.53,0.36,0.55,0.68,0.84,0.52,0.36,0.54,0.67,0.85

In [210]:
sdf_Forecasts_wVC = pd.DataFrame.merge(sdf_Forecasts,dfs_Peak_VC_MaxDirFG_wVCGroups[idFromCode('NB50')],on='SEGID',how='left')
sFieldName = 'SumWkPM'

#create map centered on Salt Lake
map_seg = gis.map('Salt Lake')

#expression to classify enrollment
arcade_expression = ("var v = $feature." + sFieldName + ";"
                     "if (v<0.5)      { return 'class1'; }"
                     "else if (v<0.7) { return 'class2'; }"
                     "else if (v<0.9) { return 'class3'; }"
                     "else if (v<1.0) { return 'class4'; }"
                     "else if (v<1.5) { return 'class5'; }"
                     "else if (v<2.0) { return 'class6'; }"
                     "else            { return 'class7'; }")

#symbology for enrollment classes
uv = [{"value":"class1", "label":"Less than 0.5", "symbol":{"type":"esriSLS","color":[  0, 97,  0], "width":0.5,"style":"esriSLSSolid"}},
      {"value":"class2", "label":"0.5 to 0.7"   , "symbol":{"type":"esriSLS","color":[ 85,145,  0], "width":1.0,"style":"esriSLSSolid"}},
      {"value":"class3", "label":"0.7 to 0.9"   , "symbol":{"type":"esriSLS","color":[255,255,  0], "width":2.0,"style":"esriSLSSolid"}},
      {"value":"class4", "label":"0.9 to 1.0"   , "symbol":{"type":"esriSLS","color":[255,170,  0], "width":4.0,"style":"esriSLSSolid"}},
      {"value":"class5", "label":"1.0 to 1.5"   , "symbol":{"type":"esriSLS","color":[255,  0,  0], "width":6.0,"style":"esriSLSSolid"}},
      {"value":"class6", "label":"1.5 to 2.0"   , "symbol":{"type":"esriSLS","color":[115,  0,  0], "width":8.0,"style":"esriSLSSolid"}},
      {"value":"class7", "label":"More than 2.0", "symbol":{"type":"esriSLS","color":[  0,  0,  0], "width":10.0,"style":"esriSLSSolid"}}]

#define sdf layer
sdf_Forecasts_wVC.spatial.plot(map_widget = map_seg,
                               renderer_type='u-a', #'u-a' stands for uniqe value with arcade expression
                               unique_values=uv,
                               arcade_expression=arcade_expression,
                               default_symbol="" #don't include an 'other' category
                               )

#define map characteristics
map_seg.layout.height='500px'
map_seg.legend=True

#map title
#display(Markdown('<h2><center>Public College Students by Census Tract</center></h2>'))

#display map
map_seg

MapView(layout=Layout(height='500px', width='100%'), legend=True)

# Export CSV

In [191]:
df_Scenarios_toProcess

,Process,ScenarioCode,ScenarioName,ScenarioYear,SegSummaryDBF,AdjScenarioCode
ScenarioID,,,,,,
0,Yes,Base19,Base,2019,v831_SE19_Net19_Summary_SEGID_Detailed_BH,NaN
1,Yes,Base24,STIP,2024,v831_SE24_Net24_Summary_SEGID_Detailed_BH,NaN
2,Yes,RTP30,2030 RTP,2030,v831_SE30_Net30_Summary_SEGID_Detailed_BH,NaN
3,Yes,RTP40,2040 RTP,2040,v831_SE40_Net40_Summary_SEGID_Detailed_BH,NaN
4,Yes,RTP50,2050 RTP,2050,v831_SE50_Net50_Summary_SEGID_Detailed_BH,NaN
5,Yes,NB30,2030 No Build,2030,v831_SE30_Net24_Summary_SEGID_Detailed_BH,RTP30
6,Yes,NB40,2040 No Build,2040,v831_SE40_Net24_Summary_SEGID_Detailed_BH,RTP40
7,Yes,NB50,2050 No Build,2050,v831_SE50_Net24_Summary_SEGID_Detailed_BH,RTP50
8,Yes,UF30,2030 Unfunded,2030,v831_SE30_Net30UF_Summary_SEGID_Detailed_BH,RTP30


In [192]:
df_Scenarios_toExport = df_Scenarios_toProcess.reset_index()

for index, df_Peak_VC_MaxDirFG_wVCGroups in enumerate(dfs_Peak_VC_MaxDirFG_wVCGroups):    
    
    filename = "VC_" + df_Scenarios_toExport['ScenarioCode'].iloc[index] + '.json'
    #display(filename)
    
    #df_Peak_VC_MaxDirFG_wVCGroups = df_Peak_VC_MaxDirFG_wVCGroups.rename(columns={"SEGID": 'S'})
    
    #export to csv in data-processed
    df_Peak_VC_MaxDirFG_wVCGroups.to_json(os.path.join(dirResults, filename),orient='records')
    
    display('Done exporting ' + filename)

'Done exporting VC_Base19.json'

'Done exporting VC_Base24.json'

'Done exporting VC_RTP30.json'

'Done exporting VC_RTP40.json'

'Done exporting VC_RTP50.json'

...

'Done exporting VC_NB40.json'

'Done exporting VC_NB50.json'

'Done exporting VC_UF30.json'

'Done exporting VC_UF40.json'

'Done exporting VC_UF50.json'

In [193]:
df_Peak_VC_MaxDirFG_wVCGroups

VCGroupCode,SEGID,AprFrPM,AprSaMD,AprSuMD,AprWkAM,AprWkPM,AugFrPM,AugSaMD,AugSuMD,AugWkAM,AugWkPM,DecFrPM,DecSaMD,DecSuMD,DecWkAM,DecWkPM,FalFrPM,FalSaMD,FalSuMD,FalWkAM,FalWkPM,FebFrPM,FebSaMD,FebSuMD,FebWkAM,FebWkPM,JanFrPM,JanSaMD,JanSuMD,JanWkAM,JanWkPM,JulFrPM,JulSaMD,JulSuMD,JulWkAM,JulWkPM,JunFrPM,JunSaMD,JunSuMD,JunWkAM,JunWkPM,MarFrPM,MarSaMD,MarSuMD,MarWkAM,MarWkPM,MayFrPM,MaySaMD,MaySuMD,MayWkAM,MayWkPM,NovFrPM,NovSaMD,NovSuMD,NovWkAM,NovWkPM,OctFrPM,OctSaMD,OctSuMD,OctWkAM,OctWkPM,SepFrPM,SepSaMD,SepSuMD,SepWkAM,SepWkPM,SprFrPM,SprSaMD,SprSuMD,SprWkAM,SprWkPM,SumFrPM,SumSaMD,SumSuMD,SumWkAM,SumWkPM,WinFrPM,WinSaMD,WinSuMD,WinWkAM,WinWkPM
0,0013_000.0,0.38,0.24,0.17,0.26,0.39,0.38,0.23,0.16,0.26,0.41,0.36,0.23,0.16,0.25,0.37,0.36,0.22,0.16,0.24,0.36,0.37,0.23,0.16,0.25,0.37,0.35,0.21,0.15,0.23,0.35,0.37,0.22,0.16,0.25,0.40,0.38,0.23,0.17,0.26,0.41,0.38,0.24,0.17,0.26,0.39,0.38,0.24,0.17,0.26,0.40,0.38,0.24,0.16,0.25,0.38,0.39,0.24,0.17,0.26,0.40,0.39,0.24,0.17,0.27,0.41,0.36,0.22,0.16,0.24,0.36,0.36,0.22,0.16,0.24,0.36,0.36,0.22,0.16,0.24,0.36
1,0013_000.6,0.36,0.23,0.17,0.26,0.35,0.37,0.22,0.16,0.26,0.37,0.35,0.22,0.15,0.25,0.33,0.35,0.22,0.15,0.24,0.33,0.36,0.22,0.16,0.25,0.33,0.34,0.21,0.14,0.23,0.31,0.35,0.22,0.15,0.25,0.36,0.37,0.22,0.16,0.26,0.37,0.37,0.23,0.17,0.26,0.35,0.37,0.23,0.16,0.26,0.36,0.36,0.23,0.16,0.25,0.34,0.38,0.24,0.16,0.27,0.36,0.38,0.24,0.16,0.27,0.36,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33,0.35,0.22,0.15,0.24,0.33
2,0013_001.3,0.31,0.23,0.16,0.23,0.29,0.31,0.22,0.15,0.22,0.29,0.30,0.23,0.16,0.22,0.28,0.30,0.23,0.16,0.21,0.27,0.31,0.23,0.16,0.22,0.28,0.30,0.22,0.15,0.21,0.27,0.30,0.22,0.14,0.22,0.29,0.30,0.22,0.15,0.22,0.29,0.32,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.22,0.28,0.31,0.23,0.16,0.22,0.29,0.31,0.23,0.16,0.23,0.29,0.30,0.23,0.16,0.21,0.27,0.30,0.23,0.16,0.21,0.27,0.30,0.23,0.16,0.21,0.27
3,0013_001.6,0.31,0.23,0.16,0.20,0.27,0.30,0.22,0.15,0.20,0.27,0.30,0.23,0.16,0.19,0.26,0.30,0.23,0.16,0.19,0.26,0.31,0.23,0.16,0.20,0.26,0.30,0.22,0.15,0.19,0.25,0.30,0.21,0.14,0.19,0.27,0.30,0.22,0.15,0.20,0.28,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.20,0.26,0.31,0.23,0.16,0.20,0.27,0.31,0.23,0.16,0.20,0.27,0.30,0.23,0.16,0.19,0.26,0.30,0.23,0.16,0.19,0.26,0.30,0.23,0.16,0.19,0.26
4,0013_001.8,0.32,0.21,0.15,0.21,0.28,0.33,0.20,0.14,0.21,0.30,0.31,0.19,0.13,0.19,0.27,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.14,0.19,0.27,0.30,0.18,0.13,0.18,0.25,0.31,0.19,0.14,0.20,0.29,0.32,0.20,0.14,0.20,0.30,0.32,0.21,0.15,0.20,0.28,0.32,0.20,0.15,0.21,0.29,0.32,0.20,0.14,0.20,0.28,0.33,0.21,0.15,0.21,0.29,0.33,0.21,0.14,0.21,0.30,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.13,0.19,0.26,0.31,0.19,0.13,0.19,0.26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2492,WFRC_8215,0.27,0.17,0.12,0.22,0.28,0.27,0.16,0.12,0.22,0.29,0.26,0.16,0.11,0.21,0.26,0.26,0.16,0.11,0.21,0.26,0.26,0.16,0.11,0.21,0.26,0.25,0.15,0.11,0.20,0.24,0.26,0.16,0.11,0.21,0.28,0.27,0.16,0.12,0.22,0.29,0.27,0.17,0.12,0.22,0.27,0.27,0.17,0.12,0.22,0.28,0.27,0.17,0.12,0.21,0.27,0.28,0.17,0.12,0.22,0.28,0.28,0.17,0.12,0.23,0.29,0.26,0.16,0.11,0.21,0.26,0.26,0.16,0.11,0.21,0.26,0.26,0.16,0.11,0.21,0.26
2493,WFRC_8216,0.31,0.10,0.07,0.28,0.37,0.33,0.10,0.07,0.29,0.38,0.29,0.09,0.07,0.26,0.35,0.30,0.09,0.07,0.26,0.35,0.30,0.09,0.07,0.26,0.35,0.29,0.09,0.07,0.25,0.35,0.31,0.09,0.07,0.28,0.37,0.32,0.10,0.07,0.29,0.38,0.31,0.10,0.07,0.27,0.37,0.32,0.10,0.07,0.28,0.37,0.31,0.10,0.07,0.27,0.36,0.33,0.10,0.07,0.28,0.37,0.33,0.10,0.07,0.28,0.38,0.30,0.09,0.07,0.26,0.35,0.30,0.09,0.07,0.26,0.35,0.30,0.09,0.07,0.26,0.35
2494,WFRC_8217,0.87,0.56,0.40,0.58,0.72,0.88,0.54,0.38,0.58,0.75,0.84,0.53,0.36,0.55,0.68,0.84,0.52,0.36,0.54,0.67,0.85